In [3]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import clip
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm

# ------------------------ #
# 1. 设备设置
# ------------------------ #

# 选择设备（GPU或CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")

# ------------------------ #
# 2. 加载CLIP模型
# ------------------------ #

# 加载CLIP模型和预处理方法
clip_model, preprocess = clip.load("ViT-B/32", device=device)
clip_model.eval()
print("CLIP模型已加载。")

# ------------------------ #
# 3. 定义CelebA数据集类
# ------------------------ #

class CelebADataset(Dataset):
    def __init__(self, img_dir, attr_path, bbox_path, partition_path, 
                 image_embeddings_path, transform=None, partition=0):
        """
        初始化数据集
        :param img_dir: 图像文件夹路径
        :param attr_path: 属性文件路径
        :param bbox_path: 边界框文件路径
        :param partition_path: 分区文件路径
        :param image_embeddings_path: 预先计算的图像嵌入文件路径
        :param transform: 图像预处理
        :param partition: 使用的数据分区 (0: train, 1: val, 2: test)
        """
        self.img_dir = img_dir
        self.transform = transform

        # 读取属性文件
        try:
            attr_df = pd.read_csv(attr_path, sep=',', header=0)
        except Exception as e:
            print(f"Error reading {attr_path}: {e}")
            raise

        print("Attributes DataFrame columns:", attr_df.columns.tolist())
        print("Attributes DataFrame head:\n", attr_df.head())

        # 确认 'image_id' 列存在
        if 'image_id' not in attr_df.columns:
            raise ValueError("属性文件中缺少 'image_id' 列。请检查文件格式。")
        else:
            print("'image_id' column found in attributes DataFrame.")

        # 读取分区文件
        try:
            partition_df = pd.read_csv(partition_path, sep=',', header=0, names=['image_id', 'partition'])
        except Exception as e:
            print(f"Error reading {partition_path}: {e}")
            raise

        print("Partition DataFrame columns:", partition_df.columns.tolist())
        print("Partition DataFrame head:\n", partition_df.head())

        # 读取边界框文件
        try:
            bbox_df = pd.read_csv(bbox_path, sep=',', header=0, names=['image_id', 'x_1', 'y_1', 'width', 'height'])
        except Exception as e:
            print(f"Error reading {bbox_path}: {e}")
            raise

        print("Bounding Box DataFrame columns:", bbox_df.columns.tolist())
        print("Bounding Box DataFrame head:\n", bbox_df.head())

        # 合并属性和分区数据
        try:
            attr_df = attr_df.merge(partition_df, on='image_id')
        except Exception as e:
            print(f"Error merging attributes and partitions: {e}")
            raise

        print("After merging with partition, columns:", attr_df.columns.tolist())
        print("Merged DataFrame head:\n", attr_df.head())

        # 合并属性、分区和边界框数据
        try:
            attr_df = attr_df.merge(bbox_df, on='image_id')
        except Exception as e:
            print(f"Error merging attributes, partitions and bounding boxes: {e}")
            raise

        print("After merging with bounding box, columns:", attr_df.columns.tolist())
        print("Final Merged DataFrame head:\n", attr_df.head())

        # 筛选分区
        print(f"Total samples before filtering: {len(attr_df)}")
        self.attr_df = attr_df[attr_df['partition'] == partition].reset_index(drop=True)
        print(f"Total samples after filtering partition {partition}: {len(self.attr_df)}")

        if len(self.attr_df) == 0:
            raise ValueError(f"训练集中没有样本。请检查数据文件的读取和过滤逻辑。")

        # 获取属性名称
        # 属性列是除了 'image_id', 'partition', 'x_1', 'y_1', 'width', 'height' 之外的所有列
        self.attr_names = [col for col in attr_df.columns if col not in ['image_id', 'partition', 'x_1', 'y_1', 'width', 'height']]
        print("Attribute names:", self.attr_names)

        # 加载预计算的图像嵌入
        self.image_embeddings = torch.load(image_embeddings_path)  # 假设是一个Tensor，shape=(num_images, 512)
        if len(self.image_embeddings) != len(self.attr_df):
            raise ValueError("图像嵌入的数量与数据集中的图像数量不一致。")
        self.image_embeddings = self.image_embeddings.float()

    def __len__(self):
        return len(self.attr_df)

    def __getitem__(self, idx):
        # 获取图像路径
        img_name = self.attr_df.iloc[idx, self.attr_df.columns.get_loc('image_id')]
        img_path = os.path.join(self.img_dir, img_name)
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Error opening image {img_path}: {e}")
            # 可以选择跳过该样本，或返回一个默认图像
            # 这里选择抛出异常
            raise

        if self.transform:
            image = self.transform(image)

        # 获取属性标签 
        attrs = self.attr_df.iloc[idx][self.attr_names].values
        attrs = (attrs + 1) // 2  # 将-1转为0，1保持1
        attrs = attrs.astype(np.float32)

        # 获取对应的图像嵌入
        clip_embedding = self.image_embeddings[idx]

        return image, attrs, clip_embedding

# ------------------------ #
# 4. 定义ClipCVAE模型（使用Renyi散度）
# ------------------------ #

class ClipCVAE_Renyi(nn.Module):
    def __init__(self, img_channels=3, img_size=64, latent_dim=128, 
                 cond_dim=40, clip_dim=512):
        super(ClipCVAE_Renyi, self).__init__()
        self.img_size = img_size
        self.latent_dim = latent_dim
        self.cond_dim = cond_dim
        self.clip_dim = clip_dim

        # 编码器部分
        self.encoder = nn.Sequential(
            nn.Conv2d(img_channels + cond_dim + clip_dim, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Flatten()
        )
        self.fc_mu = nn.Linear(512*4*4, latent_dim)
        self.fc_logvar = nn.Linear(512*4*4, latent_dim)

        # 解码器部分
        self.decoder_input = nn.Linear(latent_dim + cond_dim + clip_dim, 512*4*4)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, img_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def encode(self, x, c, clip_embedding):
        # 将条件标签、图像和CLIP嵌入拼接
        c = c.view(c.size(0), self.cond_dim, 1, 1).repeat(1, 1, self.img_size, self.img_size)
        clip_embedding = clip_embedding.view(clip_embedding.size(0), self.clip_dim, 1, 1).repeat(1, 1, self.img_size, self.img_size)
        x = torch.cat([x, c, clip_embedding], dim=1)
        x = self.encoder(x)
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z, c, clip_embedding):
        # 将潜在变量、条件标签和CLIP嵌入拼接
        z = torch.cat([z, c, clip_embedding], dim=1)
        x = self.decoder_input(z)
        x = x.view(-1, 512, 4, 4)
        x = self.decoder(x)
        return x

    def forward(self, x, c, clip_embedding):
        mu, logvar = self.encode(x, c, clip_embedding)
        z = self.reparameterize(mu, logvar)
        recon_x = self.decode(z, c, clip_embedding)
        return recon_x, mu, logvar

# ------------------------ #
# 5. 图像预处理
# ------------------------ #

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # 修正归一化
])

# ------------------------ #
# 6. 数据集路径设置
# ------------------------ #

# 请根据实际路径修改
img_dir = '/root/autodl-tmp/celeba_datasets/img_align_celeba/img_align_celeba'
attr_path = '/root/autodl-tmp/celeba_datasets/list_attr_celeba.txt'
bbox_path = '/root/autodl-tmp/celeba_datasets/list_bbox_celeba.txt'
partition_path = '/root/autodl-tmp/celeba_datasets/list_eval_partition.txt'
image_embeddings_train_path = '/root/autodl-tmp/clip_embeddings/image_embeddings_partition_0.pt'
image_embeddings_val_path = '/root/autodl-tmp/clip_embeddings/image_embeddings_partition_1.pt'

# ------------------------ #
# 7. 创建训练集和验证集
# ------------------------ #

train_dataset = CelebADataset(img_dir, attr_path, bbox_path, partition_path, 
                              image_embeddings_train_path, transform=transform, partition=0)
val_dataset = CelebADataset(img_dir, attr_path, bbox_path, partition_path, 
                            image_embeddings_val_path, transform=transform, partition=1)

# ------------------------ #
# 8. 数据加载器
# ------------------------ #

batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# ------------------------ #
# 9. 初始化模型、损失函数和优化器
# ------------------------ #

latent_dim = 128
cond_dim = 40
clip_dim = 512
model_renyi = ClipCVAE_Renyi(img_channels=3, img_size=64, latent_dim=latent_dim, 
                           cond_dim=cond_dim, clip_dim=clip_dim).to(device)

# 使用Adam优化器并添加权重衰减
optimizer = optim.Adam(model_renyi.parameters(), lr=1e-4, weight_decay=1e-5)  # 降低学习率

# 学习率调度器: 每5个epoch降低学习率
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)  # 更频繁地降低学习率

criterion = nn.MSELoss(reduction='sum')

# ------------------------ #
# 10. 定义改进的Renyi散度损失函数
# ------------------------ #

def improved_renyi_divergence_loss(recon_x, x, mu, logvar, alpha=0.9):
    """
    使用数值稳定版本的Renyi散度作为VAE的损失函数
    :param recon_x: 重构的图像
    :param x: 原始图像
    :param mu: 均值
    :param logvar: 对数方差
    :param alpha: Renyi散度参数, 注意：不使用alpha=0.5
    :return: 总损失
    """
    # 重构损失
    recon_loss = criterion(recon_x, x)
    
    # 严格限制对数方差范围以提高数值稳定性
    logvar = torch.clamp(logvar, min=-10.0, max=2.0)
    
    # 计算方差 (增加下限避免除零)
    var = torch.exp(logvar).clamp(min=1e-5, max=100.0)
    
    # 标准正态分布的参数
    mu0 = torch.zeros_like(mu)
    var0 = torch.ones_like(var)
    
    # 计算均值差平方
    mu_diff_sq = (mu - mu0).pow(2)
    
    # 使用对数空间计算某些项，避免数值溢出
    log_det_ratio = torch.log(var) - torch.log(var0)
    
    if alpha == 1.0:
        # 当alpha=1时，使用KL散度公式（但通常应避免使用alpha=1）
        renyi_div = 0.5 * (mu_diff_sq / var + var / var0 - 1 - log_det_ratio).sum()
    elif alpha == 0.5:
        # 使用更直接的计算方式实现Hellinger距离
        # 避免直接使用这个公式，可能导致不稳定
        renyi_div = torch.sum(2.0 * (1.0 - torch.exp(-0.25 * mu_diff_sq / var)))
    else:
        # 使用对数求和exp的技巧，增强数值稳定性
        weight_var = alpha * var0 + (1 - alpha) * var
        weight_var = torch.clamp(weight_var, min=1e-5)  # 确保分母不为零
        
        # 按部分计算，避免大数相乘
        term1 = 0.5 * torch.log(weight_var)
        term2 = 0.5 * (alpha * (1 - alpha) * mu_diff_sq / weight_var)
        term3 = -0.5 * (alpha * torch.log(var0) + (1 - alpha) * torch.log(var))
        
        # 按元素计算，确保数值稳定
        per_element_div = (term1 + term2 + term3) / (alpha - 1 + 1e-10)
        
        # 求和得到总散度
        renyi_div = per_element_div.sum()
    
    # 应用权重因子使Renyi散度与重构损失平衡
    weight_factor = 0.01  # 更小的权重
    
    # 确保散度是非负的
    renyi_div = torch.abs(renyi_div) * weight_factor
    
    # 增加梯度保护，防止梯度爆炸
    renyi_div = torch.clamp(renyi_div, max=1000.0)
    
    # 总损失
    total_loss = recon_loss + renyi_div
    
    return total_loss, recon_loss, renyi_div

# ------------------------ #
# 11. 训练模型
# ------------------------ #

def train_model_renyi(model, train_loader, val_loader, optimizer, scheduler, num_epochs=30, alpha=0.9):
    """
    使用改进的Renyi散度训练CVAE模型
    """
    print(f"开始训练改进版Renyi散度CVAE (alpha={alpha})...")
    
    # 记录训练和验证损失
    train_losses = []
    val_losses = []
    
    # 初始化最小损失值（用于提前停止）
    best_val_loss = float('inf')
    patience = 5
    patience_counter = 0
    
    for epoch in range(1, num_epochs + 1):
        model.train()
        train_loss = 0
        train_recon_loss = 0
        train_renyi_loss = 0
        nan_batches = 0
        
        for batch_idx, (data, attrs, clip_emb) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}")):
            data = data.to(device, non_blocking=True)
            attrs = attrs.to(device, non_blocking=True)
            clip_emb = clip_emb.to(device, non_blocking=True)
            
            optimizer.zero_grad()
            
            # 使用try-except块处理可能的错误
            try:
                recon_batch, mu, logvar = model(data, attrs, clip_emb)
                
                # 计算损失
                loss, recon_loss, renyi_loss = improved_renyi_divergence_loss(
                    recon_batch, data, mu, logvar, alpha=alpha
                )
                
                # 检查损失是否为NaN
                if torch.isnan(loss).any() or torch.isinf(loss).any():
                    nan_batches += 1
                    if batch_idx % 50 == 0:  # 减少日志输出
                        print(f"警告: 第{epoch}轮, 第{batch_idx}批次出现NaN/Inf损失。跳过此批次.")
                    continue
                
                # 反向传播
                loss.backward()
                
                # 添加梯度裁剪，防止梯度爆炸
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.1)  # 更严格的梯度裁剪
                
                # 更新权重
                optimizer.step()
                
                # 累计损失
                train_loss += loss.item()
                train_recon_loss += recon_loss.item()
                train_renyi_loss += renyi_loss.item()
                
            except RuntimeError as e:
                print(f"批次{batch_idx}处理出错: {e}")
                continue
        
        # 更新学习率
        scheduler.step()
        current_lr = optimizer.param_groups[0]["lr"]
        
        # 计算平均损失（排除NaN批次）
        processed_batches = len(train_loader) - nan_batches
        if processed_batches > 0:
            avg_loss = train_loss / processed_batches
            avg_recon_loss = train_recon_loss / processed_batches
            avg_renyi_loss = train_renyi_loss / processed_batches
            train_losses.append(avg_loss)
            
            print(f"Epoch {epoch}, 平均训练损失: {avg_loss:.4f}, 重构损失: {avg_recon_loss:.4f}, "
                  f"Renyi损失: {avg_renyi_loss:.4f}, 学习率: {current_lr:.6f}, NaN批次: {nan_batches}")
        else:
            print(f"Epoch {epoch}, 所有批次均为NaN，无法计算平均损失")
            continue
        
        # 验证集评估
        model.eval()
        val_loss = 0
        val_recon_loss = 0
        val_renyi_loss = 0
        val_nan_batches = 0
        
        with torch.no_grad():
            for data, attrs, clip_emb in val_loader:
                data = data.to(device, non_blocking=True)
                attrs = attrs.to(device, non_blocking=True)
                clip_emb = clip_emb.to(device, non_blocking=True)
                
                try:
                    recon_batch, mu, logvar = model(data, attrs, clip_emb)
                    loss, recon_loss, renyi_loss = improved_renyi_divergence_loss(
                        recon_batch, data, mu, logvar, alpha=alpha
                    )
                    
                    if torch.isnan(loss).any() or torch.isinf(loss).any():
                        val_nan_batches += 1
                        continue
                    
                    val_loss += loss.item()
                    val_recon_loss += recon_loss.item()
                    val_renyi_loss += renyi_loss.item()
                    
                except RuntimeError:
                    val_nan_batches += 1
                    continue
        
        # 计算平均验证损失（排除NaN批次）
        processed_val_batches = len(val_loader) - val_nan_batches
        if processed_val_batches > 0:
            avg_val_loss = val_loss / processed_val_batches
            avg_val_recon_loss = val_recon_loss / processed_val_batches
            avg_val_renyi_loss = val_renyi_loss / processed_val_batches
            val_losses.append(avg_val_loss)
            
            print(f"验证集平均损失: {avg_val_loss:.4f}, 重构损失: {avg_val_recon_loss:.4f}, "
                  f"Renyi损失: {avg_val_renyi_loss:.4f}, NaN批次: {val_nan_batches}")
            
            # 提前停止检查
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                patience_counter = 0
                
                # 保存最佳模型
                best_model_path = f'clip_cvae_renyi_alpha{alpha}_best.pth'
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict(),
                    'val_loss': avg_val_loss,
                    'alpha': alpha
                }, best_model_path)
                print(f"保存最佳模型: {best_model_path}")
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print(f"提前停止训练！验证损失{patience}轮未改善。")
                    return model, train_losses, val_losses
        else:
            print(f"验证集所有批次均为NaN，无法计算平均损失")
        
        # 每5个epoch或最后一个epoch保存一次模型
        if epoch % 5 == 0 or epoch == num_epochs:
            checkpoint_path = f'clip_cvae_renyi_alpha{alpha}_epoch_{epoch}.pth'
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'train_loss': avg_loss,
                'val_loss': avg_val_loss if processed_val_batches > 0 else float('inf'),
                'alpha': alpha
            }, checkpoint_path)
            print(f"模型检查点已保存: {checkpoint_path}")
    
    return model, train_losses, val_losses

# ------------------------ #
# 12. 主程序
# ------------------------ #

if __name__ == "__main__":
    try:
        # 设置Renyi散度参数 - 避免使用0.5，使用0.9
        alpha = 0.9  # 数值更稳定的选择
        
        # 正确初始化模型权重 - 修复这里的错误
        for name, param in model_renyi.named_parameters():
            if 'weight' in name:
                if param.dim() >= 2:  # 只对至少二维的张量使用Xavier初始化
                    nn.init.xavier_normal_(param, gain=0.01)  # 使用较小的增益值
                else:  # 对一维张量使用小的常数初始化
                    nn.init.normal_(param, mean=0.0, std=0.01)
            elif 'bias' in name:
                nn.init.constant_(param, 0.0)
        
        print("已成功完成模型参数初始化")
        
        # 训练使用Renyi散度的CVAE模型
        trained_model, train_losses, val_losses = train_model_renyi(
            model_renyi, train_loader, val_loader, optimizer, scheduler, 
            num_epochs=30, alpha=alpha
        )
        
        # 保存最终模型
        final_model_path = f'clip_cvae_renyi_alpha{alpha}_final.pth'
        torch.save(trained_model.state_dict(), final_model_path)
        print(f"Renyi散度CVAE模型已保存为: {final_model_path}")
        
        # 打印训练完成信息
        print(f"使用Renyi散度(alpha={alpha})的CVAE模型训练完成！")
        
    except Exception as e:
        print(f"训练过程中发生错误: {e}")
        import traceback
        traceback.print_exc()
        # 在发生错误时尝试保存当前模型状态
        emergency_save_path = f'clip_cvae_renyi_alpha{alpha}_emergency_save.pth'
        torch.save(model_renyi.state_dict(), emergency_save_path)
        print(f"发生错误，模型紧急保存为: {emergency_save_path}")

使用设备: cuda
CLIP模型已加载。
Attributes DataFrame columns: ['image_id', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive', 'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose', 'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows', 'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace', 'Wearing_Necktie', 'Young']
Attributes DataFrame head:
      image_id  5_o_Clock_Shadow  Arched_Eyebrows  Attractive  Bags_Under_Eyes  \
0  000001.jpg                -1                1           1               -1   
1  000002.jpg                -1               -1          -1                1   
2  000003.jpg                -1               -1          -1               -1   

Epoch 1/30: 100%|██████████| 1272/1272 [01:01<00:00, 20.84it/s]

Epoch 1, 平均训练损失: 403032.6394, 重构损失: 403010.6445, Renyi损失: 21.9957, 学习率: 0.000100, NaN批次: 0


验证集平均损失: 343168.7825, 重构损失: 343092.5451, Renyi损失: 76.2455, NaN批次: 0
保存最佳模型: clip_cvae_renyi_alpha0.9_best.pth


Epoch 2/30: 100%|██████████| 1272/1272 [01:00<00:00, 21.05it/s]

...

Epoch 16, 平均训练损失: 208944.2284, 重构损失: 208707.7303, Renyi损失: 236.4980, 学习率: 0.000013, NaN批次: 0


验证集平均损失: 242261.7125, 重构损失: 242027.9804, Renyi损失: 233.7272, NaN批次: 0


Epoch 17/30: 100%|██████████| 1272/1272 [00:59<00:00, 21.26it/s]

Epoch 17, 平均训练损失: 207036.2312, 重构损失: 206803.2242, Renyi损失: 233.0071, 学习率: 0.000013, NaN批次: 0


验证集平均损失: 242855.4308, 重构损失: 242626.4387, Renyi损失: 228.9960, NaN批次: 0
提前停止训练！验证损失5轮未改善。
Renyi散度CVAE模型已保存为: clip_cvae_renyi_alpha0.9_final.pth
使用Renyi散度(alpha=0.9)的CVAE模型训练完成！


In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import clip
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm

# ------------------------ #
# 1. 设备设置
# ------------------------ #

# 选择设备（GPU或CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")

# ------------------------ #
# 2. 加载CLIP模型
# ------------------------ #

# 加载CLIP模型和预处理方法
clip_model, preprocess = clip.load("ViT-B/32", device=device)
clip_model.eval()
print("CLIP模型已加载。")

# ------------------------ #
# 3. 定义CelebA数据集类
# ------------------------ #

class CelebADataset(Dataset):
    def __init__(self, img_dir, attr_path, bbox_path, partition_path, 
                 image_embeddings_path, transform=None, partition=0):
        """
        初始化数据集
        :param img_dir: 图像文件夹路径
        :param attr_path: 属性文件路径
        :param bbox_path: 边界框文件路径
        :param partition_path: 分区文件路径
        :param image_embeddings_path: 预先计算的图像嵌入文件路径
        :param transform: 图像预处理
        :param partition: 使用的数据分区 (0: train, 1: val, 2: test)
        """
        self.img_dir = img_dir
        self.transform = transform

        # 读取属性文件
        try:
            attr_df = pd.read_csv(attr_path, sep=',', header=0)
        except Exception as e:
            print(f"Error reading {attr_path}: {e}")
            raise

        print("Attributes DataFrame columns:", attr_df.columns.tolist())
        print("Attributes DataFrame head:\n", attr_df.head())

        # 确认 'image_id' 列存在
        if 'image_id' not in attr_df.columns:
            raise ValueError("属性文件中缺少 'image_id' 列。请检查文件格式。")
        else:
            print("'image_id' column found in attributes DataFrame.")

        # 读取分区文件
        try:
            partition_df = pd.read_csv(partition_path, sep=',', header=0, names=['image_id', 'partition'])
        except Exception as e:
            print(f"Error reading {partition_path}: {e}")
            raise

        print("Partition DataFrame columns:", partition_df.columns.tolist())
        print("Partition DataFrame head:\n", partition_df.head())

        # 读取边界框文件
        try:
            bbox_df = pd.read_csv(bbox_path, sep=',', header=0, names=['image_id', 'x_1', 'y_1', 'width', 'height'])
        except Exception as e:
            print(f"Error reading {bbox_path}: {e}")
            raise

        print("Bounding Box DataFrame columns:", bbox_df.columns.tolist())
        print("Bounding Box DataFrame head:\n", bbox_df.head())

        # 合并属性和分区数据
        try:
            attr_df = attr_df.merge(partition_df, on='image_id')
        except Exception as e:
            print(f"Error merging attributes and partitions: {e}")
            raise

        print("After merging with partition, columns:", attr_df.columns.tolist())
        print("Merged DataFrame head:\n", attr_df.head())

        # 合并属性、分区和边界框数据
        try:
            attr_df = attr_df.merge(bbox_df, on='image_id')
        except Exception as e:
            print(f"Error merging attributes, partitions and bounding boxes: {e}")
            raise

        print("After merging with bounding box, columns:", attr_df.columns.tolist())
        print("Final Merged DataFrame head:\n", attr_df.head())

        # 筛选分区
        print(f"Total samples before filtering: {len(attr_df)}")
        self.attr_df = attr_df[attr_df['partition'] == partition].reset_index(drop=True)
        print(f"Total samples after filtering partition {partition}: {len(self.attr_df)}")

        if len(self.attr_df) == 0:
            raise ValueError(f"训练集中没有样本。请检查数据文件的读取和过滤逻辑。")

        # 获取属性名称
        # 属性列是除了 'image_id', 'partition', 'x_1', 'y_1', 'width', 'height' 之外的所有列
        self.attr_names = [col for col in attr_df.columns if col not in ['image_id', 'partition', 'x_1', 'y_1', 'width', 'height']]
        print("Attribute names:", self.attr_names)

        # 加载预计算的图像嵌入
        self.image_embeddings = torch.load(image_embeddings_path)  # 假设是一个Tensor，shape=(num_images, 512)
        if len(self.image_embeddings) != len(self.attr_df):
            raise ValueError("图像嵌入的数量与数据集中的图像数量不一致。")
        self.image_embeddings = self.image_embeddings.float()

    def __len__(self):
        return len(self.attr_df)

    def __getitem__(self, idx):
        # 获取图像路径
        img_name = self.attr_df.iloc[idx, self.attr_df.columns.get_loc('image_id')]
        img_path = os.path.join(self.img_dir, img_name)
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Error opening image {img_path}: {e}")
            # 可以选择跳过该样本，或返回一个默认图像
            # 这里选择抛出异常
            raise

        if self.transform:
            image = self.transform(image)

        # 获取属性标签 
        attrs = self.attr_df.iloc[idx][self.attr_names].values
        attrs = (attrs + 1) // 2  # 将-1转为0，1保持1
        attrs = attrs.astype(np.float32)

        # 获取对应的图像嵌入
        clip_embedding = self.image_embeddings[idx]

        return image, attrs, clip_embedding

# ------------------------ #
# 4. 定义ClipCVAE模型（使用Renyi散度）
# ------------------------ #

class ClipCVAE_Renyi(nn.Module):
    def __init__(self, img_channels=3, img_size=64, latent_dim=128, 
                 cond_dim=40, clip_dim=512):
        super(ClipCVAE_Renyi, self).__init__()
        self.img_size = img_size
        self.latent_dim = latent_dim
        self.cond_dim = cond_dim
        self.clip_dim = clip_dim

        # 编码器部分
        self.encoder = nn.Sequential(
            nn.Conv2d(img_channels + cond_dim + clip_dim, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Flatten()
        )
        self.fc_mu = nn.Linear(512*4*4, latent_dim)
        self.fc_logvar = nn.Linear(512*4*4, latent_dim)

        # 解码器部分
        self.decoder_input = nn.Linear(latent_dim + cond_dim + clip_dim, 512*4*4)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, img_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def encode(self, x, c, clip_embedding):
        # 将条件标签、图像和CLIP嵌入拼接
        c = c.view(c.size(0), self.cond_dim, 1, 1).repeat(1, 1, self.img_size, self.img_size)
        clip_embedding = clip_embedding.view(clip_embedding.size(0), self.clip_dim, 1, 1).repeat(1, 1, self.img_size, self.img_size)
        x = torch.cat([x, c, clip_embedding], dim=1)
        x = self.encoder(x)
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z, c, clip_embedding):
        # 将潜在变量、条件标签和CLIP嵌入拼接
        z = torch.cat([z, c, clip_embedding], dim=1)
        x = self.decoder_input(z)
        x = x.view(-1, 512, 4, 4)
        x = self.decoder(x)
        return x

    def forward(self, x, c, clip_embedding):
        mu, logvar = self.encode(x, c, clip_embedding)
        z = self.reparameterize(mu, logvar)
        recon_x = self.decode(z, c, clip_embedding)
        return recon_x, mu, logvar

# ------------------------ #
# 5. 图像预处理
# ------------------------ #

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # 修正归一化
])

# ------------------------ #
# 6. 数据集路径设置
# ------------------------ #

# 请根据实际路径修改
img_dir = '/root/autodl-tmp/celeba_datasets/img_align_celeba/img_align_celeba'
attr_path = '/root/autodl-tmp/celeba_datasets/list_attr_celeba.txt'
bbox_path = '/root/autodl-tmp/celeba_datasets/list_bbox_celeba.txt'
partition_path = '/root/autodl-tmp/celeba_datasets/list_eval_partition.txt'
image_embeddings_train_path = '/root/autodl-tmp/clip_embeddings/image_embeddings_partition_0.pt'
image_embeddings_val_path = '/root/autodl-tmp/clip_embeddings/image_embeddings_partition_1.pt'

# ------------------------ #
# 7. 创建训练集和验证集
# ------------------------ #

train_dataset = CelebADataset(img_dir, attr_path, bbox_path, partition_path, 
                              image_embeddings_train_path, transform=transform, partition=0)
val_dataset = CelebADataset(img_dir, attr_path, bbox_path, partition_path, 
                            image_embeddings_val_path, transform=transform, partition=1)

# ------------------------ #
# 8. 数据加载器
# ------------------------ #

batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# ------------------------ #
# 9. 初始化模型、损失函数和优化器
# ------------------------ #

latent_dim = 128
cond_dim = 40
clip_dim = 512
model_renyi = ClipCVAE_Renyi(img_channels=3, img_size=64, latent_dim=latent_dim, 
                           cond_dim=cond_dim, clip_dim=clip_dim).to(device)

# 使用更低的学习率
optimizer = optim.Adam(model_renyi.parameters(), lr=5e-5, weight_decay=1e-5)  

# 学习率调度器: 更慢的降低
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=8, gamma=0.7)  

# 使用mean而非sum来降低损失值
criterion = nn.MSELoss(reduction='mean')

# ------------------------ #
# 10. 定义改进的Renyi散度损失函数
# ------------------------ #

def improved_renyi_divergence_loss(recon_x, x, mu, logvar, alpha=0.7):
    """
    使用数值稳定版本的Renyi散度作为VAE的损失函数
    :param recon_x: 重构的图像
    :param x: 原始图像
    :param mu: 均值
    :param logvar: 对数方差
    :param alpha: Renyi散度参数，使用0.7而非0.9，以增加多样性
    :return: 总损失
    """
    # 重构损失 - 使用mean计算，但乘以batch和通道数，使数值更具可比性
    batch_size = x.size(0)
    channels = x.size(1)
    height = x.size(2)
    width = x.size(3)
    pixels_per_batch = batch_size * channels * height * width
    
    # 使用均值，但乘以尺寸得到近似总和，这样数值不会过大
    recon_loss = criterion(recon_x, x) * pixels_per_batch / 1000  # 除以1000降低数量级
    
    # 更严格限制对数方差范围以提高数值稳定性
    logvar = torch.clamp(logvar, min=-8.0, max=2.0)
    
    # 计算方差 (增加下限避免除零)
    var = torch.exp(logvar).clamp(min=1e-5, max=10.0)
    
    # 标准正态分布的参数
    mu0 = torch.zeros_like(mu)
    var0 = torch.ones_like(var)
    
    # 计算均值差平方
    mu_diff_sq = (mu - mu0).pow(2)
    
    # 使用对数空间计算，避免数值溢出
    log_det_ratio = torch.log(var) - torch.log(var0)
    
    # 使用α=0.7计算Rényi散度，不使用α=0.5或α=0.9
    weight_var = alpha * var0 + (1 - alpha) * var
    weight_var = torch.clamp(weight_var, min=1e-5)  # 确保分母不为零
    
    # 按部分计算，避免大数相乘
    term1 = 0.5 * torch.log(weight_var)
    term2 = 0.5 * (alpha * (1 - alpha) * mu_diff_sq / weight_var)
    term3 = -0.5 * (alpha * torch.log(var0) + (1 - alpha) * torch.log(var))
    
    # 按元素计算，确保数值稳定
    per_element_div = (term1 + term2 + term3) / (alpha - 1 + 1e-10)
    
    # 求和得到总散度
    renyi_div = per_element_div.sum()
    
    # 应用权重因子使Renyi散度与重构损失平衡
    weight_factor = 0.001  # 使用更小的权重以平衡损失
    
    # 确保散度是非负的，但不要放大太多
    renyi_div = torch.abs(renyi_div) * weight_factor
    
    # 增加梯度保护，防止梯度爆炸
    renyi_div = torch.clamp(renyi_div, max=100.0)
    
    # 总损失
    total_loss = recon_loss + renyi_div
    
    return total_loss, recon_loss, renyi_div

# ------------------------ #
# 11. 训练模型
# ------------------------ #

def train_model_renyi(model, train_loader, val_loader, optimizer, scheduler, num_epochs=30, alpha=0.7):
    """
    使用改进的Renyi散度训练CVAE模型
    """
    print(f"开始训练改进版Renyi散度CVAE (alpha={alpha})...")
    
    # 记录训练和验证损失
    train_losses = []
    val_losses = []
    
    # 初始化最小损失值（用于提前停止）
    best_val_loss = float('inf')
    patience = 5
    patience_counter = 0
    
    for epoch in range(1, num_epochs + 1):
        model.train()
        train_loss = 0
        train_recon_loss = 0
        train_renyi_loss = 0
        nan_batches = 0
        
        for batch_idx, (data, attrs, clip_emb) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}")):
            data = data.to(device, non_blocking=True)
            attrs = attrs.to(device, non_blocking=True)
            clip_emb = clip_emb.to(device, non_blocking=True)
            
            optimizer.zero_grad()
            
            # 使用try-except块处理可能的错误
            try:
                recon_batch, mu, logvar = model(data, attrs, clip_emb)
                
                # 计算损失
                loss, recon_loss, renyi_loss = improved_renyi_divergence_loss(
                    recon_batch, data, mu, logvar, alpha=alpha
                )
                
                # 检查损失是否为NaN
                if torch.isnan(loss).any() or torch.isinf(loss).any():
                    nan_batches += 1
                    if batch_idx % 50 == 0:  # 减少日志输出
                        print(f"警告: 第{epoch}轮, 第{batch_idx}批次出现NaN/Inf损失。跳过此批次.")
                    continue
                
                # 反向传播
                loss.backward()
                
                # 添加梯度裁剪，防止梯度爆炸
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.05)  # 更严格的梯度裁剪
                
                # 更新权重
                optimizer.step()
                
                # 累计损失
                train_loss += loss.item()
                train_recon_loss += recon_loss.item()
                train_renyi_loss += renyi_loss.item()
                
            except RuntimeError as e:
                print(f"批次{batch_idx}处理出错: {e}")
                continue
        
        # 更新学习率
        scheduler.step()
        current_lr = optimizer.param_groups[0]["lr"]
        
        # 计算平均损失（排除NaN批次）
        processed_batches = len(train_loader) - nan_batches
        if processed_batches > 0:
            avg_loss = train_loss / processed_batches
            avg_recon_loss = train_recon_loss / processed_batches
            avg_renyi_loss = train_renyi_loss / processed_batches
            train_losses.append(avg_loss)
            
            print(f"Epoch {epoch}, 平均训练损失: {avg_loss:.4f}, 重构损失: {avg_recon_loss:.4f}, "
                  f"Renyi损失: {avg_renyi_loss:.4f}, 学习率: {current_lr:.6f}, NaN批次: {nan_batches}")
        else:
            print(f"Epoch {epoch}, 所有批次均为NaN，无法计算平均损失")
            continue
        
        # 验证集评估
        model.eval()
        val_loss = 0
        val_recon_loss = 0
        val_renyi_loss = 0
        val_nan_batches = 0
        
        with torch.no_grad():
            for data, attrs, clip_emb in val_loader:
                data = data.to(device, non_blocking=True)
                attrs = attrs.to(device, non_blocking=True)
                clip_emb = clip_emb.to(device, non_blocking=True)
                
                try:
                    recon_batch, mu, logvar = model(data, attrs, clip_emb)
                    loss, recon_loss, renyi_loss = improved_renyi_divergence_loss(
                        recon_batch, data, mu, logvar, alpha=alpha
                    )
                    
                    if torch.isnan(loss).any() or torch.isinf(loss).any():
                        val_nan_batches += 1
                        continue
                    
                    val_loss += loss.item()
                    val_recon_loss += recon_loss.item()
                    val_renyi_loss += renyi_loss.item()
                    
                except RuntimeError:
                    val_nan_batches += 1
                    continue
        
        # 计算平均验证损失（排除NaN批次）
        processed_val_batches = len(val_loader) - val_nan_batches
        if processed_val_batches > 0:
            avg_val_loss = val_loss / processed_val_batches
            avg_val_recon_loss = val_recon_loss / processed_val_batches
            avg_val_renyi_loss = val_renyi_loss / processed_val_batches
            val_losses.append(avg_val_loss)
            
            print(f"验证集平均损失: {avg_val_loss:.4f}, 重构损失: {avg_val_recon_loss:.4f}, "
                  f"Renyi损失: {avg_val_renyi_loss:.4f}, NaN批次: {val_nan_batches}")
            
            # 提前停止检查
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                patience_counter = 0
                
                # 保存最佳模型
                best_model_path = f'clip_cvae_renyi_alpha{alpha}_best.pth'
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict(),
                    'val_loss': avg_val_loss,
                    'alpha': alpha
                }, best_model_path)
                print(f"保存最佳模型: {best_model_path}")
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print(f"提前停止训练！验证损失{patience}轮未改善。")
                    return model, train_losses, val_losses
        else:
            print(f"验证集所有批次均为NaN，无法计算平均损失")
        
        # 每5个epoch或最后一个epoch保存一次模型
        if epoch % 5 == 0 or epoch == num_epochs:
            checkpoint_path = f'clip_cvae_renyi_alpha{alpha}_epoch_{epoch}.pth'
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'train_loss': avg_loss,
                'val_loss': avg_val_loss if processed_val_batches > 0 else float('inf'),
                'alpha': alpha
            }, checkpoint_path)
            print(f"模型检查点已保存: {checkpoint_path}")
    
    return model, train_losses, val_losses

# ------------------------ #
# 12. 主程序
# ------------------------ #

if __name__ == "__main__":
    try:
        # 设置Renyi散度参数为0.7，相比0.9能获得更好的多样性
        alpha = 0.7
        
        # 正确初始化模型权重 - 使用Kaiming初始化
        for name, param in model_renyi.named_parameters():
            if 'weight' in name:
                if param.dim() >= 2:  # 对卷积或线性层的权重
                    nn.init.kaiming_normal_(param, a=0.01, mode='fan_in', nonlinearity='relu')
                else:  # 对一维权重
                    nn.init.normal_(param, mean=0.0, std=0.005)
            elif 'bias' in name:
                nn.init.zeros_(param)
        
        print(f"已成功完成模型参数初始化，使用alpha={alpha}的Rényi散度")
        
        # 训练使用Renyi散度的CVAE模型
        trained_model, train_losses, val_losses = train_model_renyi(
            model_renyi, train_loader, val_loader, optimizer, scheduler, 
            num_epochs=30, alpha=alpha
        )
        
        # 保存最终模型
        final_model_path = f'clip_cvae_renyi_alpha{alpha}_final.pth'
        torch.save(trained_model.state_dict(), final_model_path)
        print(f"Renyi散度CVAE模型已保存为: {final_model_path}")
        
        # 打印训练完成信息
        print(f"使用Renyi散度(alpha={alpha})的CVAE模型训练完成！")
        
    except Exception as e:
        print(f"训练过程中发生错误: {e}")
        import traceback
        traceback.print_exc()
        # 在发生错误时尝试保存当前模型状态
        emergency_save_path = f'clip_cvae_renyi_alpha{alpha}_emergency_save.pth'
        torch.save(model_renyi.state_dict(), emergency_save_path)
        print(f"发生错误，模型紧急保存为: {emergency_save_path}")

使用设备: cuda
CLIP模型已加载。
Attributes DataFrame columns: ['image_id', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive', 'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose', 'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows', 'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace', 'Wearing_Necktie', 'Young']
Attributes DataFrame head:
      image_id  5_o_Clock_Shadow  Arched_Eyebrows  Attractive  Bags_Under_Eyes  \
0  000001.jpg                -1                1           1               -1   
1  000002.jpg                -1               -1          -1                1   
2  000003.jpg                -1               -1          -1               -1   

Epoch 1/30: 100%|██████████| 1272/1272 [01:02<00:00, 20.49it/s]

Epoch 1, 平均训练损失: 282.6987, 重构损失: 276.7783, Renyi损失: 5.9204, 学习率: 0.000050, NaN批次: 0


验证集平均损失: 113.7345, 重构损失: 102.0183, Renyi损失: 11.7161, NaN批次: 0
保存最佳模型: clip_cvae_renyi_alpha0.7_best.pth


Epoch 2/30: 100%|██████████| 1272/1272 [01:00<00:00, 21.15it/s]

...

Epoch 29, 平均训练损失: 53.3860, 重构损失: 37.7183, Renyi损失: 15.6678, 学习率: 0.000017, NaN批次: 0


验证集平均损失: 53.8553, 重构损失: 38.3647, Renyi损失: 15.4906, NaN批次: 0


Epoch 30/30: 100%|██████████| 1272/1272 [01:00<00:00, 20.97it/s]

Epoch 30, 平均训练损失: 53.2662, 重构损失: 37.5945, Renyi损失: 15.6717, 学习率: 0.000017, NaN批次: 0


验证集平均损失: 53.6242, 重构损失: 38.1038, Renyi损失: 15.5205, NaN批次: 0
保存最佳模型: clip_cvae_renyi_alpha0.7_best.pth
模型检查点已保存: clip_cvae_renyi_alpha0.7_epoch_30.pth
Renyi散度CVAE模型已保存为: clip_cvae_renyi_alpha0.7_final.pth
使用Renyi散度(alpha=0.7)的CVAE模型训练完成！


In [ ]:
import os
import math
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import clip
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm

# ------------------------ #
# 1. 设备设置
# ------------------------ #

# 选择设备（GPU或CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")

# ------------------------ #
# 2. 加载CLIP模型
# ------------------------ #

# 加载CLIP模型和预处理方法
clip_model, preprocess = clip.load("ViT-B/32", device=device)
clip_model.eval()
print("CLIP模型已加载。")

# ------------------------ #
# 3. 定义CelebA数据集类
# ------------------------ #

class CelebADataset(Dataset):
    def __init__(self, img_dir, attr_path, bbox_path, partition_path, 
                 image_embeddings_path, transform=None, partition=0):
        """
        初始化数据集
        :param img_dir: 图像文件夹路径
        :param attr_path: 属性文件路径
        :param bbox_path: 边界框文件路径
        :param partition_path: 分区文件路径
        :param image_embeddings_path: 预先计算的图像嵌入文件路径
        :param transform: 图像预处理
        :param partition: 使用的数据分区 (0: train, 1: val, 2: test)
        """
        self.img_dir = img_dir
        self.transform = transform

        # 读取属性文件
        try:
            attr_df = pd.read_csv(attr_path, sep=',', header=0)
        except Exception as e:
            print(f"Error reading {attr_path}: {e}")
            raise

        print("Attributes DataFrame columns:", attr_df.columns.tolist())
        print("Attributes DataFrame head:\n", attr_df.head())

        # 确认 'image_id' 列存在
        if 'image_id' not in attr_df.columns:
            raise ValueError("属性文件中缺少 'image_id' 列。请检查文件格式。")
        else:
            print("'image_id' column found in attributes DataFrame.")

        # 读取分区文件
        try:
            partition_df = pd.read_csv(partition_path, sep=',', header=0, names=['image_id', 'partition'])
        except Exception as e:
            print(f"Error reading {partition_path}: {e}")
            raise

        print("Partition DataFrame columns:", partition_df.columns.tolist())
        print("Partition DataFrame head:\n", partition_df.head())

        # 读取边界框文件
        try:
            bbox_df = pd.read_csv(bbox_path, sep=',', header=0, names=['image_id', 'x_1', 'y_1', 'width', 'height'])
        except Exception as e:
            print(f"Error reading {bbox_path}: {e}")
            raise

        print("Bounding Box DataFrame columns:", bbox_df.columns.tolist())
        print("Bounding Box DataFrame head:\n", bbox_df.head())

        # 合并属性和分区数据
        try:
            attr_df = attr_df.merge(partition_df, on='image_id')
        except Exception as e:
            print(f"Error merging attributes and partitions: {e}")
            raise

        print("After merging with partition, columns:", attr_df.columns.tolist())
        print("Merged DataFrame head:\n", attr_df.head())

        # 合并属性、分区和边界框数据
        try:
            attr_df = attr_df.merge(bbox_df, on='image_id')
        except Exception as e:
            print(f"Error merging attributes, partitions and bounding boxes: {e}")
            raise

        print("After merging with bounding box, columns:", attr_df.columns.tolist())
        print("Final Merged DataFrame head:\n", attr_df.head())

        # 筛选分区
        print(f"Total samples before filtering: {len(attr_df)}")
        self.attr_df = attr_df[attr_df['partition'] == partition].reset_index(drop=True)
        print(f"Total samples after filtering partition {partition}: {len(self.attr_df)}")

        if len(self.attr_df) == 0:
            raise ValueError(f"训练集中没有样本。请检查数据文件的读取和过滤逻辑。")

        # 获取属性名称
        # 属性列是除了 'image_id', 'partition', 'x_1', 'y_1', 'width', 'height' 之外的所有列
        self.attr_names = [col for col in attr_df.columns if col not in ['image_id', 'partition', 'x_1', 'y_1', 'width', 'height']]
        print("Attribute names:", self.attr_names)

        # 加载预计算的图像嵌入
        self.image_embeddings = torch.load(image_embeddings_path)  # 假设是一个Tensor，shape=(num_images, 512)
        if len(self.image_embeddings) != len(self.attr_df):
            raise ValueError("图像嵌入的数量与数据集中的图像数量不一致。")
        self.image_embeddings = self.image_embeddings.float()

    def __len__(self):
        return len(self.attr_df)

    def __getitem__(self, idx):
        # 获取图像路径
        img_name = self.attr_df.iloc[idx, self.attr_df.columns.get_loc('image_id')]
        img_path = os.path.join(self.img_dir, img_name)
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Error opening image {img_path}: {e}")
            # 可以选择跳过该样本，或返回一个默认图像
            # 这里选择抛出异常
            raise

        if self.transform:
            image = self.transform(image)

        # 获取属性标签 
        attrs = self.attr_df.iloc[idx][self.attr_names].values
        attrs = (attrs + 1) // 2  # 将-1转为0，1保持1
        attrs = attrs.astype(np.float32)

        # 获取对应的图像嵌入
        clip_embedding = self.image_embeddings[idx]

        return image, attrs, clip_embedding

# ------------------------ #
# 4. 定义ClipCVAE模型（使用Renyi散度）
# ------------------------ #

class ClipCVAE_Renyi(nn.Module):
    def __init__(self, img_channels=3, img_size=64, latent_dim=128, 
                 cond_dim=40, clip_dim=512):
        super(ClipCVAE_Renyi, self).__init__()
        self.img_size = img_size
        self.latent_dim = latent_dim
        self.cond_dim = cond_dim
        self.clip_dim = clip_dim

        # 编码器部分
        self.encoder = nn.Sequential(
            nn.Conv2d(img_channels + cond_dim + clip_dim, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Flatten()
        )
        self.fc_mu = nn.Linear(512*4*4, latent_dim)
        self.fc_logvar = nn.Linear(512*4*4, latent_dim)

        # 解码器部分
        self.decoder_input = nn.Linear(latent_dim + cond_dim + clip_dim, 512*4*4)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, img_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def encode(self, x, c, clip_embedding):
        # 将条件标签、图像和CLIP嵌入拼接
        c = c.view(c.size(0), self.cond_dim, 1, 1).repeat(1, 1, self.img_size, self.img_size)
        clip_embedding = clip_embedding.view(clip_embedding.size(0), self.clip_dim, 1, 1).repeat(1, 1, self.img_size, self.img_size)
        x = torch.cat([x, c, clip_embedding], dim=1)
        x = self.encoder(x)
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z, c, clip_embedding):
        # 将潜在变量、条件标签和CLIP嵌入拼接
        z = torch.cat([z, c, clip_embedding], dim=1)
        x = self.decoder_input(z)
        x = x.view(-1, 512, 4, 4)
        x = self.decoder(x)
        return x

    def forward(self, x, c, clip_embedding):
        mu, logvar = self.encode(x, c, clip_embedding)
        z = self.reparameterize(mu, logvar)
        recon_x = self.decode(z, c, clip_embedding)
        return recon_x, mu, logvar

# ------------------------ #
# 5. 图像预处理
# ------------------------ #

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # 归一化
])

# ------------------------ #
# 6. 数据集路径设置
# ------------------------ #

# 请根据实际路径修改
img_dir = '/root/autodl-tmp/celeba_datasets/img_align_celeba/img_align_celeba'
attr_path = '/root/autodl-tmp/celeba_datasets/list_attr_celeba.txt'
bbox_path = '/root/autodl-tmp/celeba_datasets/list_bbox_celeba.txt'
partition_path = '/root/autodl-tmp/celeba_datasets/list_eval_partition.txt'
image_embeddings_train_path = '/root/autodl-tmp/clip_embeddings/image_embeddings_partition_0.pt'
image_embeddings_val_path = '/root/autodl-tmp/clip_embeddings/image_embeddings_partition_1.pt'

# ------------------------ #
# 7. 创建训练集和验证集
# ------------------------ #

train_dataset = CelebADataset(img_dir, attr_path, bbox_path, partition_path, 
                              image_embeddings_train_path, transform=transform, partition=0)
val_dataset = CelebADataset(img_dir, attr_path, bbox_path, partition_path, 
                            image_embeddings_val_path, transform=transform, partition=1)

# ------------------------ #
# 8. 数据加载器
# ------------------------ #

batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# ------------------------ #
# 9. 修订版 Rényi 散度损失函数
# ------------------------ #

def revised_renyi_divergence_loss(recon_x, x, mu, logvar, epoch=1):
    """
    完全重构的 Rényi 散度损失函数，专注于优化 α=0.5 时的性能
    
    1. 使用不同的重构损失计算方式
    2. 重新设计 Hellinger 距离计算
    3. 强调 KL 散度和 Rényi 散度之间的平衡
    """
    # 1. 重构损失使用更合适的均方误差计算
    batch_size = x.size(0)
    
    # 使用相对较小的重构损失权重，避免其主导优化过程
    recon_loss = F.mse_loss(recon_x, x, reduction='mean') * 100
    
    # 2. 更精确的 Rényi 散度计算 (α=0.5 对应 Hellinger 距离)
    # 限制参数范围，但不要过于严格
    logvar = torch.clamp(logvar, min=-20.0, max=10.0)
    var = torch.exp(logvar)
    
    # 标准正态先验
    var0 = torch.ones_like(var)
    mu0 = torch.zeros_like(mu)
    
    # Hellinger 距离公式的精确实现
    # H²(P,Q) = 2(1 - BC(P,Q))，其中 BC 是 Bhattacharyya 系数
    
    # 计算协方差矩阵行列式
    var_term = torch.sqrt(var * var0)
    var_sum = 0.5 * (var + var0)
    
    # 计算均值项
    mu_diff_sq = (mu - mu0).pow(2)
    
    # Bhattacharyya 系数
    bc_term = var_term / torch.sqrt(var_sum)
    exp_term = torch.exp(-0.25 * mu_diff_sq / var_sum)
    bhattacharyya_coeff = bc_term * exp_term
    
    # Hellinger 距离的平方
    hellinger_sq = 2 * (1 - bhattacharyya_coeff)
    
    # 确保数值稳定性
    hellinger_sq = torch.clamp(hellinger_sq, min=0.0, max=2.0)
    
    # 求和得到总 Hellinger 距离
    sum_hellinger = hellinger_sq.sum(dim=1)
    
    # 3. 在训练早期更加注重重构，随着训练进行增加 Rényi 散度的权重
    # 平衡重构损失和 Rényi 散度 - 使用非线性曲线增长权重
    max_weight = 10.0  # 更大的最大权重
    min_weight = 0.5   # 更大的起始权重
    
    # 使用指数曲线增长
    progress = min(epoch / 15.0, 1.0)  # 15 轮逐步增长到最大值
    weight_factor = min_weight + (max_weight - min_weight) * (1 - math.exp(-3 * progress))
    
    # 计算平均 Hellinger 距离，并应用权重
    renyi_div = sum_hellinger.mean() * weight_factor
    
    # 4. 加入 KL 散度作为辅助正则化，帮助训练稳定
    # 标准 KL 散度计算
    kl_div = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp()) / batch_size
    kl_weight = max(0.01, 0.1 * (1.0 - progress))  # 随训练进程减小 KL 权重
    
    # 监控信息
    raw_renyi = sum_hellinger.mean().item()
    raw_kl = kl_div.item()
    
    # 总损失
    total_loss = recon_loss + renyi_div + kl_weight * kl_div
    
    return total_loss, recon_loss, renyi_div, raw_renyi, kl_weight * kl_div, raw_kl

# ------------------------ #
# 10. 改进的训练函数
# ------------------------ #

def train_revised_model(model, train_loader, val_loader, optimizer, scheduler, num_epochs=30):
    """
    改进的训练函数，加入更多监控和调整
    """
    print("开始使用修订版 Hellinger 距离 (α=0.5) 训练 CVAE 模型...")
    
    # 记录全面的训练统计信息
    train_stats = {
        'total_loss': [], 'recon_loss': [], 'renyi_loss': [], 
        'raw_renyi': [], 'kl_loss': [], 'raw_kl': []
    }
    val_stats = {
        'total_loss': [], 'recon_loss': [], 'renyi_loss': [], 
        'raw_renyi': [], 'kl_loss': [], 'raw_kl': []
    }
    
    # 最佳模型跟踪
    best_val_loss = float('inf')
    patience = 10  # 增加耐心值，给模型更多时间学习
    patience_counter = 0
    
    # 确保保存目录存在
    os.makedirs('model_checkpoints', exist_ok=True)
    
    for epoch in range(1, num_epochs + 1):
        model.train()
        epoch_stats = {k: 0.0 for k in train_stats.keys()}
        nan_batches = 0
        
        # 输出当前训练进度和权重情况
        progress = min(epoch / 15.0, 1.0)
        min_weight = 0.5
        max_weight = 10.0
        weight_factor = min_weight + (max_weight - min_weight) * (1 - math.exp(-3 * progress))
        kl_weight = max(0.01, 0.1 * (1.0 - progress))
        
        print(f"Epoch {epoch}/{num_epochs}, Rényi权重: {weight_factor:.4f}, KL权重: {kl_weight:.4f}")
        
        # 训练循环
        for batch_idx, (data, attrs, clip_emb) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}")):
            data = data.to(device, non_blocking=True)
            attrs = attrs.to(device, non_blocking=True)
            clip_emb = clip_emb.to(device, non_blocking=True)
            
            optimizer.zero_grad()
            
            try:
                recon_batch, mu, logvar = model(data, attrs, clip_emb)
                
                # 使用修订版损失函数
                loss, recon_loss, renyi_loss, raw_renyi, kl_loss, raw_kl = revised_renyi_divergence_loss(
                    recon_batch, data, mu, logvar, epoch
                )
                
                # 详细调试输出
                if batch_idx % 100 == 0:
                    print(f"\n批次 {batch_idx} 详细信息:")
                    print(f"  - 原始 Rényi 值: {raw_renyi:.6f}, 加权 Rényi 损失: {renyi_loss.item():.6f}")
                    print(f"  - 原始 KL 值: {raw_kl:.6f}, 加权 KL 损失: {kl_loss.item():.6f}")
                    print(f"  - 重构损失: {recon_loss.item():.6f}, 总损失: {loss.item():.6f}")
                    print(f"  - mu范围: [{mu.min().item():.4f}, {mu.max().item():.4f}], "
                          f"var范围: [{torch.exp(logvar).min().item():.6f}, {torch.exp(logvar).max().item():.6f}]")
                    
                    # 计算分布差异指标
                    with torch.no_grad():
                        kl_div_mean = 0.5 * (mu.pow(2) + torch.exp(logvar) - logvar - 1).mean().item()
                        print(f"  - 平均每维 KL 散度: {kl_div_mean:.6f}")
                
                # 检查 NaN
                if torch.isnan(loss).any() or torch.isinf(loss).any():
                    nan_batches += 1
                    if batch_idx % 100 == 0:
                        print(f"警告: 第{epoch}轮, 第{batch_idx}批次出现NaN/Inf损失。跳过此批次.")
                    continue
                
                # 反向传播和优化
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)  # 放宽梯度裁剪限制
                optimizer.step()
                
                # 累计统计信息
                epoch_stats['total_loss'] += loss.item()
                epoch_stats['recon_loss'] += recon_loss.item()
                epoch_stats['renyi_loss'] += renyi_loss.item()
                epoch_stats['raw_renyi'] += raw_renyi
                epoch_stats['kl_loss'] += kl_loss.item()
                epoch_stats['raw_kl'] += raw_kl
                
            except RuntimeError as e:
                print(f"批次{batch_idx}处理出错: {e}")
                nan_batches += 1
                continue
        
        # 更新学习率
        scheduler.step()
        current_lr = optimizer.param_groups[0]["lr"]
        
        # 计算平均损失
        processed_batches = len(train_loader) - nan_batches
        if processed_batches > 0:
            for k in epoch_stats:
                epoch_stats[k] /= processed_batches
                train_stats[k].append(epoch_stats[k])
            
            print(f"\nEpoch {epoch} 训练统计:")
            print(f"  - 平均总损失: {epoch_stats['total_loss']:.4f}")
            print(f"  - 重构损失: {epoch_stats['recon_loss']:.4f}")
            print(f"  - Rényi损失: {epoch_stats['renyi_loss']:.4f} (原始值: {epoch_stats['raw_renyi']:.6f})")
            print(f"  - KL损失: {epoch_stats['kl_loss']:.4f} (原始值: {epoch_stats['raw_kl']:.6f})")
            print(f"  - 学习率: {current_lr:.6f}, NaN批次: {nan_batches}")
        else:
            print(f"Epoch {epoch}, 所有批次均为NaN，无法计算平均损失")
            continue
        
        # 验证集评估
        model.eval()
        val_epoch_stats = {k: 0.0 for k in val_stats.keys()}
        val_nan_batches = 0
        
        with torch.no_grad():
            for data, attrs, clip_emb in val_loader:
                data = data.to(device, non_blocking=True)
                attrs = attrs.to(device, non_blocking=True)
                clip_emb = clip_emb.to(device, non_blocking=True)
                
                try:
                    recon_batch, mu, logvar = model(data, attrs, clip_emb)
                    loss, recon_loss, renyi_loss, raw_renyi, kl_loss, raw_kl = revised_renyi_divergence_loss(
                        recon_batch, data, mu, logvar, epoch
                    )
                    
                    if torch.isnan(loss).any() or torch.isinf(loss).any():
                        val_nan_batches += 1
                        continue
                    
                    # 累计验证统计
                    val_epoch_stats['total_loss'] += loss.item()
                    val_epoch_stats['recon_loss'] += recon_loss.item()
                    val_epoch_stats['renyi_loss'] += renyi_loss.item()
                    val_epoch_stats['raw_renyi'] += raw_renyi
                    val_epoch_stats['kl_loss'] += kl_loss.item()
                    val_epoch_stats['raw_kl'] += raw_kl
                    
                except RuntimeError:
                    val_nan_batches += 1
                    continue
        
        # 计算验证平均值
        processed_val_batches = len(val_loader) - val_nan_batches
        if processed_val_batches > 0:
            for k in val_epoch_stats:
                val_epoch_stats[k] /= processed_val_batches
                val_stats[k].append(val_epoch_stats[k])
            
            print(f"验证集统计:")
            print(f"  - 平均总损失: {val_epoch_stats['total_loss']:.4f}")
            print(f"  - 重构损失: {val_epoch_stats['recon_loss']:.4f}")
            print(f"  - Rényi损失: {val_epoch_stats['renyi_loss']:.4f} (原始值: {val_epoch_stats['raw_renyi']:.6f})")
            print(f"  - KL损失: {val_epoch_stats['kl_loss']:.4f} (原始值: {val_epoch_stats['raw_kl']:.6f})")
            print(f"  - NaN批次: {val_nan_batches}")
            
            # 保存当前模型和检查最佳模型
            current_model_path = f'model_checkpoints/cvae_renyi_improved_epoch_{epoch}.pth'
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_stats': train_stats,
                'val_stats': val_stats,
                'alpha': 0.5,
            }, current_model_path)
            
            # 评估模型质量 - 使用重构损失和 Rényi 值的加权组合作为判断标准
            # 降低较低的原始 Rényi 值在评估中的权重，避免模型仅优化这个指标
            model_quality = val_epoch_stats['recon_loss'] - 0.5 * val_epoch_stats['raw_renyi']
            
            if model_quality < best_val_loss:
                best_val_loss = model_quality
                patience_counter = 0
                
                best_model_path = f'model_checkpoints/cvae_renyi_improved_best.pth'
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'best_quality': model_quality,
                    'train_stats': train_stats,
                    'val_stats': val_stats,
                    'alpha': 0.5,
                }, best_model_path)
                print(f"保存最佳模型: {best_model_path} (质量指标: {model_quality:.4f})")
            else:
                patience_counter += 1
                print(f"模型质量未改善，耐心计数: {patience_counter}/{patience}")
                if patience_counter >= patience:
                    print(f"提前停止训练！模型质量{patience}轮未改善。")
                    return model, train_stats, val_stats
        else:
            print(f"验证集所有批次均为NaN，无法计算平均损失")
    
    return model, train_stats, val_stats
# ------------------------ #
# 11. 可视化函数
# ------------------------ #

def plot_training_stats(train_stats, val_stats, save_path='training_stats.png'):
    """
    绘制训练过程中的各种统计数据
    """
    import matplotlib.pyplot as plt
    
    epochs = range(1, len(train_stats['total_loss']) + 1)
    
    plt.figure(figsize=(18, 12))
    
    # 绘制总损失
    plt.subplot(2, 3, 1)
    plt.plot(epochs, train_stats['total_loss'], 'b-', label='训练')
    plt.plot(epochs, val_stats['total_loss'], 'r-', label='验证')
    plt.title('总损失')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # 绘制重构损失
    plt.subplot(2, 3, 2)
    plt.plot(epochs, train_stats['recon_loss'], 'b-', label='训练')
    plt.plot(epochs, val_stats['recon_loss'], 'r-', label='验证')
    plt.title('重构损失')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # 绘制Rényi损失
    plt.subplot(2, 3, 3)
    plt.plot(epochs, train_stats['renyi_loss'], 'b-', label='训练')
    plt.plot(epochs, val_stats['renyi_loss'], 'r-', label='验证')
    plt.title('Rényi损失 (加权)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # 绘制原始Rényi值
    plt.subplot(2, 3, 4)
    plt.plot(epochs, train_stats['raw_renyi'], 'b-', label='训练')
    plt.plot(epochs, val_stats['raw_renyi'], 'r-', label='验证')
    plt.title('原始Rényi值')
    plt.xlabel('Epoch')
    plt.ylabel('Value')
    plt.legend()
    
    # 绘制KL损失
    plt.subplot(2, 3, 5)
    plt.plot(epochs, train_stats['kl_loss'], 'b-', label='训练')
    plt.plot(epochs, val_stats['kl_loss'], 'r-', label='验证')
    plt.title('KL损失 (加权)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # 绘制原始KL值
    plt.subplot(2, 3, 6)
    plt.plot(epochs, train_stats['raw_kl'], 'b-', label='训练')
    plt.plot(epochs, val_stats['raw_kl'], 'r-', label='验证')
    plt.title('原始KL值')
    plt.xlabel('Epoch')
    plt.ylabel('Value')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()
    print(f"训练统计图已保存至 {save_path}")

# ------------------------ #
# 12. 样本生成与可视化
# ------------------------ #

def generate_samples(model, val_loader, num_samples=10, save_dir='sample_images'):
    """
    生成样本并可视化比较
    """
    import matplotlib.pyplot as plt
    from torchvision.utils import make_grid
    
    os.makedirs(save_dir, exist_ok=True)
    
    model.eval()
    
    # 随机获取一些验证集样本
    data_iter = iter(val_loader)
    data, attrs, clip_emb = next(data_iter)
    
    data = data[:num_samples].to(device)
    attrs = attrs[:num_samples].to(device)
    clip_emb = clip_emb[:num_samples].to(device)
    
    with torch.no_grad():
        # 获取原始图像、重构图像
        recon_img, mu, logvar = model(data, attrs, clip_emb)
        
        # 从潜在空间采样新图像
        z = torch.randn_like(mu)
        sampled_img = model.decode(z, attrs, clip_emb)
    
    # 将张量转换为可视化的图像
    # 去归一化，从[-1, 1]到[0, 1]
    def denormalize(x):
        return (x + 1) / 2
    
    original_imgs = denormalize(data)
    recon_imgs = denormalize(recon_img)
    sampled_imgs = denormalize(sampled_img)
    
    # 显示原始、重构和采样的图像
    fig, axes = plt.subplots(3, num_samples, figsize=(num_samples * 2, 6))
    
    # 去除坐标轴
    for ax in axes.flatten():
        ax.axis('off')
    
    # 标题
    axes[0, num_samples//2].set_title('原始图像', fontsize=12, pad=10)
    axes[1, num_samples//2].set_title('重构图像', fontsize=12, pad=10)
    axes[2, num_samples//2].set_title('从潜在空间采样', fontsize=12, pad=10)
    
    # 显示图像
    for i in range(num_samples):
        # 原始图像
        img = original_imgs[i].cpu().permute(1, 2, 0).numpy()
        img = np.clip(img, 0, 1)
        axes[0, i].imshow(img)
        
        # 重构图像
        img = recon_imgs[i].cpu().permute(1, 2, 0).numpy()
        img = np.clip(img, 0, 1)
        axes[1, i].imshow(img)
        
        # 采样图像
        img = sampled_imgs[i].cpu().permute(1, 2, 0).numpy()
        img = np.clip(img, 0, 1)
        axes[2, i].imshow(img)
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, 'sample_comparison.png'), dpi=150)
    plt.close()
    
    print(f"样本可视化已保存至 {os.path.join(save_dir, 'sample_comparison.png')}")
    
    return {
        'original': original_imgs.cpu(),
        'recon': recon_imgs.cpu(),
        'sampled': sampled_imgs.cpu()
    }

# ------------------------ #
# 13. 主程序
# ------------------------ #

if __name__ == "__main__":
    try:
        # 设置随机种子确保可重复性
        torch.manual_seed(42)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(42)
        np.random.seed(42)
        random.seed(42)
        
        # 设置 Renyi 散度参数
        alpha = 0.5
        
        # 创建新模型实例 - 避免从之前的结果继续训练
        model_renyi = ClipCVAE_Renyi(img_channels=3, img_size=64, latent_dim=128, 
                                    cond_dim=40, clip_dim=512).to(device)
        
        # 初始化权重 - 使用更适合 VAE 的初始化方法
        for name, param in model_renyi.named_parameters():
            if 'weight' in name:
                if 'fc_mu' in name or 'fc_logvar' in name:
                    # 使用较小的初始值初始化均值和方差层
                    nn.init.normal_(param, mean=0.0, std=0.01)
                elif param.dim() >= 2:
                    # 使用 He 初始化卷积层
                    nn.init.kaiming_normal_(param, mode='fan_out', nonlinearity='relu')
                else:
                    nn.init.normal_(param, mean=0.0, std=0.01)
            elif 'bias' in name:
                nn.init.zeros_(param)
        
        print(f"已成功完成模型参数初始化，使用修订版 α={alpha} 的 Rényi 散度 (Hellinger 距离) 计算方法")
        
        # 优化器 - 使用 AdamW 和较低的学习率
        optimizer = optim.AdamW(model_renyi.parameters(), lr=1e-4, weight_decay=1e-4)
        
        # 调度器 - 余弦退火，学习率平滑下降
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30, eta_min=1e-6)
        
        # 训练模型
        trained_model, train_stats, val_stats = train_revised_model(
            model_renyi, train_loader, val_loader, optimizer, scheduler, 
            num_epochs=30
        )
        
        # 保存最终模型
        final_model_path = f'model_checkpoints/cvae_renyi_alpha{alpha}_final_revised.pth'
        torch.save({
            'model_state_dict': trained_model.state_dict(),
            'train_stats': train_stats,
            'val_stats': val_stats,
            'alpha': alpha
        }, final_model_path)
        print(f"修订版 Rényi 散度 (α=0.5) CVAE 模型已保存为: {final_model_path}")
        
        # 绘制训练过程中的损失变化图表
        plot_training_stats(train_stats, val_stats, save_path='model_checkpoints/training_stats.png')
        
        # 生成样本并可视化
        sample_results = generate_samples(
            trained_model, val_loader, num_samples=8, 
            save_dir='model_checkpoints/sample_images'
        )
        
        print(f"使用修订版 Rényi 散度 (α=0.5) 的 CVAE 模型训练完成！")
        
    except Exception as e:
        print(f"训练过程中发生错误: {e}")
        import traceback
        traceback.print_exc()
        # 保存紧急状态
        emergency_save_path = f'model_checkpoints/cvae_renyi_revised_emergency_save.pth'
        torch.save(model_renyi.state_dict(), emergency_save_path)
        print(f"发生错误，模型紧急保存为: {emergency_save_path}")

In [3]:
import os
import math
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import clip
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm

# ------------------------ #
# 1. 设备设置
# ------------------------ #

# 选择设备（GPU或CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")

# ------------------------ #
# 2. 加载CLIP模型
# ------------------------ #

# 加载CLIP模型和预处理方法
clip_model, preprocess = clip.load("ViT-B/32", device=device)
clip_model.eval()
print("CLIP模型已加载。")

# ------------------------ #
# 3. 定义CelebA数据集类
# ------------------------ #

class CelebADataset(Dataset):
    def __init__(self, img_dir, attr_path, bbox_path, partition_path, 
                 image_embeddings_path, transform=None, partition=0):
        """
        初始化数据集
        :param img_dir: 图像文件夹路径
        :param attr_path: 属性文件路径
        :param bbox_path: 边界框文件路径
        :param partition_path: 分区文件路径
        :param image_embeddings_path: 预先计算的图像嵌入文件路径
        :param transform: 图像预处理
        :param partition: 使用的数据分区 (0: train, 1: val, 2: test)
        """
        self.img_dir = img_dir
        self.transform = transform

        # 读取属性文件
        try:
            attr_df = pd.read_csv(attr_path, sep=',', header=0)
        except Exception as e:
            print(f"Error reading {attr_path}: {e}")
            raise

        print("Attributes DataFrame columns:", attr_df.columns.tolist())
        print("Attributes DataFrame head:\n", attr_df.head())

        # 确认 'image_id' 列存在
        if 'image_id' not in attr_df.columns:
            raise ValueError("属性文件中缺少 'image_id' 列。请检查文件格式。")
        else:
            print("'image_id' column found in attributes DataFrame.")

        # 读取分区文件
        try:
            partition_df = pd.read_csv(partition_path, sep=',', header=0, names=['image_id', 'partition'])
        except Exception as e:
            print(f"Error reading {partition_path}: {e}")
            raise

        print("Partition DataFrame columns:", partition_df.columns.tolist())
        print("Partition DataFrame head:\n", partition_df.head())

        # 读取边界框文件
        try:
            bbox_df = pd.read_csv(bbox_path, sep=',', header=0, names=['image_id', 'x_1', 'y_1', 'width', 'height'])
        except Exception as e:
            print(f"Error reading {bbox_path}: {e}")
            raise

        print("Bounding Box DataFrame columns:", bbox_df.columns.tolist())
        print("Bounding Box DataFrame head:\n", bbox_df.head())

        # 合并属性和分区数据
        try:
            attr_df = attr_df.merge(partition_df, on='image_id')
        except Exception as e:
            print(f"Error merging attributes and partitions: {e}")
            raise

        print("After merging with partition, columns:", attr_df.columns.tolist())
        print("Merged DataFrame head:\n", attr_df.head())

        # 合并属性、分区和边界框数据
        try:
            attr_df = attr_df.merge(bbox_df, on='image_id')
        except Exception as e:
            print(f"Error merging attributes, partitions and bounding boxes: {e}")
            raise

        print("After merging with bounding box, columns:", attr_df.columns.tolist())
        print("Final Merged DataFrame head:\n", attr_df.head())

        # 筛选分区
        print(f"Total samples before filtering: {len(attr_df)}")
        self.attr_df = attr_df[attr_df['partition'] == partition].reset_index(drop=True)
        print(f"Total samples after filtering partition {partition}: {len(self.attr_df)}")

        if len(self.attr_df) == 0:
            raise ValueError(f"训练集中没有样本。请检查数据文件的读取和过滤逻辑。")

        # 获取属性名称
        # 属性列是除了 'image_id', 'partition', 'x_1', 'y_1', 'width', 'height' 之外的所有列
        self.attr_names = [col for col in attr_df.columns if col not in ['image_id', 'partition', 'x_1', 'y_1', 'width', 'height']]
        print("Attribute names:", self.attr_names)

        # 加载预计算的图像嵌入
        self.image_embeddings = torch.load(image_embeddings_path)  # 假设是一个Tensor，shape=(num_images, 512)
        if len(self.image_embeddings) != len(self.attr_df):
            raise ValueError("图像嵌入的数量与数据集中的图像数量不一致。")
        self.image_embeddings = self.image_embeddings.float()

    def __len__(self):
        return len(self.attr_df)

    def __getitem__(self, idx):
        # 获取图像路径
        img_name = self.attr_df.iloc[idx, self.attr_df.columns.get_loc('image_id')]
        img_path = os.path.join(self.img_dir, img_name)
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Error opening image {img_path}: {e}")
            # 可以选择跳过该样本，或返回一个默认图像
            # 这里选择抛出异常
            raise

        if self.transform:
            image = self.transform(image)

        # 获取属性标签 
        attrs = self.attr_df.iloc[idx][self.attr_names].values
        attrs = (attrs + 1) // 2  # 将-1转为0，1保持1
        attrs = attrs.astype(np.float32)

        # 获取对应的图像嵌入
        clip_embedding = self.image_embeddings[idx]

        return image, attrs, clip_embedding

# ------------------------ #
# 4. 定义ClipCVAE模型（使用Renyi散度）
# ------------------------ #

class ClipCVAE_Renyi(nn.Module):
    def __init__(self, img_channels=3, img_size=64, latent_dim=128, 
                 cond_dim=40, clip_dim=512):
        super(ClipCVAE_Renyi, self).__init__()
        self.img_size = img_size
        self.latent_dim = latent_dim
        self.cond_dim = cond_dim
        self.clip_dim = clip_dim

        # 编码器部分
        self.encoder = nn.Sequential(
            nn.Conv2d(img_channels + cond_dim + clip_dim, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Flatten()
        )
        self.fc_mu = nn.Linear(512*4*4, latent_dim)
        self.fc_logvar = nn.Linear(512*4*4, latent_dim)

        # 解码器部分
        self.decoder_input = nn.Linear(latent_dim + cond_dim + clip_dim, 512*4*4)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, img_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def encode(self, x, c, clip_embedding):
        # 将条件标签、图像和CLIP嵌入拼接
        c = c.view(c.size(0), self.cond_dim, 1, 1).repeat(1, 1, self.img_size, self.img_size)
        clip_embedding = clip_embedding.view(clip_embedding.size(0), self.clip_dim, 1, 1).repeat(1, 1, self.img_size, self.img_size)
        x = torch.cat([x, c, clip_embedding], dim=1)
        x = self.encoder(x)
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z, c, clip_embedding):
        # 将潜在变量、条件标签和CLIP嵌入拼接
        z = torch.cat([z, c, clip_embedding], dim=1)
        x = self.decoder_input(z)
        x = x.view(-1, 512, 4, 4)
        x = self.decoder(x)
        return x

    def forward(self, x, c, clip_embedding):
        mu, logvar = self.encode(x, c, clip_embedding)
        z = self.reparameterize(mu, logvar)
        recon_x = self.decode(z, c, clip_embedding)
        return recon_x, mu, logvar

# ------------------------ #
# 5. 图像预处理
# ------------------------ #

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # 归一化
])

# ------------------------ #
# 6. 数据集路径设置
# ------------------------ #

# 请根据实际路径修改
img_dir = '/root/autodl-tmp/celeba_datasets/img_align_celeba/img_align_celeba'
attr_path = '/root/autodl-tmp/celeba_datasets/list_attr_celeba.txt'
bbox_path = '/root/autodl-tmp/celeba_datasets/list_bbox_celeba.txt'
partition_path = '/root/autodl-tmp/celeba_datasets/list_eval_partition.txt'
image_embeddings_train_path = '/root/autodl-tmp/clip_embeddings/image_embeddings_partition_0.pt'
image_embeddings_val_path = '/root/autodl-tmp/clip_embeddings/image_embeddings_partition_1.pt'

# ------------------------ #
# 7. 创建训练集和验证集
# ------------------------ #

train_dataset = CelebADataset(img_dir, attr_path, bbox_path, partition_path, 
                              image_embeddings_train_path, transform=transform, partition=0)
val_dataset = CelebADataset(img_dir, attr_path, bbox_path, partition_path, 
                            image_embeddings_val_path, transform=transform, partition=1)

# ------------------------ #
# 8. 数据加载器
# ------------------------ #

batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# ------------------------ #
# 9. 修订版 Rényi 散度损失函数
# ------------------------ #

import torch
import torch.nn.functional as F
import math

def negative_alpha_renyi_divergence_loss(recon_x, x, mu, logvar, epoch=1, alpha=-0.5, K=1):
    """
    使用负α值的Rényi散度损失函数，基于变分Rényi边界理论
    
    参数:
    - recon_x: 重构的图像
    - x: 原始图像
    - mu: 均值向量
    - logvar: 对数方差向量
    - epoch: 当前训练轮数
    - alpha: Rényi散度参数，这里使用负值
    - K: 蒙特卡洛样本数量（通常较小）
    """
    # 重构损失计算
    batch_size = x.size(0)
    recon_loss = F.mse_loss(recon_x, x, reduction='mean') * 100
    
    # 限制参数范围，对负α稍微放宽
    logvar = torch.clamp(logvar, min=-15.0, max=15.0)
    var = torch.exp(logvar)
    
    # 标准正态先验
    var0 = torch.ones_like(var)
    mu0 = torch.zeros_like(mu)
    
    # 为负α计算修正的Rényi散度
    # 注意：负α的处理需要特别考虑数值稳定性
    
    # 当α为负值时的变分Rényi散度计算
    # 根据变分Rényi边界理论，我们需要对传统公式进行调整
    
    # 计算KL散度的成分
    mu_diff_sq = (mu - mu0).pow(2)
    trace_term = var / var0
    logdet_term = torch.log(var0) - logvar
    
    # 基础KL散度项
    kl_base = 0.5 * (mu_diff_sq / var0 + trace_term + logdet_term - 1)
    
    # 为负α值调整的Rényi散度计算
    # 当α<0时，使用重参数化技巧进行MC采样
    mc_samples = []
    for _ in range(K):
        eps = torch.randn_like(std := torch.sqrt(var))
        z = mu + eps * std
        log_q = -0.5 * (torch.log(2 * math.pi * var) + (z - mu).pow(2) / var)
        log_p = -0.5 * (torch.log(2 * math.pi * var0) + (z - mu0).pow(2) / var0)
        mc_samples.append(log_p - log_q)
    
    # 计算MC样本的负α平均
    log_weights = torch.stack(mc_samples, dim=0)
    if alpha < 0:
        # 负α时的特殊处理
        renyi_div = (1 / (alpha - 1)) * torch.logsumexp((1 - alpha) * log_weights, dim=0).mean()
    else:
        # 兼容正α
        renyi_div = (1 / (alpha - 1)) * torch.log(torch.mean(torch.exp((1 - alpha) * log_weights), dim=0)).mean()
    
    # 确保数值稳定性
    renyi_div = torch.clamp(renyi_div, min=-10.0, max=10.0)
    
    # 动态权重，随训练进程调整
    # 对于负α，初始权重可以设置大一些，因为它提供更紧的下界
    max_weight = 5.0
    min_weight = 1.0
    
    # 使用修改后的曲线增长来适应负α
    progress = min(epoch / 10.0, 1.0)  # 加快权重调整的速度
    weight_factor = min_weight + (max_weight - min_weight) * (1 - math.exp(-5 * progress))
    
    # 对于负α，我们可能不再需要额外的KL散度正则化
    # 因为负α本身已经能够提供良好的正则化效果
    
    # 监控信息
    raw_renyi = renyi_div.item()
    raw_kl = kl_base.sum(dim=1).mean().item()
    
    # 总损失 - 对于负α，可以适当减小重构损失的权重
    weighted_renyi = renyi_div * weight_factor
    total_loss = recon_loss + weighted_renyi
    
    return total_loss, recon_loss, weighted_renyi, raw_renyi, torch.tensor(0.0).to(recon_x.device), raw_kl
# ------------------------ #
# 10. 改进的训练函数
# ------------------------ #

def train_negative_alpha_model(model, train_loader, val_loader, optimizer, scheduler, num_epochs=30, alpha=-0.5):
    """
    使用负α的Rényi散度训练CVAE模型
    """
    print(f"开始使用负α值(α={alpha})的Rényi散度训练CVAE模型...")
    
    # 记录训练统计信息
    train_stats = {
        'total_loss': [], 'recon_loss': [], 'renyi_loss': [], 
        'raw_renyi': [], 'kl_loss': [], 'raw_kl': []
    }
    val_stats = {
        'total_loss': [], 'recon_loss': [], 'renyi_loss': [], 
        'raw_renyi': [], 'kl_loss': [], 'raw_kl': []
    }
    
    # 最佳模型跟踪
    best_val_loss = float('inf')
    patience = 10
    patience_counter = 0
    
    # 确保保存目录存在
    os.makedirs('model_checkpoints', exist_ok=True)
    
    for epoch in range(1, num_epochs + 1):
        model.train()
        epoch_stats = {k: 0.0 for k in train_stats.keys()}
        nan_batches = 0
        
        # 计算当前权重
        progress = min(epoch / 10.0, 1.0)
        min_weight = 1.0
        max_weight = 5.0
        weight_factor = min_weight + (max_weight - min_weight) * (1 - math.exp(-5 * progress))
        
        print(f"Epoch {epoch}/{num_epochs}, 负α Rényi权重: {weight_factor:.4f}")
        
        # 训练循环
        for batch_idx, (data, attrs, clip_emb) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}")):
            data = data.to(device, non_blocking=True)
            attrs = attrs.to(device, non_blocking=True)
            clip_emb = clip_emb.to(device, non_blocking=True)
            
            optimizer.zero_grad()
            
            try:
                recon_batch, mu, logvar = model(data, attrs, clip_emb)
                
                # 使用负α Rényi散度损失函数
                loss, recon_loss, renyi_loss, raw_renyi, kl_loss, raw_kl = negative_alpha_renyi_divergence_loss(
                    recon_batch, data, mu, logvar, epoch, alpha=alpha, K=5  # 使用5个MC样本
                )
                
                # 详细调试输出
                if batch_idx % 100 == 0:
                    print(f"\n批次 {batch_idx} 详细信息:")
                    print(f"  - 原始负α Rényi值: {raw_renyi:.6f}, 加权Rényi损失: {renyi_loss.item():.6f}")
                    print(f"  - 原始KL值: {raw_kl:.6f}")
                    print(f"  - 重构损失: {recon_loss.item():.6f}, 总损失: {loss.item():.6f}")
                    print(f"  - mu范围: [{mu.min().item():.4f}, {mu.max().item():.4f}], "
                          f"var范围: [{torch.exp(logvar).min().item():.6f}, {torch.exp(logvar).max().item():.6f}]")
                    
                    # 计算VR边界和MC样本的统计信息
                    with torch.no_grad():
                        eps = torch.randn_like(std := torch.sqrt(torch.exp(logvar)))
                        z_samples = mu + eps * std
                        vr_bound = negative_alpha_renyi_divergence_loss(
                            recon_batch, data, mu, logvar, epoch, alpha=alpha, K=100  # 更多样本用于比较
                        )[2].item() / weight_factor
                        
                        print(f"  - VR边界(K=100): {vr_bound:.6f}, MC偏差估计: {vr_bound - raw_renyi:.6f}")
                
                # 检查NaN
                if torch.isnan(loss).any() or torch.isinf(loss).any():
                    nan_batches += 1
                    if batch_idx % 100 == 0:
                        print(f"警告: 第{epoch}轮, 第{batch_idx}批次出现NaN/Inf损失。跳过此批次.")
                    continue
                
                # 反向传播和优化
                loss.backward()
                # 更严格的梯度裁剪，因为负α可能导致更大的梯度
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()
                
                # 累计统计信息
                epoch_stats['total_loss'] += loss.item()
                epoch_stats['recon_loss'] += recon_loss.item()
                epoch_stats['renyi_loss'] += renyi_loss.item()
                epoch_stats['raw_renyi'] += raw_renyi
                epoch_stats['kl_loss'] += kl_loss.item()
                epoch_stats['raw_kl'] += raw_kl
                
            except RuntimeError as e:
                print(f"批次{batch_idx}处理出错: {e}")
                nan_batches += 1
                continue
        
        # 更新学习率
        scheduler.step()
        current_lr = optimizer.param_groups[0]["lr"]
        
        # 计算平均损失
        processed_batches = len(train_loader) - nan_batches
        if processed_batches > 0:
            for k in epoch_stats:
                epoch_stats[k] /= processed_batches
                train_stats[k].append(epoch_stats[k])
            
            print(f"\nEpoch {epoch} 训练统计:")
            print(f"  - 平均总损失: {epoch_stats['total_loss']:.4f}")
            print(f"  - 重构损失: {epoch_stats['recon_loss']:.4f}")
            print(f"  - 负α Rényi损失: {epoch_stats['renyi_loss']:.4f} (原始值: {epoch_stats['raw_renyi']:.6f})")
            print(f"  - 原始KL值: {epoch_stats['raw_kl']:.6f}")
            print(f"  - 学习率: {current_lr:.6f}, NaN批次: {nan_batches}")
        else:
            print(f"Epoch {epoch}, 所有批次均为NaN，无法计算平均损失")
            continue
        
        # 验证集评估
        model.eval()
        val_epoch_stats = {k: 0.0 for k in val_stats.keys()}
        val_nan_batches = 0
        
        with torch.no_grad():
            for data, attrs, clip_emb in val_loader:
                data = data.to(device, non_blocking=True)
                attrs = attrs.to(device, non_blocking=True)
                clip_emb = clip_emb.to(device, non_blocking=True)
                
                try:
                    recon_batch, mu, logvar = model(data, attrs, clip_emb)
                    loss, recon_loss, renyi_loss, raw_renyi, kl_loss, raw_kl = negative_alpha_renyi_divergence_loss(
                        recon_batch, data, mu, logvar, epoch, alpha=alpha, K=5
                    )
                    
                    if torch.isnan(loss).any() or torch.isinf(loss).any():
                        val_nan_batches += 1
                        continue
                    
                    # 累计验证统计
                    val_epoch_stats['total_loss'] += loss.item()
                    val_epoch_stats['recon_loss'] += recon_loss.item()
                    val_epoch_stats['renyi_loss'] += renyi_loss.item()
                    val_epoch_stats['raw_renyi'] += raw_renyi
                    val_epoch_stats['kl_loss'] += kl_loss.item()
                    val_epoch_stats['raw_kl'] += raw_kl
                    
                except RuntimeError:
                    val_nan_batches += 1
                    continue
        
        # 计算验证平均值
        processed_val_batches = len(val_loader) - val_nan_batches
        if processed_val_batches > 0:
            for k in val_epoch_stats:
                val_epoch_stats[k] /= processed_val_batches
                val_stats[k].append(val_epoch_stats[k])
            
            print(f"验证集统计:")
            print(f"  - 平均总损失: {val_epoch_stats['total_loss']:.4f}")
            print(f"  - 重构损失: {val_epoch_stats['recon_loss']:.4f}")
            print(f"  - 负α Rényi损失: {val_epoch_stats['renyi_loss']:.4f} (原始值: {val_epoch_stats['raw_renyi']:.6f})")
            print(f"  - 原始KL值: {val_epoch_stats['raw_kl']:.6f}")
            print(f"  - NaN批次: {val_nan_batches}")
            
            # 保存当前模型和检查最佳模型
            current_model_path = f'model_checkpoints/cvae_renyi_negative_alpha_epoch_{epoch}.pth'
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_stats': train_stats,
                'val_stats': val_stats,
                'alpha': alpha,
            }, current_model_path)
            
            # 负α的模型质量评估 - 主要关注重构损失
            model_quality = val_epoch_stats['recon_loss']
            
            if model_quality < best_val_loss:
                best_val_loss = model_quality
                patience_counter = 0
                
                best_model_path = f'model_checkpoints/cvae_renyi_negative_alpha_best.pth'
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'best_quality': model_quality,
                    'train_stats': train_stats,
                    'val_stats': val_stats,
                    'alpha': alpha,
                }, best_model_path)
                print(f"保存最佳模型: {best_model_path} (质量指标: {model_quality:.4f})")
            else:
                patience_counter += 1
                print(f"模型质量未改善，耐心计数: {patience_counter}/{patience}")
                if patience_counter >= patience:
                    print(f"提前停止训练！模型质量{patience}轮未改善。")
                    return model, train_stats, val_stats
        else:
            print(f"验证集所有批次均为NaN，无法计算平均损失")
    
    return model, train_stats, val_stats
# ------------------------ #
# 11. 可视化函数
# ------------------------ #

def plot_training_stats(train_stats, val_stats, save_path='training_stats.png'):
    """
    绘制训练过程中的各种统计数据
    """
    import matplotlib.pyplot as plt
    
    epochs = range(1, len(train_stats['total_loss']) + 1)
    
    plt.figure(figsize=(18, 12))
    
    # 绘制总损失
    plt.subplot(2, 3, 1)
    plt.plot(epochs, train_stats['total_loss'], 'b-', label='训练')
    plt.plot(epochs, val_stats['total_loss'], 'r-', label='验证')
    plt.title('总损失')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # 绘制重构损失
    plt.subplot(2, 3, 2)
    plt.plot(epochs, train_stats['recon_loss'], 'b-', label='训练')
    plt.plot(epochs, val_stats['recon_loss'], 'r-', label='验证')
    plt.title('重构损失')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # 绘制Rényi损失
    plt.subplot(2, 3, 3)
    plt.plot(epochs, train_stats['renyi_loss'], 'b-', label='训练')
    plt.plot(epochs, val_stats['renyi_loss'], 'r-', label='验证')
    plt.title('Rényi损失 (加权)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # 绘制原始Rényi值
    plt.subplot(2, 3, 4)
    plt.plot(epochs, train_stats['raw_renyi'], 'b-', label='训练')
    plt.plot(epochs, val_stats['raw_renyi'], 'r-', label='验证')
    plt.title('原始Rényi值')
    plt.xlabel('Epoch')
    plt.ylabel('Value')
    plt.legend()
    
    # 绘制KL损失
    plt.subplot(2, 3, 5)
    plt.plot(epochs, train_stats['kl_loss'], 'b-', label='训练')
    plt.plot(epochs, val_stats['kl_loss'], 'r-', label='验证')
    plt.title('KL损失 (加权)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # 绘制原始KL值
    plt.subplot(2, 3, 6)
    plt.plot(epochs, train_stats['raw_kl'], 'b-', label='训练')
    plt.plot(epochs, val_stats['raw_kl'], 'r-', label='验证')
    plt.title('原始KL值')
    plt.xlabel('Epoch')
    plt.ylabel('Value')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()
    print(f"训练统计图已保存至 {save_path}")

# ------------------------ #
# 12. 样本生成与可视化
# ------------------------ #

def generate_samples(model, val_loader, num_samples=10, save_dir='sample_images'):
    """
    生成样本并可视化比较
    """
    import matplotlib.pyplot as plt
    from torchvision.utils import make_grid
    
    os.makedirs(save_dir, exist_ok=True)
    
    model.eval()
    
    # 随机获取一些验证集样本
    data_iter = iter(val_loader)
    data, attrs, clip_emb = next(data_iter)
    
    data = data[:num_samples].to(device)
    attrs = attrs[:num_samples].to(device)
    clip_emb = clip_emb[:num_samples].to(device)
    
    with torch.no_grad():
        # 获取原始图像、重构图像
        recon_img, mu, logvar = model(data, attrs, clip_emb)
        
        # 从潜在空间采样新图像
        z = torch.randn_like(mu)
        sampled_img = model.decode(z, attrs, clip_emb)
    
    # 将张量转换为可视化的图像
    # 去归一化，从[-1, 1]到[0, 1]
    def denormalize(x):
        return (x + 1) / 2
    
    original_imgs = denormalize(data)
    recon_imgs = denormalize(recon_img)
    sampled_imgs = denormalize(sampled_img)
    
    # 显示原始、重构和采样的图像
    fig, axes = plt.subplots(3, num_samples, figsize=(num_samples * 2, 6))
    
    # 去除坐标轴
    for ax in axes.flatten():
        ax.axis('off')
    
    # 标题
    axes[0, num_samples//2].set_title('原始图像', fontsize=12, pad=10)
    axes[1, num_samples//2].set_title('重构图像', fontsize=12, pad=10)
    axes[2, num_samples//2].set_title('从潜在空间采样', fontsize=12, pad=10)
    
    # 显示图像
    for i in range(num_samples):
        # 原始图像
        img = original_imgs[i].cpu().permute(1, 2, 0).numpy()
        img = np.clip(img, 0, 1)
        axes[0, i].imshow(img)
        
        # 重构图像
        img = recon_imgs[i].cpu().permute(1, 2, 0).numpy()
        img = np.clip(img, 0, 1)
        axes[1, i].imshow(img)
        
        # 采样图像
        img = sampled_imgs[i].cpu().permute(1, 2, 0).numpy()
        img = np.clip(img, 0, 1)
        axes[2, i].imshow(img)
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, 'sample_comparison.png'), dpi=150)
    plt.close()
    
    print(f"样本可视化已保存至 {os.path.join(save_dir, 'sample_comparison.png')}")
    
    return {
        'original': original_imgs.cpu(),
        'recon': recon_imgs.cpu(),
        'sampled': sampled_imgs.cpu()
    }

# ------------------------ #
# 13. 主程序
# ------------------------ #
if __name__ == "__main__":
    try:
        # 设置随机种子确保可重复性
        torch.manual_seed(42)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(42)
        np.random.seed(42)
        random.seed(42)
        
        # 设置负α值
        alpha = -0.5
        
        # 创建模型实例
        model_renyi_neg = ClipCVAE_Renyi(img_channels=3, img_size=64, latent_dim=128, 
                                   cond_dim=40, clip_dim=512).to(device)
        
        # 初始化权重
        for name, param in model_renyi_neg.named_parameters():
            if 'weight' in name:
                if 'fc_mu' in name or 'fc_logvar' in name:
                    # 对于负α，可以使用稍小的初始化以增加稳定性
                    nn.init.normal_(param, mean=0.0, std=0.005)
                elif param.dim() >= 2:
                    nn.init.kaiming_normal_(param, mode='fan_out', nonlinearity='relu')
                else:
                    nn.init.normal_(param, mean=0.0, std=0.01)
            elif 'bias' in name:
                nn.init.zeros_(param)
        
        print(f"已成功完成模型参数初始化，使用负α值(α={alpha})的Rényi散度计算方法")
        
        # 优化器 - 对于负α使用更小的学习率和更大的权重衰减
        optimizer = optim.AdamW(model_renyi_neg.parameters(), lr=5e-5, weight_decay=1e-3)
        
        # 调度器 - 使用余弦退火但最小值较高
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30, eta_min=1e-5)
        
        # 训练模型
        trained_model, train_stats, val_stats = train_negative_alpha_model(
            model_renyi_neg, train_loader, val_loader, optimizer, scheduler, 
            num_epochs=30, alpha=alpha
        )
        
        # 保存最终模型
        final_model_path = f'model_checkpoints/cvae_renyi_alpha{alpha}_final.pth'
        torch.save({
            'model_state_dict': trained_model.state_dict(),
            'train_stats': train_stats,
            'val_stats': val_stats,
            'alpha': alpha
        }, final_model_path)
        print(f"负α值(α={alpha})的Rényi散度CVAE模型已保存为: {final_model_path}")
        
        # 绘制训练过程中的损失变化图表
        plot_training_stats(train_stats, val_stats, save_path=f'model_checkpoints/training_stats_alpha{alpha}.png')
        
        # 生成样本并可视化
        sample_results = generate_samples(
            trained_model, val_loader, num_samples=8, 
            save_dir=f'model_checkpoints/sample_images_alpha{alpha}'
        )
        
        print(f"使用负α值(α={alpha})的Rényi散度CVAE模型训练完成！")
        
    except Exception as e:
        print(f"训练过程中发生错误: {e}")
        import traceback
        traceback.print_exc()
        # 保存紧急状态
        emergency_save_path = f'model_checkpoints/cvae_renyi_negative_alpha_emergency_save.pth'
        torch.save(model_renyi_neg.state_dict(), emergency_save_path)
        print(f"发生错误，模型紧急保存为: {emergency_save_path}")

使用设备: cuda
CLIP模型已加载。
Attributes DataFrame columns: ['image_id', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive', 'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose', 'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows', 'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace', 'Wearing_Necktie', 'Young']
Attributes DataFrame head:
      image_id  5_o_Clock_Shadow  Arched_Eyebrows  Attractive  Bags_Under_Eyes  \
0  000001.jpg                -1                1           1               -1   
1  000002.jpg                -1               -1          -1                1   
2  000003.jpg                -1               -1          -1               -1   

Epoch 1/30:   0%|          | 5/1272 [00:00<03:14,  6.53it/s]


批次 0 详细信息:
  - 原始负α Rényi值: -1.072954, 加权Rényi损失: -2.761651
  - 原始KL值: 0.000691
  - 重构损失: 36.100555, 总损失: 33.338905
  - mu范围: [-0.0130, 0.0140], var范围: [0.984246, 1.012372]
  - VR边界(K=100): -3.070112, MC偏差估计: -1.997158


Epoch 1/30:   8%|▊         | 101/1272 [00:05<00:57, 20.47it/s]


批次 100 详细信息:
  - 原始负α Rényi值: -1.072959, 加权Rényi损失: -2.761666
  - 原始KL值: 0.016280
  - 重构损失: 34.457294, 总损失: 31.695629
  - mu范围: [-0.0536, 0.0695], var范围: [0.919736, 1.044981]
  - VR边界(K=100): -3.070176, MC偏差估计: -1.997216


...

验证集统计:
  - 平均总损失: -0.4708
  - 重构损失: 3.9249
  - 负α Rényi损失: -4.3957 (原始值: -0.883903)
  - 原始KL值: 94.172303
  - NaN批次: 0
保存最佳模型: model_checkpoints/cvae_renyi_negative_alpha_best.pth (质量指标: 3.9249)
负α值(α=-0.5)的Rényi散度CVAE模型已保存为: model_checkpoints/cvae_renyi_alpha-0.5_final.pth


/tmp/ipykernel_14774/3868200464.py:638: UserWarning: Glyph 24635 (\N{CJK UNIFIED IDEOGRAPH-603B}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_14774/3868200464.py:638: UserWarning: Glyph 25439 (\N{CJK UNIFIED IDEOGRAPH-635F}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_14774/3868200464.py:638: UserWarning: Glyph 22833 (\N{CJK UNIFIED IDEOGRAPH-5931}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_14774/3868200464.py:638: UserWarning: Glyph 35757 (\N{CJK UNIFIED IDEOGRAPH-8BAD}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_14774/3868200464.py:638: UserWarning: Glyph 32451 (\N{CJK UNIFIED IDEOGRAPH-7EC3}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_14774/3868200464.py:638: UserWarning: Glyph 39564 (\N{CJK UNIFIED IDEOGRAPH-9A8C}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_14774/3868200464.py:638: UserWarning: Glyph 35777 (\N{CJK UNIFIED I

训练统计图已保存至 model_checkpoints/training_stats_alpha-0.5.png


/tmp/ipykernel_14774/3868200464.py:712: UserWarning: Glyph 21407 (\N{CJK UNIFIED IDEOGRAPH-539F}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_14774/3868200464.py:712: UserWarning: Glyph 22987 (\N{CJK UNIFIED IDEOGRAPH-59CB}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_14774/3868200464.py:712: UserWarning: Glyph 22270 (\N{CJK UNIFIED IDEOGRAPH-56FE}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_14774/3868200464.py:712: UserWarning: Glyph 20687 (\N{CJK UNIFIED IDEOGRAPH-50CF}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_14774/3868200464.py:712: UserWarning: Glyph 37325 (\N{CJK UNIFIED IDEOGRAPH-91CD}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_14774/3868200464.py:712: UserWarning: Glyph 26500 (\N{CJK UNIFIED IDEOGRAPH-6784}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_14774/3868200464.py:712: UserWarning: Glyph 20174 (\N{CJK UNIFIED I

样本可视化已保存至 model_checkpoints/sample_images_alpha-0.5/sample_comparison.png
使用负α值(α=-0.5)的Rényi散度CVAE模型训练完成！


### alpha = -1.0

In [ ]:
import os
import math
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import clip
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm

# ------------------------ #
# 1. 设备设置
# ------------------------ #

# 选择设备（GPU或CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")

# ------------------------ #
# 2. 加载CLIP模型
# ------------------------ #

# 加载CLIP模型和预处理方法
clip_model, preprocess = clip.load("ViT-B/32", device=device)
clip_model.eval()
print("CLIP模型已加载。")

# ------------------------ #
# 3. 定义CelebA数据集类
# ------------------------ #

class CelebADataset(Dataset):
    def __init__(self, img_dir, attr_path, bbox_path, partition_path, 
                 image_embeddings_path, transform=None, partition=0):
        """
        初始化数据集
        :param img_dir: 图像文件夹路径
        :param attr_path: 属性文件路径
        :param bbox_path: 边界框文件路径
        :param partition_path: 分区文件路径
        :param image_embeddings_path: 预先计算的图像嵌入文件路径
        :param transform: 图像预处理
        :param partition: 使用的数据分区 (0: train, 1: val, 2: test)
        """
        self.img_dir = img_dir
        self.transform = transform

        # 读取属性文件
        try:
            attr_df = pd.read_csv(attr_path, sep=',', header=0)
        except Exception as e:
            print(f"Error reading {attr_path}: {e}")
            raise

        print("Attributes DataFrame columns:", attr_df.columns.tolist())
        print("Attributes DataFrame head:\n", attr_df.head())

        # 确认 'image_id' 列存在
        if 'image_id' not in attr_df.columns:
            raise ValueError("属性文件中缺少 'image_id' 列。请检查文件格式。")
        else:
            print("'image_id' column found in attributes DataFrame.")

        # 读取分区文件
        try:
            partition_df = pd.read_csv(partition_path, sep=',', header=0, names=['image_id', 'partition'])
        except Exception as e:
            print(f"Error reading {partition_path}: {e}")
            raise

        print("Partition DataFrame columns:", partition_df.columns.tolist())
        print("Partition DataFrame head:\n", partition_df.head())

        # 读取边界框文件
        try:
            bbox_df = pd.read_csv(bbox_path, sep=',', header=0, names=['image_id', 'x_1', 'y_1', 'width', 'height'])
        except Exception as e:
            print(f"Error reading {bbox_path}: {e}")
            raise

        print("Bounding Box DataFrame columns:", bbox_df.columns.tolist())
        print("Bounding Box DataFrame head:\n", bbox_df.head())

        # 合并属性和分区数据
        try:
            attr_df = attr_df.merge(partition_df, on='image_id')
        except Exception as e:
            print(f"Error merging attributes and partitions: {e}")
            raise

        print("After merging with partition, columns:", attr_df.columns.tolist())
        print("Merged DataFrame head:\n", attr_df.head())

        # 合并属性、分区和边界框数据
        try:
            attr_df = attr_df.merge(bbox_df, on='image_id')
        except Exception as e:
            print(f"Error merging attributes, partitions and bounding boxes: {e}")
            raise

        print("After merging with bounding box, columns:", attr_df.columns.tolist())
        print("Final Merged DataFrame head:\n", attr_df.head())

        # 筛选分区
        print(f"Total samples before filtering: {len(attr_df)}")
        self.attr_df = attr_df[attr_df['partition'] == partition].reset_index(drop=True)
        print(f"Total samples after filtering partition {partition}: {len(self.attr_df)}")

        if len(self.attr_df) == 0:
            raise ValueError(f"训练集中没有样本。请检查数据文件的读取和过滤逻辑。")

        # 获取属性名称
        # 属性列是除了 'image_id', 'partition', 'x_1', 'y_1', 'width', 'height' 之外的所有列
        self.attr_names = [col for col in attr_df.columns if col not in ['image_id', 'partition', 'x_1', 'y_1', 'width', 'height']]
        print("Attribute names:", self.attr_names)

        # 加载预计算的图像嵌入
        self.image_embeddings = torch.load(image_embeddings_path)  # 假设是一个Tensor，shape=(num_images, 512)
        if len(self.image_embeddings) != len(self.attr_df):
            raise ValueError("图像嵌入的数量与数据集中的图像数量不一致。")
        self.image_embeddings = self.image_embeddings.float()

    def __len__(self):
        return len(self.attr_df)

    def __getitem__(self, idx):
        # 获取图像路径
        img_name = self.attr_df.iloc[idx, self.attr_df.columns.get_loc('image_id')]
        img_path = os.path.join(self.img_dir, img_name)
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Error opening image {img_path}: {e}")
            # 可以选择跳过该样本，或返回一个默认图像
            # 这里选择抛出异常
            raise

        if self.transform:
            image = self.transform(image)

        # 获取属性标签 
        attrs = self.attr_df.iloc[idx][self.attr_names].values
        attrs = (attrs + 1) // 2  # 将-1转为0，1保持1
        attrs = attrs.astype(np.float32)

        # 获取对应的图像嵌入
        clip_embedding = self.image_embeddings[idx]

        return image, attrs, clip_embedding

# ------------------------ #
# 4. 定义ClipCVAE模型（使用Renyi散度）
# ------------------------ #

class ClipCVAE_Renyi(nn.Module):
    def __init__(self, img_channels=3, img_size=64, latent_dim=128, 
                 cond_dim=40, clip_dim=512):
        super(ClipCVAE_Renyi, self).__init__()
        self.img_size = img_size
        self.latent_dim = latent_dim
        self.cond_dim = cond_dim
        self.clip_dim = clip_dim

        # 编码器部分
        self.encoder = nn.Sequential(
            nn.Conv2d(img_channels + cond_dim + clip_dim, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Flatten()
        )
        self.fc_mu = nn.Linear(512*4*4, latent_dim)
        self.fc_logvar = nn.Linear(512*4*4, latent_dim)

        # 解码器部分
        self.decoder_input = nn.Linear(latent_dim + cond_dim + clip_dim, 512*4*4)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, img_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def encode(self, x, c, clip_embedding):
        # 将条件标签、图像和CLIP嵌入拼接
        c = c.view(c.size(0), self.cond_dim, 1, 1).repeat(1, 1, self.img_size, self.img_size)
        clip_embedding = clip_embedding.view(clip_embedding.size(0), self.clip_dim, 1, 1).repeat(1, 1, self.img_size, self.img_size)
        x = torch.cat([x, c, clip_embedding], dim=1)
        x = self.encoder(x)
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z, c, clip_embedding):
        # 将潜在变量、条件标签和CLIP嵌入拼接
        z = torch.cat([z, c, clip_embedding], dim=1)
        x = self.decoder_input(z)
        x = x.view(-1, 512, 4, 4)
        x = self.decoder(x)
        return x

    def forward(self, x, c, clip_embedding):
        mu, logvar = self.encode(x, c, clip_embedding)
        z = self.reparameterize(mu, logvar)
        recon_x = self.decode(z, c, clip_embedding)
        return recon_x, mu, logvar

# ------------------------ #
# 5. 图像预处理
# ------------------------ #

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # 归一化
])

# ------------------------ #
# 6. 数据集路径设置
# ------------------------ #

# 请根据实际路径修改
img_dir = '/root/autodl-tmp/celeba_datasets/img_align_celeba/img_align_celeba'
attr_path = '/root/autodl-tmp/celeba_datasets/list_attr_celeba.txt'
bbox_path = '/root/autodl-tmp/celeba_datasets/list_bbox_celeba.txt'
partition_path = '/root/autodl-tmp/celeba_datasets/list_eval_partition.txt'
image_embeddings_train_path = '/root/autodl-tmp/clip_embeddings/image_embeddings_partition_0.pt'
image_embeddings_val_path = '/root/autodl-tmp/clip_embeddings/image_embeddings_partition_1.pt'

# ------------------------ #
# 7. 创建训练集和验证集
# ------------------------ #

train_dataset = CelebADataset(img_dir, attr_path, bbox_path, partition_path, 
                              image_embeddings_train_path, transform=transform, partition=0)
val_dataset = CelebADataset(img_dir, attr_path, bbox_path, partition_path, 
                            image_embeddings_val_path, transform=transform, partition=1)

# ------------------------ #
# 8. 数据加载器
# ------------------------ #

batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# ------------------------ #
# 9. 修订版 Rényi 散度损失函数
# ------------------------ #

import torch
import torch.nn.functional as F
import math

def negative_alpha_renyi_divergence_loss(recon_x, x, mu, logvar, epoch=1, alpha=-1.0, K=1):
    """
    使用α=-1的Rényi散度损失函数，基于变分Rényi边界理论
    
    参数:
    - recon_x: 重构的图像
    - x: 原始图像
    - mu: 均值向量
    - logvar: 对数方差向量
    - epoch: 当前训练轮数
    - alpha: Rényi散度参数，这里设为-1.0
    - K: 蒙特卡洛样本数量
    """
    # 重构损失计算
    batch_size = x.size(0)
    recon_loss = F.mse_loss(recon_x, x, reduction='mean') * 100
    
    # 对于α=-1，需要进一步放宽logvar的范围以防止数值不稳定
    logvar = torch.clamp(logvar, min=-12.0, max=12.0)
    var = torch.exp(logvar)
    
    # 标准正态先验
    var0 = torch.ones_like(var)
    mu0 = torch.zeros_like(mu)
    
    # 计算KL散度的成分（用于监控）
    mu_diff_sq = (mu - mu0).pow(2)
    trace_term = var / var0
    logdet_term = torch.log(var0) - logvar
    
    # 基础KL散度项（用于监控）
    kl_base = 0.5 * (mu_diff_sq / var0 + trace_term + logdet_term - 1)
    
    # α=-1时的Rényi散度计算需要额外的稳定性措施
    # 使用更多的MC样本来减少方差
    mc_samples = []
    for _ in range(max(K, 10)):  # 对于α=-1，确保至少使用10个样本
        eps = torch.randn_like(std := torch.sqrt(var))
        z = mu + eps * std
        
        # 计算log概率的差值，确保数值稳定性
        log_q = -0.5 * (torch.log(2 * math.pi * var) + (z - mu).pow(2) / var)
        log_p = -0.5 * (torch.log(2 * math.pi * var0) + (z - mu0).pow(2) / var0)
        log_ratio = log_p - log_q
        
        # 裁剪极端值
        log_ratio = torch.clamp(log_ratio, min=-20.0, max=20.0)
        mc_samples.append(log_ratio)
    
    # 计算MC样本的Rényi散度估计
    log_weights = torch.stack(mc_samples, dim=0)
    
    # α=-1时的特殊处理
    # 当α=-1时，(1-α)=2，这意味着我们需要计算exp(2*log_ratio)的平均值，可能导致数值溢出
    # 使用logsumexp技巧确保数值稳定性
    # 公式: 1/(α-1) * log(E[exp((1-α)*log(p/q))])
    # 对于α=-1: -0.5 * log(E[exp(2*log(p/q))])
    
    # 使用logsumexp实现数值稳定的计算
    renyi_div = (1 / (alpha - 1)) * torch.logsumexp((1 - alpha) * log_weights, dim=0).mean()
    
    # 确保数值稳定性
    renyi_div = torch.clamp(renyi_div, min=-15.0, max=15.0)
    
    # α=-1时的权重设置
    # 由于α值更负，VR边界可能提供更宽松的下界，需要更谨慎地调整权重
    max_weight = 3.0  # 降低最大权重以增加稳定性
    min_weight = 0.5  # 从较小的权重开始
    
    # 使用更缓慢的权重增长曲线
    progress = min(epoch / 15.0, 1.0)  # 延长过渡期
    weight_factor = min_weight + (max_weight - min_weight) * (1 - math.exp(-3 * progress))
    
    # 监控信息
    raw_renyi = renyi_div.item()
    raw_kl = kl_base.sum(dim=1).mean().item()
    
    # 总损失计算
    weighted_renyi = renyi_div * weight_factor
    
    # 对于α=-1，增强重构损失的权重，减少正则化影响
    # 这有助于防止过度正则化导致的模式崩溃
    total_loss = recon_loss + weighted_renyi
    
    # 额外添加MC样本统计信息以便于调试
    with torch.no_grad():
        mc_mean = log_weights.mean().item()
        mc_std = log_weights.std().item()
        mc_min = log_weights.min().item()
        mc_max = log_weights.max().item()
        print(f"MC样本统计: 均值={mc_mean:.4f}, 标准差={mc_std:.4f}, 最小值={mc_min:.4f}, 最大值={mc_max:.4f}")
    
    return total_loss, recon_loss, weighted_renyi, raw_renyi, torch.tensor(0.0).to(recon_x.device), raw_kl
# ------------------------ #
# 10. 改进的训练函数
# ------------------------ #
def train_negative_alpha_model(model, train_loader, val_loader, optimizer, scheduler, num_epochs=30, alpha=-1.0):
    """
    使用α=-1的Rényi散度训练CVAE模型
    """
    print(f"开始使用负α值(α={alpha})的Rényi散度训练CVAE模型...")
    
    # 记录训练统计信息
    train_stats = {
        'total_loss': [], 'recon_loss': [], 'renyi_loss': [], 
        'raw_renyi': [], 'kl_loss': [], 'raw_kl': [], 
        'mc_stats': []  # 添加MC样本统计跟踪
    }
    val_stats = {
        'total_loss': [], 'recon_loss': [], 'renyi_loss': [], 
        'raw_renyi': [], 'kl_loss': [], 'raw_kl': []
    }
    
    # 最佳模型跟踪
    best_val_loss = float('inf')
    patience = 15  # 增加耐心值，因为α=-1训练可能更加不稳定
    patience_counter = 0
    
    # 确保保存目录存在
    os.makedirs('model_checkpoints', exist_ok=True)
    
    # α=-1的特殊处理 - 执行梯度累积来稳定训练
    accumulation_steps = 4  # 每4个批次更新一次
    
    for epoch in range(1, num_epochs + 1):
        model.train()
        epoch_stats = {k: 0.0 for k in train_stats.keys() if k != 'mc_stats'}
        epoch_stats['mc_stats'] = []
        nan_batches = 0
        
        # 计算当前权重
        progress = min(epoch / 15.0, 1.0)
        min_weight = 0.5
        max_weight = 3.0
        weight_factor = min_weight + (max_weight - min_weight) * (1 - math.exp(-3 * progress))
        
        print(f"Epoch {epoch}/{num_epochs}, α={alpha} Rényi权重: {weight_factor:.4f}")
        
        # 训练循环
        for batch_idx, (data, attrs, clip_emb) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}")):
            data = data.to(device, non_blocking=True)
            attrs = attrs.to(device, non_blocking=True)
            clip_emb = clip_emb.to(device, non_blocking=True)
            
            # 只在累积步骤的开始时清零梯度
            if batch_idx % accumulation_steps == 0:
                optimizer.zero_grad()
            
            try:
                recon_batch, mu, logvar = model(data, attrs, clip_emb)
                
                # 使用α=-1的Rényi散度损失函数，增加样本数量以提高稳定性
                loss, recon_loss, renyi_loss, raw_renyi, kl_loss, raw_kl = negative_alpha_renyi_divergence_loss(
                    recon_batch, data, mu, logvar, epoch, alpha=alpha, K=10
                )
                
                # 按累积步骤缩放损失
                loss = loss / accumulation_steps
                
                # 详细调试输出
                if batch_idx % 50 == 0:  # 增加调试输出频率
                    print(f"\n批次 {batch_idx} 详细信息:")
                    print(f"  - 原始α={alpha} Rényi值: {raw_renyi:.6f}, 加权Rényi损失: {renyi_loss.item():.6f}")
                    print(f"  - 原始KL值: {raw_kl:.6f}")
                    print(f"  - 重构损失: {recon_loss.item():.6f}, 总损失: {loss.item()*accumulation_steps:.6f}")
                    print(f"  - mu范围: [{mu.min().item():.4f}, {mu.max().item():.4f}], "
                          f"var范围: [{torch.exp(logvar).min().item():.6f}, {torch.exp(logvar).max().item():.6f}]")
                    
                    # 特殊调试输出，用于MC样本质量评估
                    with torch.no_grad():
                        # 收集更大样本量下的统计数据
                        vr_estimates = []
                        for _ in range(5):  # 运行5次以获得更可靠的估计
                            vr_tmp = negative_alpha_renyi_divergence_loss(
                                recon_batch, data, mu, logvar, epoch, alpha=alpha, K=100
                            )[3]  # 获取原始Rényi值
                            vr_estimates.append(vr_tmp)
                        
                        vr_mean = sum(vr_estimates) / len(vr_estimates)
                        vr_std = (sum((x - vr_mean)**2 for x in vr_estimates) / len(vr_estimates))**0.5
                        
                        print(f"  - VR边界统计(K=100, 5次): 均值={vr_mean:.6f}, 标准差={vr_std:.6f}")
                        print(f"  - MC偏差估计: {vr_mean - raw_renyi:.6f}")
                
                # 检查NaN
                if torch.isnan(loss).any() or torch.isinf(loss).any():
                    nan_batches += 1
                    if batch_idx % 50 == 0:
                        print(f"警告: 第{epoch}轮, 第{batch_idx}批次出现NaN/Inf损失。跳过此批次.")
                    continue
                
                # 反向传播
                loss.backward()
                
                # 仅在累积步骤结束时更新权重
                if (batch_idx + 1) % accumulation_steps == 0 or (batch_idx + 1) == len(train_loader):
                    # 非常严格的梯度裁剪，因为α=-1可能导致极大的梯度
                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
                    optimizer.step()
                
                # 累计统计信息
                epoch_stats['total_loss'] += loss.item() * accumulation_steps
                epoch_stats['recon_loss'] += recon_loss.item()
                epoch_stats['renyi_loss'] += renyi_loss.item()
                epoch_stats['raw_renyi'] += raw_renyi
                epoch_stats['kl_loss'] += kl_loss.item()
                epoch_stats['raw_kl'] += raw_kl
                
                # 收集MC样本统计
                if batch_idx % 20 == 0:  # 每20个批次收集一次
                    mc_stat = {
                        'batch': batch_idx,
                        'raw_renyi': raw_renyi,
                        'raw_kl': raw_kl,
                        'mu_min': mu.min().item(),
                        'mu_max': mu.max().item(),
                        'var_min': torch.exp(logvar).min().item(),
                        'var_max': torch.exp(logvar).max().item(),
                    }
                    epoch_stats['mc_stats'].append(mc_stat)
                
            except RuntimeError as e:
                print(f"批次{batch_idx}处理出错: {e}")
                nan_batches += 1
                continue
        
        # 更新学习率
        scheduler.step()
        current_lr = optimizer.param_groups[0]["lr"]
        
        # 计算平均损失
        processed_batches = len(train_loader) - nan_batches
        if processed_batches > 0:
            for k in epoch_stats:
                if k != 'mc_stats':
                    epoch_stats[k] /= processed_batches
                    train_stats[k].append(epoch_stats[k])
            
            # 添加MC统计
            train_stats['mc_stats'].append(epoch_stats['mc_stats'])
            
            print(f"\nEpoch {epoch} 训练统计:")
            print(f"  - 平均总损失: {epoch_stats['total_loss']:.4f}")
            print(f"  - 重构损失: {epoch_stats['recon_loss']:.4f}")
            print(f"  - α={alpha} Rényi损失: {epoch_stats['renyi_loss']:.4f} (原始值: {epoch_stats['raw_renyi']:.6f})")
            print(f"  - 原始KL值: {epoch_stats['raw_kl']:.6f}")
            print(f"  - 学习率: {current_lr:.6f}, NaN批次: {nan_batches}")
            
            # 分析MC统计的趋势
            if len(epoch_stats['mc_stats']) > 0:
                raw_renyi_values = [stat['raw_renyi'] for stat in epoch_stats['mc_stats']]
                print(f"  - MC统计趋势: 最小Rényi={min(raw_renyi_values):.6f}, 最大Rényi={max(raw_renyi_values):.6f}")
        else:
            print(f"Epoch {epoch}, 所有批次均为NaN，无法计算平均损失")
            continue
        
        # 验证集评估
        model.eval()
        val_epoch_stats = {k: 0.0 for k in val_stats.keys()}
        val_nan_batches = 0
        
        with torch.no_grad():
            for data, attrs, clip_emb in val_loader:
                data = data.to(device, non_blocking=True)
                attrs = attrs.to(device, non_blocking=True)
                clip_emb = clip_emb.to(device, non_blocking=True)
                
                try:
                    recon_batch, mu, logvar = model(data, attrs, clip_emb)
                    loss, recon_loss, renyi_loss, raw_renyi, kl_loss, raw_kl = negative_alpha_renyi_divergence_loss(
                        recon_batch, data, mu, logvar, epoch, alpha=alpha, K=10
                    )
                    
                    if torch.isnan(loss).any() or torch.isinf(loss).any():
                        val_nan_batches += 1
                        continue
                    
                    # 累计验证统计
                    val_epoch_stats['total_loss'] += loss.item()
                    val_epoch_stats['recon_loss'] += recon_loss.item()
                    val_epoch_stats['renyi_loss'] += renyi_loss.item()
                    val_epoch_stats['raw_renyi'] += raw_renyi
                    val_epoch_stats['kl_loss'] += kl_loss.item()
                    val_epoch_stats['raw_kl'] += raw_kl
                    
                except RuntimeError:
                    val_nan_batches += 1
                    continue
        
        # 计算验证平均值
        processed_val_batches = len(val_loader) - val_nan_batches
        if processed_val_batches > 0:
            for k in val_epoch_stats:
                val_epoch_stats[k] /= processed_val_batches
                val_stats[k].append(val_epoch_stats[k])
            
            print(f"验证集统计:")
            print(f"  - 平均总损失: {val_epoch_stats['total_loss']:.4f}")
            print(f"  - 重构损失: {val_epoch_stats['recon_loss']:.4f}")
            print(f"  - α={alpha} Rényi损失: {val_epoch_stats['renyi_loss']:.4f} (原始值: {val_epoch_stats['raw_renyi']:.6f})")
            print(f"  - 原始KL值: {val_epoch_stats['raw_kl']:.6f}")
            print(f"  - NaN批次: {val_nan_batches}")
            
            # 保存当前模型
            current_model_path = f'model_checkpoints/cvae_renyi_alpha{alpha}_epoch_{epoch}.pth'
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_stats': train_stats,
                'val_stats': val_stats,
                'alpha': alpha,
            }, current_model_path)
            
            # 对于α=-1，使用重构损失和Rényi损失的组合作为质量指标
            # 对于更负的α，我们更关注重构质量
            model_quality = val_epoch_stats['recon_loss'] - 0.2 * val_epoch_stats['raw_renyi']
            
            if model_quality < best_val_loss:
                best_val_loss = model_quality
                patience_counter = 0
                
                best_model_path = f'model_checkpoints/cvae_renyi_alpha{alpha}_best.pth'
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'best_quality': model_quality,
                    'train_stats': train_stats,
                    'val_stats': val_stats,
                    'alpha': alpha,
                }, best_model_path)
                print(f"保存最佳模型: {best_model_path} (质量指标: {model_quality:.4f})")
            else:
                patience_counter += 1
                print(f"模型质量未改善，耐心计数: {patience_counter}/{patience}")
                if patience_counter >= patience:
                    print(f"提前停止训练！模型质量{patience}轮未改善。")
                    return model, train_stats, val_stats
        else:
            print(f"验证集所有批次均为NaN，无法计算平均损失")
    
    return model, train_stats, val_stats
# ------------------------ #
# 11. 可视化函数
# ------------------------ #

def plot_training_stats(train_stats, val_stats, save_path='training_stats.png'):
    """
    绘制训练过程中的各种统计数据
    """
    import matplotlib.pyplot as plt
    
    epochs = range(1, len(train_stats['total_loss']) + 1)
    
    plt.figure(figsize=(18, 12))
    
    # 绘制总损失
    plt.subplot(2, 3, 1)
    plt.plot(epochs, train_stats['total_loss'], 'b-', label='训练')
    plt.plot(epochs, val_stats['total_loss'], 'r-', label='验证')
    plt.title('总损失')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # 绘制重构损失
    plt.subplot(2, 3, 2)
    plt.plot(epochs, train_stats['recon_loss'], 'b-', label='训练')
    plt.plot(epochs, val_stats['recon_loss'], 'r-', label='验证')
    plt.title('重构损失')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # 绘制Rényi损失
    plt.subplot(2, 3, 3)
    plt.plot(epochs, train_stats['renyi_loss'], 'b-', label='训练')
    plt.plot(epochs, val_stats['renyi_loss'], 'r-', label='验证')
    plt.title('Rényi损失 (加权)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # 绘制原始Rényi值
    plt.subplot(2, 3, 4)
    plt.plot(epochs, train_stats['raw_renyi'], 'b-', label='训练')
    plt.plot(epochs, val_stats['raw_renyi'], 'r-', label='验证')
    plt.title('原始Rényi值')
    plt.xlabel('Epoch')
    plt.ylabel('Value')
    plt.legend()
    
    # 绘制KL损失
    plt.subplot(2, 3, 5)
    plt.plot(epochs, train_stats['kl_loss'], 'b-', label='训练')
    plt.plot(epochs, val_stats['kl_loss'], 'r-', label='验证')
    plt.title('KL损失 (加权)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # 绘制原始KL值
    plt.subplot(2, 3, 6)
    plt.plot(epochs, train_stats['raw_kl'], 'b-', label='训练')
    plt.plot(epochs, val_stats['raw_kl'], 'r-', label='验证')
    plt.title('原始KL值')
    plt.xlabel('Epoch')
    plt.ylabel('Value')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()
    print(f"训练统计图已保存至 {save_path}")

# ------------------------ #
# 12. 样本生成与可视化
# ------------------------ #

def generate_samples(model, val_loader, num_samples=10, save_dir='sample_images'):
    """
    生成样本并可视化比较
    """
    import matplotlib.pyplot as plt
    from torchvision.utils import make_grid
    
    os.makedirs(save_dir, exist_ok=True)
    
    model.eval()
    
    # 随机获取一些验证集样本
    data_iter = iter(val_loader)
    data, attrs, clip_emb = next(data_iter)
    
    data = data[:num_samples].to(device)
    attrs = attrs[:num_samples].to(device)
    clip_emb = clip_emb[:num_samples].to(device)
    
    with torch.no_grad():
        # 获取原始图像、重构图像
        recon_img, mu, logvar = model(data, attrs, clip_emb)
        
        # 从潜在空间采样新图像
        z = torch.randn_like(mu)
        sampled_img = model.decode(z, attrs, clip_emb)
    
    # 将张量转换为可视化的图像
    # 去归一化，从[-1, 1]到[0, 1]
    def denormalize(x):
        return (x + 1) / 2
    
    original_imgs = denormalize(data)
    recon_imgs = denormalize(recon_img)
    sampled_imgs = denormalize(sampled_img)
    
    # 显示原始、重构和采样的图像
    fig, axes = plt.subplots(3, num_samples, figsize=(num_samples * 2, 6))
    
    # 去除坐标轴
    for ax in axes.flatten():
        ax.axis('off')
    
    # 标题
    axes[0, num_samples//2].set_title('原始图像', fontsize=12, pad=10)
    axes[1, num_samples//2].set_title('重构图像', fontsize=12, pad=10)
    axes[2, num_samples//2].set_title('从潜在空间采样', fontsize=12, pad=10)
    
    # 显示图像
    for i in range(num_samples):
        # 原始图像
        img = original_imgs[i].cpu().permute(1, 2, 0).numpy()
        img = np.clip(img, 0, 1)
        axes[0, i].imshow(img)
        
        # 重构图像
        img = recon_imgs[i].cpu().permute(1, 2, 0).numpy()
        img = np.clip(img, 0, 1)
        axes[1, i].imshow(img)
        
        # 采样图像
        img = sampled_imgs[i].cpu().permute(1, 2, 0).numpy()
        img = np.clip(img, 0, 1)
        axes[2, i].imshow(img)
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, 'sample_comparison.png'), dpi=150)
    plt.close()
    
    print(f"样本可视化已保存至 {os.path.join(save_dir, 'sample_comparison.png')}")
    
    return {
        'original': original_imgs.cpu(),
        'recon': recon_imgs.cpu(),
        'sampled': sampled_imgs.cpu()
    }

# ------------------------ #
# 13. 主程序
# ------------------------ #
if __name__ == "__main__":
    try:
        # 设置随机种子确保可重复性
        torch.manual_seed(42)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(42)
        np.random.seed(42)
        random.seed(42)
        
        # 设置α=-1
        alpha = -1.0
        
        # 创建模型实例
        model_renyi_neg = ClipCVAE_Renyi(img_channels=3, img_size=64, latent_dim=128, 
                                   cond_dim=40, clip_dim=512).to(device)
        
        # 对于α=-1，使用更保守的初始化
        for name, param in model_renyi_neg.named_parameters():
            if 'weight' in name:
                if 'fc_mu' in name or 'fc_logvar' in name:
                    # 使用更小的初始值初始化均值和方差层
                    nn.init.normal_(param, mean=0.0, std=0.001)
                elif param.dim() >= 2:
                    # 使用较小的初始化范围
                    nn.init.kaiming_normal_(param, mode='fan_in', nonlinearity='relu', a=0.1)
                else:
                    nn.init.normal_(param, mean=0.0, std=0.005)
            elif 'bias' in name:
                nn.init.zeros_(param)
        
        print(f"已成功完成模型参数初始化，使用α={alpha}的Rényi散度计算方法")
        
        # 对于α=-1，使用更小的学习率、更大的权重衰减和更保守的学习率调度
        optimizer = optim.AdamW(model_renyi_neg.parameters(), lr=1e-5, weight_decay=1e-2, eps=1e-5)
        
        # 使用更平缓的学习率调度
        scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer, T_0=5, T_mult=2, eta_min=1e-6
        )
        
        # 训练模型
        trained_model, train_stats, val_stats = train_negative_alpha_model(
            model_renyi_neg, train_loader, val_loader, optimizer, scheduler, 
            num_epochs=30, alpha=alpha
        )
        
        # 保存最终模型
        final_model_path = f'model_checkpoints/cvae_renyi_alpha{alpha}_final.pth'
        torch.save({
            'model_state_dict': trained_model.state_dict(),
            'train_stats': train_stats,
            'val_stats': val_stats,
            'alpha': alpha
        }, final_model_path)
        print(f"α={alpha}的Rényi散度CVAE模型已保存为: {final_model_path}")
        
        # 绘制训练过程中的损失变化图表
        plot_training_stats(train_stats, val_stats, save_path=f'model_checkpoints/training_stats_alpha{alpha}.png')
        
        # 生成样本并可视化
        sample_results = generate_samples(
            trained_model, val_loader, num_samples=8, 
            save_dir=f'model_checkpoints/sample_images_alpha{alpha}'
        )
        
        print(f"使用α={alpha}的Rényi散度CVAE模型训练完成！")
        
    except Exception as e:
        print(f"训练过程中发生错误: {e}")
        import traceback
        traceback.print_exc()
        # 保存紧急状态
        emergency_save_path = f'model_checkpoints/cvae_renyi_alpha{alpha}_emergency_save.pth'
        torch.save(model_renyi_neg.state_dict(), emergency_save_path)
        print(f"发生错误，模型紧急保存为: {emergency_save_path}")

使用设备: cuda
CLIP模型已加载。
Attributes DataFrame columns: ['image_id', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive', 'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose', 'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows', 'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace', 'Wearing_Necktie', 'Young']
Attributes DataFrame head:
      image_id  5_o_Clock_Shadow  Arched_Eyebrows  Attractive  Bags_Under_Eyes  \
0  000001.jpg                -1                1           1               -1   
1  000002.jpg                -1               -1          -1                1   
2  000003.jpg                -1               -1          -1               -1   

Epoch 1/30:   0%|          | 1/1272 [00:01<26:45,  1.26s/it]

MC样本统计: 均值=-0.0000, 标准差=0.0003, 最小值=-0.0041, 最大值=0.0043

批次 0 详细信息:
  - 原始α=-1.0 Rényi值: -1.151292, 加权Rényi损失: -1.097381
  - 原始KL值: 0.000005
  - 重构损失: 36.120869, 总损失: 35.023487
  - mu范围: [-0.0010, 0.0012], var范围: [0.998716, 1.001016]
MC样本统计: 均值=-0.0000, 标准差=0.0003, 最小值=-0.0087, 最大值=0.0056
MC样本统计: 均值=0.0000, 标准差=0.0003, 最小值=-0.0065, 最大值=0.0066
MC样本统计: 均值=0.0000, 标准差=0.0003, 最小值=-0.0070, 最大值=0.0089
MC样本统计: 均值=-0.0000, 标准差=0.0003, 最小值=-0.0071, 最大值=0.0064
MC样本统计: 均值=0.0000, 标准差=0.0003, 最小值=-0.0073, 最大值=0.0064
  - VR边界统计(K=100, 5次): 均值=-2.302585, 标准差=0.000000
  - MC偏差估计: -1.151293


Epoch 1/30:   0%|          | 4/1272 [00:01<06:01,  3.51it/s]

MC样本统计: 均值=0.0000, 标准差=0.0003, 最小值=-0.0046, 最大值=0.0048
MC样本统计: 均值=0.0000, 标准差=0.0003, 最小值=-0.0043, 最大值=0.0043
MC样本统计: 均值=-0.0000, 标准差=0.0003, 最小值=-0.0039, 最大值=0.0049
MC样本统计: 均值=-0.0000, 标准差=0.0003, 最小值=-0.0048, 最大值=0.0059
MC样本统计: 均值=0.0000, 标准差=0.0003, 最小值=-0.0057, 最大值=0.0043


...

Epoch 3/30:  58%|█████▊    | 737/1272 [00:39<00:30, 17.69it/s]

MC样本统计: 均值=-0.0000, 标准差=0.0023, 最小值=-0.0114, 最大值=0.0356
MC样本统计: 均值=-0.0000, 标准差=0.0023, 最小值=-0.0115, 最大值=0.0255
MC样本统计: 均值=-0.0000, 标准差=0.0023, 最小值=-0.0107, 最大值=0.0235
MC样本统计: 均值=-0.0000, 标准差=0.0023, 最小值=-0.0095, 最大值=0.0280


Epoch 3/30:  58%|█████▊    | 741/1272 [00:39<00:30, 17.54it/s]

MC样本统计: 均值=-0.0000, 标准差=0.0023, 最小值=-0.0098, 最大值=0.0296
MC样本统计: 均值=0.0000, 标准差=0.0023, 最小值=-0.0095, 最大值=0.0274
MC样本统计: 均值=-0.0000, 标准差=0.0023, 最小值=-0.0112, 最大值=0.0266
MC样本统计: 均值=-0.0000, 标准差=0.0023, 最小值=-0.0095, 最大值=0.0258


Epoch 3/30:  59%|█████▊    | 745/1272 [00:39<00:30, 17.45it/s]

In [1]:
import os
import math
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import clip
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm

# ------------------------ #
# 1. 设备设置
# ------------------------ #

# 选择设备（GPU或CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")

# ------------------------ #
# 2. 加载CLIP模型
# ------------------------ #

# 加载CLIP模型和预处理方法
clip_model, preprocess = clip.load("ViT-B/32", device=device)
clip_model.eval()
print("CLIP模型已加载。")

# ------------------------ #
# 3. 定义CelebA数据集类
# ------------------------ #

class CelebADataset(Dataset):
    def __init__(self, img_dir, attr_path, bbox_path, partition_path, 
                 image_embeddings_path, transform=None, partition=0):
        """
        初始化数据集
        :param img_dir: 图像文件夹路径
        :param attr_path: 属性文件路径
        :param bbox_path: 边界框文件路径
        :param partition_path: 分区文件路径
        :param image_embeddings_path: 预先计算的图像嵌入文件路径
        :param transform: 图像预处理
        :param partition: 使用的数据分区 (0: train, 1: val, 2: test)
        """
        self.img_dir = img_dir
        self.transform = transform

        # 读取属性文件
        try:
            attr_df = pd.read_csv(attr_path, sep=',', header=0)
        except Exception as e:
            print(f"Error reading {attr_path}: {e}")
            raise

        print("Attributes DataFrame columns:", attr_df.columns.tolist())
        print("Attributes DataFrame head:\n", attr_df.head())

        # 确认 'image_id' 列存在
        if 'image_id' not in attr_df.columns:
            raise ValueError("属性文件中缺少 'image_id' 列。请检查文件格式。")
        else:
            print("'image_id' column found in attributes DataFrame.")

        # 读取分区文件
        try:
            partition_df = pd.read_csv(partition_path, sep=',', header=0, names=['image_id', 'partition'])
        except Exception as e:
            print(f"Error reading {partition_path}: {e}")
            raise

        print("Partition DataFrame columns:", partition_df.columns.tolist())
        print("Partition DataFrame head:\n", partition_df.head())

        # 读取边界框文件
        try:
            bbox_df = pd.read_csv(bbox_path, sep=',', header=0, names=['image_id', 'x_1', 'y_1', 'width', 'height'])
        except Exception as e:
            print(f"Error reading {bbox_path}: {e}")
            raise

        print("Bounding Box DataFrame columns:", bbox_df.columns.tolist())
        print("Bounding Box DataFrame head:\n", bbox_df.head())

        # 合并属性和分区数据
        try:
            attr_df = attr_df.merge(partition_df, on='image_id')
        except Exception as e:
            print(f"Error merging attributes and partitions: {e}")
            raise

        print("After merging with partition, columns:", attr_df.columns.tolist())
        print("Merged DataFrame head:\n", attr_df.head())

        # 合并属性、分区和边界框数据
        try:
            attr_df = attr_df.merge(bbox_df, on='image_id')
        except Exception as e:
            print(f"Error merging attributes, partitions and bounding boxes: {e}")
            raise

        print("After merging with bounding box, columns:", attr_df.columns.tolist())
        print("Final Merged DataFrame head:\n", attr_df.head())

        # 筛选分区
        print(f"Total samples before filtering: {len(attr_df)}")
        self.attr_df = attr_df[attr_df['partition'] == partition].reset_index(drop=True)
        print(f"Total samples after filtering partition {partition}: {len(self.attr_df)}")

        if len(self.attr_df) == 0:
            raise ValueError(f"训练集中没有样本。请检查数据文件的读取和过滤逻辑。")

        # 获取属性名称
        # 属性列是除了 'image_id', 'partition', 'x_1', 'y_1', 'width', 'height' 之外的所有列
        self.attr_names = [col for col in attr_df.columns if col not in ['image_id', 'partition', 'x_1', 'y_1', 'width', 'height']]
        print("Attribute names:", self.attr_names)

        # 加载预计算的图像嵌入
        self.image_embeddings = torch.load(image_embeddings_path)  # 假设是一个Tensor，shape=(num_images, 512)
        if len(self.image_embeddings) != len(self.attr_df):
            raise ValueError("图像嵌入的数量与数据集中的图像数量不一致。")
        self.image_embeddings = self.image_embeddings.float()

    def __len__(self):
        return len(self.attr_df)

    def __getitem__(self, idx):
        # 获取图像路径
        img_name = self.attr_df.iloc[idx, self.attr_df.columns.get_loc('image_id')]
        img_path = os.path.join(self.img_dir, img_name)
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Error opening image {img_path}: {e}")
            # 可以选择跳过该样本，或返回一个默认图像
            # 这里选择抛出异常
            raise

        if self.transform:
            image = self.transform(image)

        # 获取属性标签 
        attrs = self.attr_df.iloc[idx][self.attr_names].values
        attrs = (attrs + 1) // 2  # 将-1转为0，1保持1
        attrs = attrs.astype(np.float32)

        # 获取对应的图像嵌入
        clip_embedding = self.image_embeddings[idx]

        return image, attrs, clip_embedding

# ------------------------ #
# 4. 定义ClipCVAE模型（使用Renyi散度）
# ------------------------ #

class ClipCVAE_Renyi(nn.Module):
    def __init__(self, img_channels=3, img_size=64, latent_dim=128, 
                 cond_dim=40, clip_dim=512):
        super(ClipCVAE_Renyi, self).__init__()
        self.img_size = img_size
        self.latent_dim = latent_dim
        self.cond_dim = cond_dim
        self.clip_dim = clip_dim

        # 编码器部分
        self.encoder = nn.Sequential(
            nn.Conv2d(img_channels + cond_dim + clip_dim, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Flatten()
        )
        self.fc_mu = nn.Linear(512*4*4, latent_dim)
        self.fc_logvar = nn.Linear(512*4*4, latent_dim)

        # 解码器部分
        self.decoder_input = nn.Linear(latent_dim + cond_dim + clip_dim, 512*4*4)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, img_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def encode(self, x, c, clip_embedding):
        # 将条件标签、图像和CLIP嵌入拼接
        c = c.view(c.size(0), self.cond_dim, 1, 1).repeat(1, 1, self.img_size, self.img_size)
        clip_embedding = clip_embedding.view(clip_embedding.size(0), self.clip_dim, 1, 1).repeat(1, 1, self.img_size, self.img_size)
        x = torch.cat([x, c, clip_embedding], dim=1)
        x = self.encoder(x)
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z, c, clip_embedding):
        # 将潜在变量、条件标签和CLIP嵌入拼接
        z = torch.cat([z, c, clip_embedding], dim=1)
        x = self.decoder_input(z)
        x = x.view(-1, 512, 4, 4)
        x = self.decoder(x)
        return x

    def forward(self, x, c, clip_embedding):
        mu, logvar = self.encode(x, c, clip_embedding)
        z = self.reparameterize(mu, logvar)
        recon_x = self.decode(z, c, clip_embedding)
        return recon_x, mu, logvar

# ------------------------ #
# 5. 图像预处理
# ------------------------ #

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # 归一化
])

# ------------------------ #
# 6. 数据集路径设置
# ------------------------ #

# 请根据实际路径修改
img_dir = '/root/autodl-tmp/celeba_datasets/img_align_celeba/img_align_celeba'
attr_path = '/root/autodl-tmp/celeba_datasets/list_attr_celeba.txt'
bbox_path = '/root/autodl-tmp/celeba_datasets/list_bbox_celeba.txt'
partition_path = '/root/autodl-tmp/celeba_datasets/list_eval_partition.txt'
image_embeddings_train_path = '/root/autodl-tmp/clip_embeddings/image_embeddings_partition_0.pt'
image_embeddings_val_path = '/root/autodl-tmp/clip_embeddings/image_embeddings_partition_1.pt'

# ------------------------ #
# 7. 创建训练集和验证集
# ------------------------ #

train_dataset = CelebADataset(img_dir, attr_path, bbox_path, partition_path, 
                              image_embeddings_train_path, transform=transform, partition=0)
val_dataset = CelebADataset(img_dir, attr_path, bbox_path, partition_path, 
                            image_embeddings_val_path, transform=transform, partition=1)

# ------------------------ #
# 8. 数据加载器
# ------------------------ #

batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# ------------------------ #
# 9. 修订版 Rényi 散度损失函数
# ------------------------ #

import torch
import torch.nn.functional as F
import math

def negative_alpha_renyi_divergence_loss(recon_x, x, mu, logvar, epoch=1, alpha=-0.3, K=1):
    """
    使用α=-0.3的Rényi散度损失函数，基于变分Rényi边界理论
    
    参数:
    - recon_x: 重构的图像
    - x: 原始图像
    - mu: 均值向量
    - logvar: 对数方差向量
    - epoch: 当前训练轮数
    - alpha: Rényi散度参数，这里使用-0.3
    - K: 蒙特卡洛样本数量
    """
    # 重构损失计算
    batch_size = x.size(0)
    recon_loss = F.mse_loss(recon_x, x, reduction='mean') * 100
    
    # 限制参数范围 - 对于α=-0.3可以比α=-1宽松一些
    logvar = torch.clamp(logvar, min=-15.0, max=12.0)
    var = torch.exp(logvar)
    
    # 标准正态先验
    var0 = torch.ones_like(var)
    mu0 = torch.zeros_like(mu)
    
    # 计算KL散度的成分（用于监控）
    mu_diff_sq = (mu - mu0).pow(2)
    trace_term = var / var0
    logdet_term = torch.log(var0) - logvar
    
    # 基础KL散度项（用于监控）
    kl_base = 0.5 * (mu_diff_sq / var0 + trace_term + logdet_term - 1)
    
    # α=-0.3时的Rényi散度计算 - 使用蒙特卡洛采样
    # 由于α值比-1更接近0，数值稳定性应该更好
    mc_samples = []
    for _ in range(max(K, 5)):  # 至少使用5个样本
        eps = torch.randn_like(std := torch.sqrt(var))
        z = mu + eps * std
        
        # 计算log概率的差值
        log_q = -0.5 * (torch.log(2 * math.pi * var) + (z - mu).pow(2) / var)
        log_p = -0.5 * (torch.log(2 * math.pi * var0) + (z - mu0).pow(2) / var0)
        log_ratio = log_p - log_q
        
        # 裁剪极端值 - 比α=-1可以更宽松
        log_ratio = torch.clamp(log_ratio, min=-15.0, max=15.0)
        mc_samples.append(log_ratio)
    
    # 计算MC样本的Rényi散度估计
    log_weights = torch.stack(mc_samples, dim=0)
    
    # α=-0.3时的公式: 1/(α-1) * log(E[exp((1-α)*log(p/q))])
    # 对于α=-0.3: 1/(-0.3-1) * log(E[exp((1-(-0.3))*log(p/q))]) = -0.77 * log(E[exp(1.3*log(p/q))])
    renyi_div = (1 / (alpha - 1)) * torch.logsumexp((1 - alpha) * log_weights, dim=0).mean()
    
    # 确保数值稳定性 - 比α=-1可以更宽松
    renyi_div = torch.clamp(renyi_div, min=-12.0, max=12.0)
    
    # α=-0.3的权重设置 - 可以比α=-1更大一些
    max_weight = 4.0  # 最大权重
    min_weight = 0.8  # 起始权重
    
    # 使用权重增长曲线
    progress = min(epoch / 12.0, 1.0)
    weight_factor = min_weight + (max_weight - min_weight) * (1 - math.exp(-4 * progress))
    
    # 监控信息
    raw_renyi = renyi_div.item()
    raw_kl = kl_base.sum(dim=1).mean().item()
    
    # 总损失计算
    weighted_renyi = renyi_div * weight_factor
    total_loss = recon_loss + weighted_renyi
    
    # 额外监控 MC 样本的统计信息
    with torch.no_grad():
        mc_mean = log_weights.mean().item()
        mc_std = log_weights.std().item()
    
    return total_loss, recon_loss, weighted_renyi, raw_renyi, torch.tensor(0.0).to(recon_x.device), raw_kl, mc_mean, mc_std
# ------------------------ #
# 10. 改进的训练函数
# ------------------------ #
def train_negative_alpha_model(model, train_loader, val_loader, optimizer, scheduler, num_epochs=30, alpha=-0.3):
    """
    使用α=-0.3的Rényi散度训练CVAE模型
    """
    print(f"开始使用负α值(α={alpha})的Rényi散度训练CVAE模型...")
    
    # 记录训练统计信息
    train_stats = {
        'total_loss': [], 'recon_loss': [], 'renyi_loss': [], 
        'raw_renyi': [], 'kl_loss': [], 'raw_kl': [],
        'mc_mean': [], 'mc_std': []  # 添加MC样本统计
    }
    val_stats = {
        'total_loss': [], 'recon_loss': [], 'renyi_loss': [], 
        'raw_renyi': [], 'kl_loss': [], 'raw_kl': [],
        'mc_mean': [], 'mc_std': []
    }
    
    # 最佳模型跟踪
    best_val_loss = float('inf')
    patience = 12  # 对于α=-0.3，使用适中的耐心值
    patience_counter = 0
    
    # 确保保存目录存在
    os.makedirs('model_checkpoints', exist_ok=True)
    
    # α=-0.3的特殊处理 - 梯度累积步数可以比α=-1少
    accumulation_steps = 2  # 每2个批次更新一次
    
    for epoch in range(1, num_epochs + 1):
        model.train()
        epoch_stats = {k: 0.0 for k in train_stats.keys()}
        nan_batches = 0
        
        # 计算当前权重
        progress = min(epoch / 12.0, 1.0)
        min_weight = 0.8
        max_weight = 4.0
        weight_factor = min_weight + (max_weight - min_weight) * (1 - math.exp(-4 * progress))
        
        print(f"Epoch {epoch}/{num_epochs}, α={alpha} Rényi权重: {weight_factor:.4f}")
        
        # 训练循环
        for batch_idx, (data, attrs, clip_emb) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}")):
            data = data.to(device, non_blocking=True)
            attrs = attrs.to(device, non_blocking=True)
            clip_emb = clip_emb.to(device, non_blocking=True)
            
            # 只在累积步骤的开始时清零梯度
            if batch_idx % accumulation_steps == 0:
                optimizer.zero_grad()
            
            try:
                recon_batch, mu, logvar = model(data, attrs, clip_emb)
                
                # 使用α=-0.3的Rényi散度损失函数
                loss, recon_loss, renyi_loss, raw_renyi, kl_loss, raw_kl, mc_mean, mc_std = negative_alpha_renyi_divergence_loss(
                    recon_batch, data, mu, logvar, epoch, alpha=alpha, K=5
                )
                
                # 按累积步骤缩放损失
                loss = loss / accumulation_steps
                
                # 详细调试输出
                if batch_idx % 100 == 0:
                    print(f"\n批次 {batch_idx} 详细信息:")
                    print(f"  - 原始α={alpha} Rényi值: {raw_renyi:.6f}, 加权Rényi损失: {renyi_loss.item():.6f}")
                    print(f"  - 原始KL值: {raw_kl:.6f}")
                    print(f"  - 重构损失: {recon_loss.item():.6f}, 总损失: {loss.item()*accumulation_steps:.6f}")
                    print(f"  - MC样本均值: {mc_mean:.4f}, MC样本标准差: {mc_std:.4f}")
                    print(f"  - mu范围: [{mu.min().item():.4f}, {mu.max().item():.4f}], "
                          f"var范围: [{torch.exp(logvar).min().item():.6f}, {torch.exp(logvar).max().item():.6f}]")
                
                # 检查NaN
                if torch.isnan(loss).any() or torch.isinf(loss).any():
                    nan_batches += 1
                    if batch_idx % 100 == 0:
                        print(f"警告: 第{epoch}轮, 第{batch_idx}批次出现NaN/Inf损失。跳过此批次.")
                    continue
                
                # 反向传播
                loss.backward()
                
                # 仅在累积步骤结束时更新权重
                if (batch_idx + 1) % accumulation_steps == 0 or (batch_idx + 1) == len(train_loader):
                    # 适度的梯度裁剪 - α=-0.3比α=-1可以更宽松
                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                    optimizer.step()
                
                # 累计统计信息
                epoch_stats['total_loss'] += loss.item() * accumulation_steps
                epoch_stats['recon_loss'] += recon_loss.item()
                epoch_stats['renyi_loss'] += renyi_loss.item()
                epoch_stats['raw_renyi'] += raw_renyi
                epoch_stats['kl_loss'] += kl_loss.item()
                epoch_stats['raw_kl'] += raw_kl
                epoch_stats['mc_mean'] += mc_mean
                epoch_stats['mc_std'] += mc_std
                
            except RuntimeError as e:
                print(f"批次{batch_idx}处理出错: {e}")
                nan_batches += 1
                continue
        
        # 更新学习率
        scheduler.step()
        current_lr = optimizer.param_groups[0]["lr"]
        
        # 计算平均损失
        processed_batches = len(train_loader) - nan_batches
        if processed_batches > 0:
            for k in epoch_stats:
                epoch_stats[k] /= processed_batches
                train_stats[k].append(epoch_stats[k])
            
            print(f"\nEpoch {epoch} 训练统计:")
            print(f"  - 平均总损失: {epoch_stats['total_loss']:.4f}")
            print(f"  - 重构损失: {epoch_stats['recon_loss']:.4f}")
            print(f"  - α={alpha} Rényi损失: {epoch_stats['renyi_loss']:.4f} (原始值: {epoch_stats['raw_renyi']:.6f})")
            print(f"  - 原始KL值: {epoch_stats['raw_kl']:.6f}")
            print(f"  - MC样本均值: {epoch_stats['mc_mean']:.4f}, MC样本标准差: {epoch_stats['mc_std']:.4f}")
            print(f"  - 学习率: {current_lr:.6f}, NaN批次: {nan_batches}")
        else:
            print(f"Epoch {epoch}, 所有批次均为NaN，无法计算平均损失")
            continue
        
        # 验证集评估
        model.eval()
        val_epoch_stats = {k: 0.0 for k in val_stats.keys()}
        val_nan_batches = 0
        
        with torch.no_grad():
            for data, attrs, clip_emb in val_loader:
                data = data.to(device, non_blocking=True)
                attrs = attrs.to(device, non_blocking=True)
                clip_emb = clip_emb.to(device, non_blocking=True)
                
                try:
                    recon_batch, mu, logvar = model(data, attrs, clip_emb)
                    loss, recon_loss, renyi_loss, raw_renyi, kl_loss, raw_kl, mc_mean, mc_std = negative_alpha_renyi_divergence_loss(
                        recon_batch, data, mu, logvar, epoch, alpha=alpha, K=5
                    )
                    
                    if torch.isnan(loss).any() or torch.isinf(loss).any():
                        val_nan_batches += 1
                        continue
                    
                    # 累计验证统计
                    val_epoch_stats['total_loss'] += loss.item()
                    val_epoch_stats['recon_loss'] += recon_loss.item()
                    val_epoch_stats['renyi_loss'] += renyi_loss.item()
                    val_epoch_stats['raw_renyi'] += raw_renyi
                    val_epoch_stats['kl_loss'] += kl_loss.item()
                    val_epoch_stats['raw_kl'] += raw_kl
                    val_epoch_stats['mc_mean'] += mc_mean
                    val_epoch_stats['mc_std'] += mc_std
                    
                except RuntimeError:
                    val_nan_batches += 1
                    continue
        
        # 计算验证平均值
        processed_val_batches = len(val_loader) - val_nan_batches
        if processed_val_batches > 0:
            for k in val_epoch_stats:
                val_epoch_stats[k] /= processed_val_batches
                val_stats[k].append(val_epoch_stats[k])
            
            print(f"验证集统计:")
            print(f"  - 平均总损失: {val_epoch_stats['total_loss']:.4f}")
            print(f"  - 重构损失: {val_epoch_stats['recon_loss']:.4f}")
            print(f"  - α={alpha} Rényi损失: {val_epoch_stats['renyi_loss']:.4f} (原始值: {val_epoch_stats['raw_renyi']:.6f})")
            print(f"  - 原始KL值: {val_epoch_stats['raw_kl']:.6f}")
            print(f"  - MC样本均值: {val_epoch_stats['mc_mean']:.4f}, MC样本标准差: {val_epoch_stats['mc_std']:.4f}")
            print(f"  - NaN批次: {val_nan_batches}")
            
            # 保存当前模型
            current_model_path = f'model_checkpoints/cvae_renyi_alpha{alpha}_epoch_{epoch}.pth'
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_stats': train_stats,
                'val_stats': val_stats,
                'alpha': alpha,
            }, current_model_path)
            
            # 对于α=-0.3，使用重构损失和Rényi值的平衡组合作为质量指标
            model_quality = val_epoch_stats['recon_loss'] - 0.3 * val_epoch_stats['raw_renyi']
            
            if model_quality < best_val_loss:
                best_val_loss = model_quality
                patience_counter = 0
                
                best_model_path = f'model_checkpoints/cvae_renyi_alpha{alpha}_best.pth'
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'best_quality': model_quality,
                    'train_stats': train_stats,
                    'val_stats': val_stats,
                    'alpha': alpha,
                }, best_model_path)
                print(f"保存最佳模型: {best_model_path} (质量指标: {model_quality:.4f})")
            else:
                patience_counter += 1
                print(f"模型质量未改善，耐心计数: {patience_counter}/{patience}")
                if patience_counter >= patience:
                    print(f"提前停止训练！模型质量{patience}轮未改善。")
                    return model, train_stats, val_stats
        else:
            print(f"验证集所有批次均为NaN，无法计算平均损失")
    
    return model, train_stats, val_stats
# ------------------------ #
# 11. 可视化函数
# ------------------------ #

def plot_training_stats(train_stats, val_stats, save_path='training_stats.png'):
    """
    绘制训练过程中的各种统计数据
    """
    import matplotlib.pyplot as plt
    
    epochs = range(1, len(train_stats['total_loss']) + 1)
    
    plt.figure(figsize=(18, 12))
    
    # 绘制总损失
    plt.subplot(2, 3, 1)
    plt.plot(epochs, train_stats['total_loss'], 'b-', label='训练')
    plt.plot(epochs, val_stats['total_loss'], 'r-', label='验证')
    plt.title('总损失')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # 绘制重构损失
    plt.subplot(2, 3, 2)
    plt.plot(epochs, train_stats['recon_loss'], 'b-', label='训练')
    plt.plot(epochs, val_stats['recon_loss'], 'r-', label='验证')
    plt.title('重构损失')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # 绘制Rényi损失
    plt.subplot(2, 3, 3)
    plt.plot(epochs, train_stats['renyi_loss'], 'b-', label='训练')
    plt.plot(epochs, val_stats['renyi_loss'], 'r-', label='验证')
    plt.title('Rényi损失 (加权)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # 绘制原始Rényi值
    plt.subplot(2, 3, 4)
    plt.plot(epochs, train_stats['raw_renyi'], 'b-', label='训练')
    plt.plot(epochs, val_stats['raw_renyi'], 'r-', label='验证')
    plt.title('原始Rényi值')
    plt.xlabel('Epoch')
    plt.ylabel('Value')
    plt.legend()
    
    # 绘制KL损失
    plt.subplot(2, 3, 5)
    plt.plot(epochs, train_stats['kl_loss'], 'b-', label='训练')
    plt.plot(epochs, val_stats['kl_loss'], 'r-', label='验证')
    plt.title('KL损失 (加权)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # 绘制原始KL值
    plt.subplot(2, 3, 6)
    plt.plot(epochs, train_stats['raw_kl'], 'b-', label='训练')
    plt.plot(epochs, val_stats['raw_kl'], 'r-', label='验证')
    plt.title('原始KL值')
    plt.xlabel('Epoch')
    plt.ylabel('Value')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()
    print(f"训练统计图已保存至 {save_path}")

# ------------------------ #
# 12. 样本生成与可视化
# ------------------------ #

def generate_samples(model, val_loader, num_samples=10, save_dir='sample_images'):
    """
    生成样本并可视化比较
    """
    import matplotlib.pyplot as plt
    from torchvision.utils import make_grid
    
    os.makedirs(save_dir, exist_ok=True)
    
    model.eval()
    
    # 随机获取一些验证集样本
    data_iter = iter(val_loader)
    data, attrs, clip_emb = next(data_iter)
    
    data = data[:num_samples].to(device)
    attrs = attrs[:num_samples].to(device)
    clip_emb = clip_emb[:num_samples].to(device)
    
    with torch.no_grad():
        # 获取原始图像、重构图像
        recon_img, mu, logvar = model(data, attrs, clip_emb)
        
        # 从潜在空间采样新图像
        z = torch.randn_like(mu)
        sampled_img = model.decode(z, attrs, clip_emb)
    
    # 将张量转换为可视化的图像
    # 去归一化，从[-1, 1]到[0, 1]
    def denormalize(x):
        return (x + 1) / 2
    
    original_imgs = denormalize(data)
    recon_imgs = denormalize(recon_img)
    sampled_imgs = denormalize(sampled_img)
    
    # 显示原始、重构和采样的图像
    fig, axes = plt.subplots(3, num_samples, figsize=(num_samples * 2, 6))
    
    # 去除坐标轴
    for ax in axes.flatten():
        ax.axis('off')
    
    # 标题
    axes[0, num_samples//2].set_title('原始图像', fontsize=12, pad=10)
    axes[1, num_samples//2].set_title('重构图像', fontsize=12, pad=10)
    axes[2, num_samples//2].set_title('从潜在空间采样', fontsize=12, pad=10)
    
    # 显示图像
    for i in range(num_samples):
        # 原始图像
        img = original_imgs[i].cpu().permute(1, 2, 0).numpy()
        img = np.clip(img, 0, 1)
        axes[0, i].imshow(img)
        
        # 重构图像
        img = recon_imgs[i].cpu().permute(1, 2, 0).numpy()
        img = np.clip(img, 0, 1)
        axes[1, i].imshow(img)
        
        # 采样图像
        img = sampled_imgs[i].cpu().permute(1, 2, 0).numpy()
        img = np.clip(img, 0, 1)
        axes[2, i].imshow(img)
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, 'sample_comparison.png'), dpi=150)
    plt.close()
    
    print(f"样本可视化已保存至 {os.path.join(save_dir, 'sample_comparison.png')}")
    
    return {
        'original': original_imgs.cpu(),
        'recon': recon_imgs.cpu(),
        'sampled': sampled_imgs.cpu()
    }

# ------------------------ #
# 13. 主程序
# ------------------------ #
if __name__ == "__main__":
    try:
        # 设置随机种子确保可重复性
        torch.manual_seed(42)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(42)
        np.random.seed(42)
        random.seed(42)
        
        # 设置α=-0.3
        alpha = -0.3
        
        # 创建模型实例
        model_renyi_neg = ClipCVAE_Renyi(img_channels=3, img_size=64, latent_dim=128, 
                                   cond_dim=40, clip_dim=512).to(device)
        
        # 初始化权重 - 对于α=-0.3，可以比α=-1使用更标准的初始化
        for name, param in model_renyi_neg.named_parameters():
            if 'weight' in name:
                if 'fc_mu' in name or 'fc_logvar' in name:
                    nn.init.normal_(param, mean=0.0, std=0.003)
                elif param.dim() >= 2:
                    nn.init.kaiming_normal_(param, mode='fan_out', nonlinearity='relu')
                else:
                    nn.init.normal_(param, mean=0.0, std=0.01)
            elif 'bias' in name:
                nn.init.zeros_(param)
        
        print(f"已成功完成模型参数初始化，使用α={alpha}的Rényi散度计算方法")
        
        # 对于α=-0.3，学习率和权重衰减可以介于α=-1和正α之间
        optimizer = optim.AdamW(model_renyi_neg.parameters(), lr=3e-5, weight_decay=5e-3, eps=1e-5)
        
        # 使用余弦退火调度器
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30, eta_min=1e-6)
        
        # 训练模型
        trained_model, train_stats, val_stats = train_negative_alpha_model(
            model_renyi_neg, train_loader, val_loader, optimizer, scheduler, 
            num_epochs=30, alpha=alpha
        )
        
        # 保存最终模型
        final_model_path = f'model_checkpoints/cvae_renyi_alpha{alpha}_final.pth'
        torch.save({
            'model_state_dict': trained_model.state_dict(),
            'train_stats': train_stats,
            'val_stats': val_stats,
            'alpha': alpha
        }, final_model_path)
        print(f"α={alpha}的Rényi散度CVAE模型已保存为: {final_model_path}")
        
        # 绘制训练过程中的损失变化图表
        plot_training_stats(train_stats, val_stats, save_path=f'model_checkpoints/training_stats_alpha{alpha}.png')
        
        # 生成样本并可视化
        sample_results = generate_samples(
            trained_model, val_loader, num_samples=8, 
            save_dir=f'model_checkpoints/sample_images_alpha{alpha}'
        )
        
        print(f"使用α={alpha}的Rényi散度CVAE模型训练完成！")
        
    except Exception as e:
        print(f"训练过程中发生错误: {e}")
        import traceback
        traceback.print_exc()
        # 保存紧急状态
        emergency_save_path = f'model_checkpoints/cvae_renyi_alpha{alpha}_emergency_save.pth'
        torch.save(model_renyi_neg.state_dict(), emergency_save_path)
        print(f"发生错误，模型紧急保存为: {emergency_save_path}")

使用设备: cuda
CLIP模型已加载。
Attributes DataFrame columns: ['image_id', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive', 'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose', 'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows', 'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace', 'Wearing_Necktie', 'Young']
Attributes DataFrame head:
      image_id  5_o_Clock_Shadow  Arched_Eyebrows  Attractive  Bags_Under_Eyes  \
0  000001.jpg                -1                1           1               -1   
1  000002.jpg                -1               -1          -1                1   
2  000003.jpg                -1               -1          -1               -1   

Epoch 1/30:   0%|          | 1/1272 [00:01<27:53,  1.32s/it]


批次 0 详细信息:
  - 原始α=-0.3 Rényi值: -1.238026, 加权Rényi损失: -2.113433
  - 原始KL值: 0.000249
  - 重构损失: 36.100555, 总损失: 33.987122
  - MC样本均值: -0.0000, MC样本标准差: 0.0020
  - mu范围: [-0.0078, 0.0084], var范围: [0.990517, 1.007405]


Epoch 1/30:   8%|▊         | 102/1272 [00:06<00:57, 20.41it/s]


批次 100 详细信息:
  - 原始α=-0.3 Rényi值: -1.238029, 加权Rényi损失: -2.113440
  - 原始KL值: 0.000259
  - 重构损失: 35.527096, 总损失: 33.413654
  - MC样本均值: -0.0000, MC样本标准差: 0.0020
  - mu范围: [-0.0079, 0.0079], var范围: [0.989779, 1.008034]


...

验证集统计:
  - 平均总损失: 0.6869
  - 重构损失: 4.5425
  - α=-0.3 Rényi损失: -3.8556 (原始值: -0.978243)
  - 原始KL值: 70.649321
  - MC样本均值: -0.5519, MC样本标准差: 1.1419
  - NaN批次: 0
保存最佳模型: model_checkpoints/cvae_renyi_alpha-0.3_best.pth (质量指标: 4.8360)
α=-0.3的Rényi散度CVAE模型已保存为: model_checkpoints/cvae_renyi_alpha-0.3_final.pth


/tmp/ipykernel_6118/2602325695.py:646: UserWarning: Glyph 24635 (\N{CJK UNIFIED IDEOGRAPH-603B}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_6118/2602325695.py:646: UserWarning: Glyph 25439 (\N{CJK UNIFIED IDEOGRAPH-635F}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_6118/2602325695.py:646: UserWarning: Glyph 22833 (\N{CJK UNIFIED IDEOGRAPH-5931}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_6118/2602325695.py:646: UserWarning: Glyph 35757 (\N{CJK UNIFIED IDEOGRAPH-8BAD}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_6118/2602325695.py:646: UserWarning: Glyph 32451 (\N{CJK UNIFIED IDEOGRAPH-7EC3}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_6118/2602325695.py:646: UserWarning: Glyph 39564 (\N{CJK UNIFIED IDEOGRAPH-9A8C}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_6118/2602325695.py:646: UserWarning: Glyph 35777 (\N{CJK UNIFIED IDEOGRAP

训练统计图已保存至 model_checkpoints/training_stats_alpha-0.3.png


/tmp/ipykernel_6118/2602325695.py:720: UserWarning: Glyph 21407 (\N{CJK UNIFIED IDEOGRAPH-539F}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_6118/2602325695.py:720: UserWarning: Glyph 22987 (\N{CJK UNIFIED IDEOGRAPH-59CB}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_6118/2602325695.py:720: UserWarning: Glyph 22270 (\N{CJK UNIFIED IDEOGRAPH-56FE}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_6118/2602325695.py:720: UserWarning: Glyph 20687 (\N{CJK UNIFIED IDEOGRAPH-50CF}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_6118/2602325695.py:720: UserWarning: Glyph 37325 (\N{CJK UNIFIED IDEOGRAPH-91CD}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_6118/2602325695.py:720: UserWarning: Glyph 26500 (\N{CJK UNIFIED IDEOGRAPH-6784}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_6118/2602325695.py:720: UserWarning: Glyph 20174 (\N{CJK UNIFIED IDEOGRAP

样本可视化已保存至 model_checkpoints/sample_images_alpha-0.3/sample_comparison.png
使用α=-0.3的Rényi散度CVAE模型训练完成！


In [2]:
import os
import math
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import clip
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm

# ------------------------ #
# 1. 设备设置
# ------------------------ #

# 选择设备（GPU或CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")

# ------------------------ #
# 2. 加载CLIP模型
# ------------------------ #

# 加载CLIP模型和预处理方法
clip_model, preprocess = clip.load("ViT-B/32", device=device)
clip_model.eval()
print("CLIP模型已加载。")

# ------------------------ #
# 3. 定义CelebA数据集类
# ------------------------ #

class CelebADataset(Dataset):
    def __init__(self, img_dir, attr_path, bbox_path, partition_path, 
                 image_embeddings_path, transform=None, partition=0):
        """
        初始化数据集
        :param img_dir: 图像文件夹路径
        :param attr_path: 属性文件路径
        :param bbox_path: 边界框文件路径
        :param partition_path: 分区文件路径
        :param image_embeddings_path: 预先计算的图像嵌入文件路径
        :param transform: 图像预处理
        :param partition: 使用的数据分区 (0: train, 1: val, 2: test)
        """
        self.img_dir = img_dir
        self.transform = transform

        # 读取属性文件
        try:
            attr_df = pd.read_csv(attr_path, sep=',', header=0)
        except Exception as e:
            print(f"Error reading {attr_path}: {e}")
            raise

        print("Attributes DataFrame columns:", attr_df.columns.tolist())
        print("Attributes DataFrame head:\n", attr_df.head())

        # 确认 'image_id' 列存在
        if 'image_id' not in attr_df.columns:
            raise ValueError("属性文件中缺少 'image_id' 列。请检查文件格式。")
        else:
            print("'image_id' column found in attributes DataFrame.")

        # 读取分区文件
        try:
            partition_df = pd.read_csv(partition_path, sep=',', header=0, names=['image_id', 'partition'])
        except Exception as e:
            print(f"Error reading {partition_path}: {e}")
            raise

        print("Partition DataFrame columns:", partition_df.columns.tolist())
        print("Partition DataFrame head:\n", partition_df.head())

        # 读取边界框文件
        try:
            bbox_df = pd.read_csv(bbox_path, sep=',', header=0, names=['image_id', 'x_1', 'y_1', 'width', 'height'])
        except Exception as e:
            print(f"Error reading {bbox_path}: {e}")
            raise

        print("Bounding Box DataFrame columns:", bbox_df.columns.tolist())
        print("Bounding Box DataFrame head:\n", bbox_df.head())

        # 合并属性和分区数据
        try:
            attr_df = attr_df.merge(partition_df, on='image_id')
        except Exception as e:
            print(f"Error merging attributes and partitions: {e}")
            raise

        print("After merging with partition, columns:", attr_df.columns.tolist())
        print("Merged DataFrame head:\n", attr_df.head())

        # 合并属性、分区和边界框数据
        try:
            attr_df = attr_df.merge(bbox_df, on='image_id')
        except Exception as e:
            print(f"Error merging attributes, partitions and bounding boxes: {e}")
            raise

        print("After merging with bounding box, columns:", attr_df.columns.tolist())
        print("Final Merged DataFrame head:\n", attr_df.head())

        # 筛选分区
        print(f"Total samples before filtering: {len(attr_df)}")
        self.attr_df = attr_df[attr_df['partition'] == partition].reset_index(drop=True)
        print(f"Total samples after filtering partition {partition}: {len(self.attr_df)}")

        if len(self.attr_df) == 0:
            raise ValueError(f"训练集中没有样本。请检查数据文件的读取和过滤逻辑。")

        # 获取属性名称
        # 属性列是除了 'image_id', 'partition', 'x_1', 'y_1', 'width', 'height' 之外的所有列
        self.attr_names = [col for col in attr_df.columns if col not in ['image_id', 'partition', 'x_1', 'y_1', 'width', 'height']]
        print("Attribute names:", self.attr_names)

        # 加载预计算的图像嵌入
        self.image_embeddings = torch.load(image_embeddings_path)  # 假设是一个Tensor，shape=(num_images, 512)
        if len(self.image_embeddings) != len(self.attr_df):
            raise ValueError("图像嵌入的数量与数据集中的图像数量不一致。")
        self.image_embeddings = self.image_embeddings.float()

    def __len__(self):
        return len(self.attr_df)

    def __getitem__(self, idx):
        # 获取图像路径
        img_name = self.attr_df.iloc[idx, self.attr_df.columns.get_loc('image_id')]
        img_path = os.path.join(self.img_dir, img_name)
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Error opening image {img_path}: {e}")
            # 可以选择跳过该样本，或返回一个默认图像
            # 这里选择抛出异常
            raise

        if self.transform:
            image = self.transform(image)

        # 获取属性标签 
        attrs = self.attr_df.iloc[idx][self.attr_names].values
        attrs = (attrs + 1) // 2  # 将-1转为0，1保持1
        attrs = attrs.astype(np.float32)

        # 获取对应的图像嵌入
        clip_embedding = self.image_embeddings[idx]

        return image, attrs, clip_embedding

# ------------------------ #
# 4. 定义ClipCVAE模型（使用Renyi散度）
# ------------------------ #

class ClipCVAE_Renyi(nn.Module):
    def __init__(self, img_channels=3, img_size=64, latent_dim=128, 
                 cond_dim=40, clip_dim=512):
        super(ClipCVAE_Renyi, self).__init__()
        self.img_size = img_size
        self.latent_dim = latent_dim
        self.cond_dim = cond_dim
        self.clip_dim = clip_dim

        # 编码器部分
        self.encoder = nn.Sequential(
            nn.Conv2d(img_channels + cond_dim + clip_dim, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Flatten()
        )
        self.fc_mu = nn.Linear(512*4*4, latent_dim)
        self.fc_logvar = nn.Linear(512*4*4, latent_dim)

        # 解码器部分
        self.decoder_input = nn.Linear(latent_dim + cond_dim + clip_dim, 512*4*4)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, img_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def encode(self, x, c, clip_embedding):
        # 将条件标签、图像和CLIP嵌入拼接
        c = c.view(c.size(0), self.cond_dim, 1, 1).repeat(1, 1, self.img_size, self.img_size)
        clip_embedding = clip_embedding.view(clip_embedding.size(0), self.clip_dim, 1, 1).repeat(1, 1, self.img_size, self.img_size)
        x = torch.cat([x, c, clip_embedding], dim=1)
        x = self.encoder(x)
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z, c, clip_embedding):
        # 将潜在变量、条件标签和CLIP嵌入拼接
        z = torch.cat([z, c, clip_embedding], dim=1)
        x = self.decoder_input(z)
        x = x.view(-1, 512, 4, 4)
        x = self.decoder(x)
        return x

    def forward(self, x, c, clip_embedding):
        mu, logvar = self.encode(x, c, clip_embedding)
        z = self.reparameterize(mu, logvar)
        recon_x = self.decode(z, c, clip_embedding)
        return recon_x, mu, logvar

# ------------------------ #
# 5. 图像预处理
# ------------------------ #

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # 归一化
])

# ------------------------ #
# 6. 数据集路径设置
# ------------------------ #

# 请根据实际路径修改
img_dir = '/root/autodl-tmp/celeba_datasets/img_align_celeba/img_align_celeba'
attr_path = '/root/autodl-tmp/celeba_datasets/list_attr_celeba.txt'
bbox_path = '/root/autodl-tmp/celeba_datasets/list_bbox_celeba.txt'
partition_path = '/root/autodl-tmp/celeba_datasets/list_eval_partition.txt'
image_embeddings_train_path = '/root/autodl-tmp/clip_embeddings/image_embeddings_partition_0.pt'
image_embeddings_val_path = '/root/autodl-tmp/clip_embeddings/image_embeddings_partition_1.pt'

# ------------------------ #
# 7. 创建训练集和验证集
# ------------------------ #

train_dataset = CelebADataset(img_dir, attr_path, bbox_path, partition_path, 
                              image_embeddings_train_path, transform=transform, partition=0)
val_dataset = CelebADataset(img_dir, attr_path, bbox_path, partition_path, 
                            image_embeddings_val_path, transform=transform, partition=1)

# ------------------------ #
# 8. 数据加载器
# ------------------------ #

batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# ------------------------ #
# 9. 修订版 Rényi 散度损失函数
# ------------------------ #

import torch
import torch.nn.functional as F
import math

def negative_alpha_renyi_divergence_loss(recon_x, x, mu, logvar, epoch=1, alpha=-0.7, K=1):
    """
    使用α=-0.7的Rényi散度损失函数，基于变分Rényi边界理论
    
    参数:
    - recon_x: 重构的图像
    - x: 原始图像
    - mu: 均值向量
    - logvar: 对数方差向量
    - epoch: 当前训练轮数
    - alpha: Rényi散度参数，这里使用-0.7
    - K: 蒙特卡洛样本数量
    """
    # 重构损失计算
    batch_size = x.size(0)
    recon_loss = F.mse_loss(recon_x, x, reduction='mean') * 100
    
    # 限制参数范围 - α=-0.7时的范围介于α=-0.3和α=-1.0之间
    logvar = torch.clamp(logvar, min=-14.0, max=10.0)
    var = torch.exp(logvar)
    
    # 标准正态先验
    var0 = torch.ones_like(var)
    mu0 = torch.zeros_like(mu)
    
    # 计算KL散度的成分（用于监控）
    mu_diff_sq = (mu - mu0).pow(2)
    trace_term = var / var0
    logdet_term = torch.log(var0) - logvar
    
    # 基础KL散度项（用于监控）
    kl_base = 0.5 * (mu_diff_sq / var0 + trace_term + logdet_term - 1)
    
    # α=-0.7时的Rényi散度计算 - 使用蒙特卡洛采样
    # 由于α值比-1更接近0，数值稳定性应该比α=-1好
    mc_samples = []
    for _ in range(max(K, 8)):  # 至少使用8个样本，比α=-0.3多但比α=-1少
        eps = torch.randn_like(std := torch.sqrt(var))
        z = mu + eps * std
        
        # 计算log概率的差值
        log_q = -0.5 * (torch.log(2 * math.pi * var) + (z - mu).pow(2) / var)
        log_p = -0.5 * (torch.log(2 * math.pi * var0) + (z - mu0).pow(2) / var0)
        log_ratio = log_p - log_q
        
        # 裁剪极端值 - α=-0.7的裁剪范围
        log_ratio = torch.clamp(log_ratio, min=-12.0, max=12.0)
        mc_samples.append(log_ratio)
    
    # 计算MC样本的Rényi散度估计
    log_weights = torch.stack(mc_samples, dim=0)
    
    # α=-0.7时的公式: 1/(α-1) * log(E[exp((1-α)*log(p/q))])
    # 对于α=-0.7: 1/(-0.7-1) * log(E[exp((1-(-0.7))*log(p/q))]) = -0.59 * log(E[exp(1.7*log(p/q))])
    renyi_div = (1 / (alpha - 1)) * torch.logsumexp((1 - alpha) * log_weights, dim=0).mean()
    
    # 确保数值稳定性 - α=-0.7的裁剪范围
    renyi_div = torch.clamp(renyi_div, min=-10.0, max=10.0)
    
    # α=-0.7的权重设置 - 介于α=-0.3和α=-1之间
    max_weight = 3.5  # 最大权重
    min_weight = 0.6  # 起始权重
    
    # 使用权重增长曲线
    progress = min(epoch / 14.0, 1.0)  # 14轮过渡期，介于α=-0.3和α=-1之间
    weight_factor = min_weight + (max_weight - min_weight) * (1 - math.exp(-3.5 * progress))
    
    # 监控信息
    raw_renyi = renyi_div.item()
    raw_kl = kl_base.sum(dim=1).mean().item()
    
    # 总损失计算
    weighted_renyi = renyi_div * weight_factor
    total_loss = recon_loss + weighted_renyi
    
    # 额外监控 MC 样本的统计信息
    with torch.no_grad():
        mc_mean = log_weights.mean().item()
        mc_std = log_weights.std().item()
    
    return total_loss, recon_loss, weighted_renyi, raw_renyi, torch.tensor(0.0).to(recon_x.device), raw_kl, mc_mean, mc_std
# ------------------------ #
# 10. 改进的训练函数
# ------------------------ #
def train_negative_alpha_model(model, train_loader, val_loader, optimizer, scheduler, num_epochs=30, alpha=-0.7):
    """
    使用α=-0.7的Rényi散度训练CVAE模型
    """
    print(f"开始使用负α值(α={alpha})的Rényi散度训练CVAE模型...")
    
    # 记录训练统计信息
    train_stats = {
        'total_loss': [], 'recon_loss': [], 'renyi_loss': [], 
        'raw_renyi': [], 'kl_loss': [], 'raw_kl': [],
        'mc_mean': [], 'mc_std': []  # 添加MC样本统计
    }
    val_stats = {
        'total_loss': [], 'recon_loss': [], 'renyi_loss': [], 
        'raw_renyi': [], 'kl_loss': [], 'raw_kl': [],
        'mc_mean': [], 'mc_std': []
    }
    
    # 最佳模型跟踪
    best_val_loss = float('inf')
    patience = 14  # 对于α=-0.7，使用介于α=-0.3和α=-1之间的耐心值
    patience_counter = 0
    
    # 确保保存目录存在
    os.makedirs('model_checkpoints', exist_ok=True)
    
    # α=-0.7的特殊处理 - 梯度累积步数介于α=-0.3和α=-1之间
    accumulation_steps = 3  # 每3个批次更新一次
    
    for epoch in range(1, num_epochs + 1):
        model.train()
        epoch_stats = {k: 0.0 for k in train_stats.keys()}
        nan_batches = 0
        
        # 计算当前权重
        progress = min(epoch / 14.0, 1.0)
        min_weight = 0.6
        max_weight = 3.5
        weight_factor = min_weight + (max_weight - min_weight) * (1 - math.exp(-3.5 * progress))
        
        print(f"Epoch {epoch}/{num_epochs}, α={alpha} Rényi权重: {weight_factor:.4f}")
        
        # 训练循环
        for batch_idx, (data, attrs, clip_emb) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}")):
            data = data.to(device, non_blocking=True)
            attrs = attrs.to(device, non_blocking=True)
            clip_emb = clip_emb.to(device, non_blocking=True)
            
            # 只在累积步骤的开始时清零梯度
            if batch_idx % accumulation_steps == 0:
                optimizer.zero_grad()
            
            try:
                recon_batch, mu, logvar = model(data, attrs, clip_emb)
                
                # 使用α=-0.7的Rényi散度损失函数
                loss, recon_loss, renyi_loss, raw_renyi, kl_loss, raw_kl, mc_mean, mc_std = negative_alpha_renyi_divergence_loss(
                    recon_batch, data, mu, logvar, epoch, alpha=alpha, K=8
                )
                
                # 按累积步骤缩放损失
                loss = loss / accumulation_steps
                
                # 详细调试输出
                if batch_idx % 100 == 0:
                    print(f"\n批次 {batch_idx} 详细信息:")
                    print(f"  - 原始α={alpha} Rényi值: {raw_renyi:.6f}, 加权Rényi损失: {renyi_loss.item():.6f}")
                    print(f"  - 原始KL值: {raw_kl:.6f}")
                    print(f"  - 重构损失: {recon_loss.item():.6f}, 总损失: {loss.item()*accumulation_steps:.6f}")
                    print(f"  - MC样本均值: {mc_mean:.4f}, MC样本标准差: {mc_std:.4f}")
                    print(f"  - mu范围: [{mu.min().item():.4f}, {mu.max().item():.4f}], "
                          f"var范围: [{torch.exp(logvar).min().item():.6f}, {torch.exp(logvar).max().item():.6f}]")
                    
                    # 额外调试信息，特别关注α=-0.7的数值特性
                    with torch.no_grad():
                        # 收集分布统计数据
                        variance = torch.exp(logvar)  # 修复：计算方差
                        mu_abs_mean = mu.abs().mean().item()
                        var_mean = variance.mean().item()  # 修复：使用计算得到的方差
                        logvar_mean = logvar.mean().item()
                        
                        print(f"  - 分布统计: |μ|平均={mu_abs_mean:.4f}, σ²平均={var_mean:.4f}, log(σ²)平均={logvar_mean:.4f}")
                        
                        # 检查重参数化过程
                        z = model.reparameterize(mu, logvar)
                        z_mean = z.mean().item()
                        z_std = z.std().item()
                        z_min = z.min().item()
                        z_max = z.max().item()
                        
                        print(f"  - 潜变量z统计: 均值={z_mean:.4f}, 标准差={z_std:.4f}, 范围=[{z_min:.4f}, {z_max:.4f}]")
                
                # 检查NaN和异常值
                if torch.isnan(loss).any() or torch.isinf(loss).any() or raw_renyi > 50 or raw_renyi < -50:
                    nan_batches += 1
                    if batch_idx % 100 == 0:
                        print(f"警告: 第{epoch}轮, 第{batch_idx}批次出现异常损失值。跳过此批次.")
                    continue
                
                # 反向传播
                loss.backward()
                
                # 仅在累积步骤结束时更新权重
                if (batch_idx + 1) % accumulation_steps == 0 or (batch_idx + 1) == len(train_loader):
                    # 适度的梯度裁剪 - α=-0.7的适当裁剪值
                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.8)
                    optimizer.step()
                
                # 累计统计信息
                epoch_stats['total_loss'] += loss.item() * accumulation_steps
                epoch_stats['recon_loss'] += recon_loss.item()
                epoch_stats['renyi_loss'] += renyi_loss.item()
                epoch_stats['raw_renyi'] += raw_renyi
                epoch_stats['kl_loss'] += kl_loss.item()
                epoch_stats['raw_kl'] += raw_kl
                epoch_stats['mc_mean'] += mc_mean
                epoch_stats['mc_std'] += mc_std
                
            except RuntimeError as e:
                print(f"批次{batch_idx}处理出错: {e}")
                nan_batches += 1
                continue
        
        # 更新学习率
        scheduler.step()
        current_lr = optimizer.param_groups[0]["lr"]
        
        # 计算平均损失
        processed_batches = len(train_loader) - nan_batches
        if processed_batches > 0:
            for k in epoch_stats:
                epoch_stats[k] /= processed_batches
                train_stats[k].append(epoch_stats[k])
            
            print(f"\nEpoch {epoch} 训练统计:")
            print(f"  - 平均总损失: {epoch_stats['total_loss']:.4f}")
            print(f"  - 重构损失: {epoch_stats['recon_loss']:.4f}")
            print(f"  - α={alpha} Rényi损失: {epoch_stats['renyi_loss']:.4f} (原始值: {epoch_stats['raw_renyi']:.6f})")
            print(f"  - 原始KL值: {epoch_stats['raw_kl']:.6f}")
            print(f"  - MC样本均值: {epoch_stats['mc_mean']:.4f}, MC样本标准差: {epoch_stats['mc_std']:.4f}")
            print(f"  - 学习率: {current_lr:.6f}, NaN批次: {nan_batches}")
        else:
            print(f"Epoch {epoch}, 所有批次均为NaN，无法计算平均损失")
            continue
        
        # 验证集评估
        model.eval()
        val_epoch_stats = {k: 0.0 for k in val_stats.keys()}
        val_nan_batches = 0
        
        with torch.no_grad():
            for data, attrs, clip_emb in val_loader:
                data = data.to(device, non_blocking=True)
                attrs = attrs.to(device, non_blocking=True)
                clip_emb = clip_emb.to(device, non_blocking=True)
                
                try:
                    recon_batch, mu, logvar = model(data, attrs, clip_emb)
                    loss, recon_loss, renyi_loss, raw_renyi, kl_loss, raw_kl, mc_mean, mc_std = negative_alpha_renyi_divergence_loss(
                        recon_batch, data, mu, logvar, epoch, alpha=alpha, K=8
                    )
                    
                    if torch.isnan(loss).any() or torch.isinf(loss).any() or raw_renyi > 50 or raw_renyi < -50:
                        val_nan_batches += 1
                        continue
                    
                    # 累计验证统计
                    val_epoch_stats['total_loss'] += loss.item()
                    val_epoch_stats['recon_loss'] += recon_loss.item()
                    val_epoch_stats['renyi_loss'] += renyi_loss.item()
                    val_epoch_stats['raw_renyi'] += raw_renyi
                    val_epoch_stats['kl_loss'] += kl_loss.item()
                    val_epoch_stats['raw_kl'] += raw_kl
                    val_epoch_stats['mc_mean'] += mc_mean
                    val_epoch_stats['mc_std'] += mc_std
                    
                except RuntimeError:
                    val_nan_batches += 1
                    continue
        
        # 计算验证平均值
        processed_val_batches = len(val_loader) - val_nan_batches
        if processed_val_batches > 0:
            for k in val_epoch_stats:
                val_epoch_stats[k] /= processed_val_batches
                val_stats[k].append(val_epoch_stats[k])
            
            print(f"验证集统计:")
            print(f"  - 平均总损失: {val_epoch_stats['total_loss']:.4f}")
            print(f"  - 重构损失: {val_epoch_stats['recon_loss']:.4f}")
            print(f"  - α={alpha} Rényi损失: {val_epoch_stats['renyi_loss']:.4f} (原始值: {val_epoch_stats['raw_renyi']:.6f})")
            print(f"  - 原始KL值: {val_epoch_stats['raw_kl']:.6f}")
            print(f"  - MC样本均值: {val_epoch_stats['mc_mean']:.4f}, MC样本标准差: {val_epoch_stats['mc_std']:.4f}")
            print(f"  - NaN批次: {val_nan_batches}")
            
            # 对每5个epoch保存一次完整模型，减少存储需求
            if epoch % 5 == 0 or epoch == 1 or epoch == num_epochs:
                current_model_path = f'model_checkpoints/cvae_renyi_alpha{alpha}_epoch_{epoch}.pth'
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'train_stats': train_stats,
                    'val_stats': val_stats,
                    'alpha': alpha,
                }, current_model_path)
                print(f"已保存模型至: {current_model_path}")
            
            # 对于α=-0.7，使用重构损失和Rényi值的加权组合作为质量指标
            # α=-0.7时权重介于α=-0.3和α=-1之间
            model_quality = val_epoch_stats['recon_loss'] - 0.25 * val_epoch_stats['raw_renyi']
            
            if model_quality < best_val_loss:
                best_val_loss = model_quality
                patience_counter = 0
                
                best_model_path = f'model_checkpoints/cvae_renyi_alpha{alpha}_best.pth'
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'best_quality': model_quality,
                    'train_stats': train_stats,
                    'val_stats': val_stats,
                    'alpha': alpha,
                }, best_model_path)
                print(f"保存最佳模型: {best_model_path} (质量指标: {model_quality:.4f})")
            else:
                patience_counter += 1
                print(f"模型质量未改善，耐心计数: {patience_counter}/{patience}")
                if patience_counter >= patience:
                    print(f"提前停止训练！模型质量{patience}轮未改善。")
                    return model, train_stats, val_stats
        else:
            print(f"验证集所有批次均为NaN，无法计算平均损失")
    
    return model, train_stats, val_stats


# ------------------------ #
# 11. 可视化函数
# ------------------------ #

def plot_training_stats(train_stats, val_stats, save_path='training_stats.png'):
    """
    绘制训练过程中的各种统计数据
    """
    import matplotlib.pyplot as plt
    
    epochs = range(1, len(train_stats['total_loss']) + 1)
    
    plt.figure(figsize=(18, 12))
    
    # 绘制总损失
    plt.subplot(2, 3, 1)
    plt.plot(epochs, train_stats['total_loss'], 'b-', label='训练')
    plt.plot(epochs, val_stats['total_loss'], 'r-', label='验证')
    plt.title('总损失')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # 绘制重构损失
    plt.subplot(2, 3, 2)
    plt.plot(epochs, train_stats['recon_loss'], 'b-', label='训练')
    plt.plot(epochs, val_stats['recon_loss'], 'r-', label='验证')
    plt.title('重构损失')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # 绘制Rényi损失
    plt.subplot(2, 3, 3)
    plt.plot(epochs, train_stats['renyi_loss'], 'b-', label='训练')
    plt.plot(epochs, val_stats['renyi_loss'], 'r-', label='验证')
    plt.title('Rényi损失 (加权)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # 绘制原始Rényi值
    plt.subplot(2, 3, 4)
    plt.plot(epochs, train_stats['raw_renyi'], 'b-', label='训练')
    plt.plot(epochs, val_stats['raw_renyi'], 'r-', label='验证')
    plt.title('原始Rényi值')
    plt.xlabel('Epoch')
    plt.ylabel('Value')
    plt.legend()
    
    # 绘制KL损失
    plt.subplot(2, 3, 5)
    plt.plot(epochs, train_stats['kl_loss'], 'b-', label='训练')
    plt.plot(epochs, val_stats['kl_loss'], 'r-', label='验证')
    plt.title('KL损失 (加权)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # 绘制原始KL值
    plt.subplot(2, 3, 6)
    plt.plot(epochs, train_stats['raw_kl'], 'b-', label='训练')
    plt.plot(epochs, val_stats['raw_kl'], 'r-', label='验证')
    plt.title('原始KL值')
    plt.xlabel('Epoch')
    plt.ylabel('Value')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()
    print(f"训练统计图已保存至 {save_path}")

# ------------------------ #
# 12. 样本生成与可视化
# ------------------------ #

def generate_samples(model, val_loader, num_samples=10, save_dir='sample_images'):
    """
    生成样本并可视化比较
    """
    import matplotlib.pyplot as plt
    from torchvision.utils import make_grid
    
    os.makedirs(save_dir, exist_ok=True)
    
    model.eval()
    
    # 随机获取一些验证集样本
    data_iter = iter(val_loader)
    data, attrs, clip_emb = next(data_iter)
    
    data = data[:num_samples].to(device)
    attrs = attrs[:num_samples].to(device)
    clip_emb = clip_emb[:num_samples].to(device)
    
    with torch.no_grad():
        # 获取原始图像、重构图像
        recon_img, mu, logvar = model(data, attrs, clip_emb)
        
        # 从潜在空间采样新图像
        z = torch.randn_like(mu)
        sampled_img = model.decode(z, attrs, clip_emb)
    
    # 将张量转换为可视化的图像
    # 去归一化，从[-1, 1]到[0, 1]
    def denormalize(x):
        return (x + 1) / 2
    
    original_imgs = denormalize(data)
    recon_imgs = denormalize(recon_img)
    sampled_imgs = denormalize(sampled_img)
    
    # 显示原始、重构和采样的图像
    fig, axes = plt.subplots(3, num_samples, figsize=(num_samples * 2, 6))
    
    # 去除坐标轴
    for ax in axes.flatten():
        ax.axis('off')
    
    # 标题
    axes[0, num_samples//2].set_title('原始图像', fontsize=12, pad=10)
    axes[1, num_samples//2].set_title('重构图像', fontsize=12, pad=10)
    axes[2, num_samples//2].set_title('从潜在空间采样', fontsize=12, pad=10)
    
    # 显示图像
    for i in range(num_samples):
        # 原始图像
        img = original_imgs[i].cpu().permute(1, 2, 0).numpy()
        img = np.clip(img, 0, 1)
        axes[0, i].imshow(img)
        
        # 重构图像
        img = recon_imgs[i].cpu().permute(1, 2, 0).numpy()
        img = np.clip(img, 0, 1)
        axes[1, i].imshow(img)
        
        # 采样图像
        img = sampled_imgs[i].cpu().permute(1, 2, 0).numpy()
        img = np.clip(img, 0, 1)
        axes[2, i].imshow(img)
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, 'sample_comparison.png'), dpi=150)
    plt.close()
    
    print(f"样本可视化已保存至 {os.path.join(save_dir, 'sample_comparison.png')}")
    
    return {
        'original': original_imgs.cpu(),
        'recon': recon_imgs.cpu(),
        'sampled': sampled_imgs.cpu()
    }

# ------------------------ #
# 13. 主程序
# ------------------------ #
if __name__ == "__main__":
    try:
        # 设置随机种子确保可重复性
        torch.manual_seed(42)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(42)
        np.random.seed(42)
        random.seed(42)
        
        # 设置α=-0.7
        alpha = -0.7
        
        # 创建模型实例
        model_renyi_neg = ClipCVAE_Renyi(img_channels=3, img_size=64, latent_dim=128, 
                                   cond_dim=40, clip_dim=512).to(device)
        
        # 初始化权重 - 为α=-0.7调整初始化参数
        for name, param in model_renyi_neg.named_parameters():
            if 'weight' in name:
                if 'fc_mu' in name or 'fc_logvar' in name:
                    # 使用介于α=-0.3和α=-1之间的初始化标准差
                    nn.init.normal_(param, mean=0.0, std=0.002)
                elif param.dim() >= 2:
                    nn.init.kaiming_normal_(param, mode='fan_out', nonlinearity='relu')
                else:
                    nn.init.normal_(param, mean=0.0, std=0.008)
            elif 'bias' in name:
                nn.init.zeros_(param)
        
        print(f"已成功完成模型参数初始化，使用α={alpha}的Rényi散度计算方法")
        
        # 优化器参数介于α=-0.3和α=-1之间
        optimizer = optim.AdamW(model_renyi_neg.parameters(), lr=2e-5, weight_decay=8e-3, eps=1e-5)
        
        # 使用余弦退火调度器，添加一次热重启以帮助模型跳出局部最小值
        scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer, T_0=8, T_mult=1, eta_min=5e-7
        )
        
        # 训练模型
        trained_model, train_stats, val_stats = train_negative_alpha_model(
            model_renyi_neg, train_loader, val_loader, optimizer, scheduler, 
            num_epochs=30, alpha=alpha
        )
        
        # 保存最终模型
        final_model_path = f'model_checkpoints/cvae_renyi_alpha{alpha}_final.pth'
        torch.save({
            'model_state_dict': trained_model.state_dict(),
            'train_stats': train_stats,
            'val_stats': val_stats,
            'alpha': alpha
        }, final_model_path)
        print(f"α={alpha}的Rényi散度CVAE模型已保存为: {final_model_path}")
        
        # 创建更详细的训练过程可视化函数
        def plot_enhanced_training_stats(train_stats, val_stats, save_path='training_stats.png'):
            """
            绘制更详细的训练过程统计图，特别针对α=-0.7
            """
            import matplotlib.pyplot as plt
            
            epochs = range(1, len(train_stats['total_loss']) + 1)
            
            # 创建一个更大的图形，包含更多统计数据
            plt.figure(figsize=(20, 16))
            
            # 1. 绘制总损失
            plt.subplot(3, 3, 1)
            plt.plot(epochs, train_stats['total_loss'], 'b-', label='训练')
            plt.plot(epochs, val_stats['total_loss'], 'r-', label='验证')
            plt.title('总损失')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.legend()
            
            # 2. 绘制重构损失
            plt.subplot(3, 3, 2)
            plt.plot(epochs, train_stats['recon_loss'], 'b-', label='训练')
            plt.plot(epochs, val_stats['recon_loss'], 'r-', label='验证')
            plt.title('重构损失')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.legend()
            
            # 3. 绘制Rényi损失
            plt.subplot(3, 3, 3)
            plt.plot(epochs, train_stats['renyi_loss'], 'b-', label='训练')
            plt.plot(epochs, val_stats['renyi_loss'], 'r-', label='验证')
            plt.title('Rényi损失 (加权)')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.legend()
            
            # 4. 绘制原始Rényi值
            plt.subplot(3, 3, 4)
            plt.plot(epochs, train_stats['raw_renyi'], 'b-', label='训练')
            plt.plot(epochs, val_stats['raw_renyi'], 'r-', label='验证')
            plt.title('原始Rényi值')
            plt.xlabel('Epoch')
            plt.ylabel('Value')
            plt.legend()
            
            # 5. 绘制原始KL值
            plt.subplot(3, 3, 5)
            plt.plot(epochs, train_stats['raw_kl'], 'b-', label='训练')
            plt.plot(epochs, val_stats['raw_kl'], 'r-', label='验证')
            plt.title('原始KL值')
            plt.xlabel('Epoch')
            plt.ylabel('Value')
            plt.legend()
            
            # 6. 绘制MC样本均值
            plt.subplot(3, 3, 6)
            plt.plot(epochs, train_stats['mc_mean'], 'b-', label='训练')
            plt.plot(epochs, val_stats['mc_mean'], 'r-', label='验证')
            plt.title('MC样本均值')
            plt.xlabel('Epoch')
            plt.ylabel('Value')
            plt.legend()
            
            # 7. 绘制MC样本标准差
            plt.subplot(3, 3, 7)
            plt.plot(epochs, train_stats['mc_std'], 'b-', label='训练')
            plt.plot(epochs, val_stats['mc_std'], 'r-', label='验证')
            plt.title('MC样本标准差')
            plt.xlabel('Epoch')
            plt.ylabel('Value')
            plt.legend()
            
            # 8. 绘制质量指标演变
            quality_train = [train_stats['recon_loss'][i] - 0.25 * train_stats['raw_renyi'][i] 
                            for i in range(len(epochs))]
            quality_val = [val_stats['recon_loss'][i] - 0.25 * val_stats['raw_renyi'][i] 
                          for i in range(len(epochs))]
            
            plt.subplot(3, 3, 8)
            plt.plot(epochs, quality_train, 'b-', label='训练')
            plt.plot(epochs, quality_val, 'r-', label='验证')
            plt.title('质量指标 (recon_loss - 0.25 * raw_renyi)')
            plt.xlabel('Epoch')
            plt.ylabel('Value')
            plt.legend()
            
            # 9. 绘制学习率变化（如果有记录）
            if 'learning_rate' in train_stats:
                plt.subplot(3, 3, 9)
                plt.plot(epochs, train_stats['learning_rate'], 'g-')
                plt.title('学习率')
                plt.xlabel('Epoch')
                plt.ylabel('Learning Rate')
            else:
                # 绘制Rényi对KL的比值
                renyi_to_kl_ratio = [train_stats['raw_renyi'][i] / (train_stats['raw_kl'][i] + 1e-10) 
                                    for i in range(len(epochs))]
                plt.subplot(3, 3, 9)
                plt.plot(epochs, renyi_to_kl_ratio, 'g-')
                plt.title('Rényi值/KL值比值')
                plt.xlabel('Epoch')
                plt.ylabel('Ratio')
            
            plt.tight_layout()
            plt.savefig(save_path, dpi=150)
            plt.close()
            print(f"增强版训练统计图已保存至 {save_path}")
        
        # 使用增强版可视化函数
        plot_enhanced_training_stats(train_stats, val_stats, 
                                     save_path=f'model_checkpoints/training_stats_alpha{alpha}_enhanced.png')
        
        # 使用标准可视化函数
        plot_training_stats(train_stats, val_stats, 
                            save_path=f'model_checkpoints/training_stats_alpha{alpha}.png')
        
        # 生成样本并可视化
        sample_results = generate_samples(
            trained_model, val_loader, num_samples=10, 
            save_dir=f'model_checkpoints/sample_images_alpha{alpha}'
        )
        
        print(f"使用α={alpha}的Rényi散度CVAE模型训练完成！")
        
        # 添加比较功能 - 如果有KL模型和其他alpha值的模型可用
        other_alpha_models = []
        model_paths = {
            'KL': '/root/autodl-tmp/models/clip_cvae_celeba.pth',
            '-0.3': 'model_checkpoints/cvae_renyi_alpha-0.3_final.pth',
            '-1.0': 'model_checkpoints/cvae_renyi_alpha-1.0_final.pth'
        }
        
        available_models = []
        for alpha_value, path in model_paths.items():
            if os.path.exists(path):
                available_models.append((alpha_value, path))
        
        if len(available_models) > 0:
            print(f"检测到{len(available_models)}个可比较的模型:")
            for alpha_value, path in available_models:
                print(f"  - α={alpha_value}: {path}")
            print("请考虑使用'evaluate_models'函数比较不同α值的效果。")
        
        # 尝试比较当前训练的α=-0.7模型与KL模型
        kl_model_path = '/root/autodl-tmp/models/clip_cvae_celeba.pth'
        if os.path.exists(kl_model_path) and os.path.exists(final_model_path):
            print("\n是否要对α=-0.7模型和KL模型进行快速比较？")
            print("这将对两个模型在少量测试图像上进行重构对比。")
            print("请确保您有相应的测试数据集。")
            
            # 由于这是一个Jupyter环境，此处不等待输入，直接进行简单评估
            try:
                # 创建测试数据集实例
                print("加载测试集进行简单比较...")
                test_embeddings_path = '/root/autodl-tmp/clip_embeddings/image_embeddings_partition_2.pt'
                if os.path.exists(test_embeddings_path):
                    test_dataset = CelebADataset(img_dir, attr_path, bbox_path, partition_path, 
                                                test_embeddings_path, transform=transform, partition=2)
                    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=2)
                    
                    # 加载KL模型
                    kl_model = ClipCVAE(img_channels=3, img_size=64, latent_dim=128, 
                                        cond_dim=40, clip_dim=512).to(device)
                    kl_model.load_state_dict(torch.load(kl_model_path))
                    kl_model.eval()
                    
                    # 对两个模型进行简单对比
                    with torch.no_grad():
                        # 获取一批测试数据
                        data_iter = iter(test_loader)
                        data, attrs, clip_emb = next(data_iter)
                        data = data[:8].to(device)  # 只使用8张图片
                        attrs = attrs[:8].to(device)
                        clip_emb = clip_emb[:8].to(device)
                        
                        # 重构图像
                        kl_recon, _, _ = kl_model(data, attrs, clip_emb)
                        renyi_recon, _, _ = trained_model(data, attrs, clip_emb)
                        
                        # 计算重构误差
                        kl_mse = F.mse_loss(kl_recon, data).item()
                        renyi_mse = F.mse_loss(renyi_recon, data).item()
                        
                        print(f"快速比较结果 (8张测试图像):")
                        print(f"  - KL模型重构MSE: {kl_mse:.6f}")
                        print(f"  - Rényi模型(α={alpha})重构MSE: {renyi_mse:.6f}")
                        print(f"  - 改进幅度: {((kl_mse - renyi_mse) / kl_mse * 100):.2f}%")
                        
                        # 可视化比较
                        # 去归一化为[0,1]
                        def denorm(x):
                            return (x + 1) / 2
                        
                        orig_imgs = denorm(data)
                        kl_imgs = denorm(kl_recon)
                        renyi_imgs = denorm(renyi_recon)
                        
                        # 保存比较图
                        import matplotlib.pyplot as plt
                        
                        fig, axes = plt.subplots(3, 8, figsize=(16, 6))
                        
                        for ax in axes.flatten():
                            ax.axis('off')
                            
                        for i in range(8):
                            # 原始图像
                            axes[0, i].imshow(orig_imgs[i].cpu().permute(1, 2, 0).numpy())
                            if i == 0:
                                axes[0, i].set_title("原始", fontsize=12)
                            
                            # KL重构
                            axes[1, i].imshow(kl_imgs[i].cpu().permute(1, 2, 0).numpy())
                            if i == 0:
                                axes[1, i].set_title("KL重构", fontsize=12)
                            
                            # Rényi重构
                            axes[2, i].imshow(renyi_imgs[i].cpu().permute(1, 2, 0).numpy())
                            if i == 0:
                                axes[2, i].set_title(f"Rényi重构 (α={alpha})", fontsize=12)
                        
                        plt.tight_layout()
                        comp_path = f'model_checkpoints/comparison_kl_vs_renyi_alpha{alpha}.png'
                        plt.savefig(comp_path, dpi=150)
                        plt.close()
                        print(f"比较可视化已保存至: {comp_path}")
                        
                else:
                    print("找不到测试集嵌入，跳过快速比较。")
            except Exception as comp_err:
                print(f"比较过程中出错: {comp_err}")
        
    except Exception as e:
        print(f"训练过程中发生错误: {e}")
        import traceback
        traceback.print_exc()
        # 保存紧急状态
        emergency_save_path = f'model_checkpoints/cvae_renyi_alpha{alpha}_emergency_save.pth'
        torch.save(model_renyi_neg.state_dict(), emergency_save_path)
        print(f"发生错误，模型紧急保存为: {emergency_save_path}")

使用设备: cuda
CLIP模型已加载。
Attributes DataFrame columns: ['image_id', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive', 'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose', 'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows', 'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace', 'Wearing_Necktie', 'Young']
Attributes DataFrame head:
      image_id  5_o_Clock_Shadow  Arched_Eyebrows  Attractive  Bags_Under_Eyes  \
0  000001.jpg                -1                1           1               -1   
1  000002.jpg                -1               -1          -1                1   
2  000003.jpg                -1               -1          -1               -1   

Epoch 1/30:   0%|          | 1/1272 [00:00<12:02,  1.76it/s]


批次 0 详细信息:
  - 原始α=-0.7 Rényi值: -1.223199, 加权Rényi损失: -1.518575
  - 原始KL值: 0.000056
  - 重构损失: 36.092659, 总损失: 34.574086
  - MC样本均值: -0.0000, MC样本标准差: 0.0009
  - mu范围: [-0.0037, 0.0040], var范围: [0.995508, 1.003539]
  - 分布统计: |μ|平均=0.0006, σ²平均=1.0001, log(σ²)平均=0.0001
  - 潜变量z统计: 均值=-0.0051, 标准差=1.0040, 范围=[-3.9397, 3.6580]


Epoch 1/30:   8%|▊         | 102/1272 [00:06<01:06, 17.53it/s]


批次 100 详细信息:
  - 原始α=-0.7 Rényi值: -1.223205, 加权Rényi损失: -1.518582
  - 原始KL值: 0.000058
  - 重构损失: 35.630398, 总损失: 34.111816
  - MC样本均值: 0.0000, MC样本标准差: 0.0010
  - mu范围: [-0.0037, 0.0038], var范围: [0.995111, 1.003847]
  - 分布统计: |μ|平均=0.0006, σ²平均=1.0001, log(σ²)平均=0.0001
  - 潜变量z统计: 均值=-0.0099, 标准差=0.9958, 范围=[-4.1632, 3.4638]


...

增强版训练统计图已保存至 model_checkpoints/training_stats_alpha-0.7_enhanced.png


/tmp/ipykernel_12155/2329540904.py:670: UserWarning: Glyph 24635 (\N{CJK UNIFIED IDEOGRAPH-603B}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_12155/2329540904.py:670: UserWarning: Glyph 25439 (\N{CJK UNIFIED IDEOGRAPH-635F}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_12155/2329540904.py:670: UserWarning: Glyph 22833 (\N{CJK UNIFIED IDEOGRAPH-5931}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_12155/2329540904.py:670: UserWarning: Glyph 35757 (\N{CJK UNIFIED IDEOGRAPH-8BAD}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_12155/2329540904.py:670: UserWarning: Glyph 32451 (\N{CJK UNIFIED IDEOGRAPH-7EC3}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_12155/2329540904.py:670: UserWarning: Glyph 39564 (\N{CJK UNIFIED IDEOGRAPH-9A8C}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_12155/2329540904.py:670: UserWarning: Glyph 35777 (\N{CJK UNIFIED I

训练统计图已保存至 model_checkpoints/training_stats_alpha-0.7.png


/tmp/ipykernel_12155/2329540904.py:744: UserWarning: Glyph 21407 (\N{CJK UNIFIED IDEOGRAPH-539F}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_12155/2329540904.py:744: UserWarning: Glyph 22987 (\N{CJK UNIFIED IDEOGRAPH-59CB}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_12155/2329540904.py:744: UserWarning: Glyph 22270 (\N{CJK UNIFIED IDEOGRAPH-56FE}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_12155/2329540904.py:744: UserWarning: Glyph 20687 (\N{CJK UNIFIED IDEOGRAPH-50CF}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_12155/2329540904.py:744: UserWarning: Glyph 37325 (\N{CJK UNIFIED IDEOGRAPH-91CD}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_12155/2329540904.py:744: UserWarning: Glyph 26500 (\N{CJK UNIFIED IDEOGRAPH-6784}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_12155/2329540904.py:744: UserWarning: Glyph 20174 (\N{CJK UNIFIED I

样本可视化已保存至 model_checkpoints/sample_images_alpha-0.7/sample_comparison.png
使用α=-0.7的Rényi散度CVAE模型训练完成！
检测到2个可比较的模型:
  - α=-0.3: model_checkpoints/cvae_renyi_alpha-0.3_final.pth
  - α=-1.0: model_checkpoints/cvae_renyi_alpha-1.0_final.pth
请考虑使用'evaluate_models'函数比较不同α值的效果。


In [1]:
import os
import math
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import clip
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm

# ------------------------ #
# 1. 设备设置
# ------------------------ #

# 选择设备（GPU或CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")

# ------------------------ #
# 2. 加载CLIP模型
# ------------------------ #

# 加载CLIP模型和预处理方法
clip_model, preprocess = clip.load("ViT-B/32", device=device)
clip_model.eval()
print("CLIP模型已加载。")

# ------------------------ #
# 3. 定义CelebA数据集类
# ------------------------ #

class CelebADataset(Dataset):
    def __init__(self, img_dir, attr_path, bbox_path, partition_path, 
                 image_embeddings_path, transform=None, partition=0):
        """
        初始化数据集
        :param img_dir: 图像文件夹路径
        :param attr_path: 属性文件路径
        :param bbox_path: 边界框文件路径
        :param partition_path: 分区文件路径
        :param image_embeddings_path: 预先计算的图像嵌入文件路径
        :param transform: 图像预处理
        :param partition: 使用的数据分区 (0: train, 1: val, 2: test)
        """
        self.img_dir = img_dir
        self.transform = transform

        # 读取属性文件
        try:
            attr_df = pd.read_csv(attr_path, sep=',', header=0)
        except Exception as e:
            print(f"Error reading {attr_path}: {e}")
            raise

        print("Attributes DataFrame columns:", attr_df.columns.tolist())
        print("Attributes DataFrame head:\n", attr_df.head())

        # 确认 'image_id' 列存在
        if 'image_id' not in attr_df.columns:
            raise ValueError("属性文件中缺少 'image_id' 列。请检查文件格式。")
        else:
            print("'image_id' column found in attributes DataFrame.")

        # 读取分区文件
        try:
            partition_df = pd.read_csv(partition_path, sep=',', header=0, names=['image_id', 'partition'])
        except Exception as e:
            print(f"Error reading {partition_path}: {e}")
            raise

        print("Partition DataFrame columns:", partition_df.columns.tolist())
        print("Partition DataFrame head:\n", partition_df.head())

        # 读取边界框文件
        try:
            bbox_df = pd.read_csv(bbox_path, sep=',', header=0, names=['image_id', 'x_1', 'y_1', 'width', 'height'])
        except Exception as e:
            print(f"Error reading {bbox_path}: {e}")
            raise

        print("Bounding Box DataFrame columns:", bbox_df.columns.tolist())
        print("Bounding Box DataFrame head:\n", bbox_df.head())

        # 合并属性和分区数据
        try:
            attr_df = attr_df.merge(partition_df, on='image_id')
        except Exception as e:
            print(f"Error merging attributes and partitions: {e}")
            raise

        print("After merging with partition, columns:", attr_df.columns.tolist())
        print("Merged DataFrame head:\n", attr_df.head())

        # 合并属性、分区和边界框数据
        try:
            attr_df = attr_df.merge(bbox_df, on='image_id')
        except Exception as e:
            print(f"Error merging attributes, partitions and bounding boxes: {e}")
            raise

        print("After merging with bounding box, columns:", attr_df.columns.tolist())
        print("Final Merged DataFrame head:\n", attr_df.head())

        # 筛选分区
        print(f"Total samples before filtering: {len(attr_df)}")
        self.attr_df = attr_df[attr_df['partition'] == partition].reset_index(drop=True)
        print(f"Total samples after filtering partition {partition}: {len(self.attr_df)}")

        if len(self.attr_df) == 0:
            raise ValueError(f"训练集中没有样本。请检查数据文件的读取和过滤逻辑。")

        # 获取属性名称
        # 属性列是除了 'image_id', 'partition', 'x_1', 'y_1', 'width', 'height' 之外的所有列
        self.attr_names = [col for col in attr_df.columns if col not in ['image_id', 'partition', 'x_1', 'y_1', 'width', 'height']]
        print("Attribute names:", self.attr_names)

        # 加载预计算的图像嵌入
        self.image_embeddings = torch.load(image_embeddings_path)  # 假设是一个Tensor，shape=(num_images, 512)
        if len(self.image_embeddings) != len(self.attr_df):
            raise ValueError("图像嵌入的数量与数据集中的图像数量不一致。")
        self.image_embeddings = self.image_embeddings.float()

    def __len__(self):
        return len(self.attr_df)

    def __getitem__(self, idx):
        # 获取图像路径
        img_name = self.attr_df.iloc[idx, self.attr_df.columns.get_loc('image_id')]
        img_path = os.path.join(self.img_dir, img_name)
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Error opening image {img_path}: {e}")
            # 可以选择跳过该样本，或返回一个默认图像
            # 这里选择抛出异常
            raise

        if self.transform:
            image = self.transform(image)

        # 获取属性标签 
        attrs = self.attr_df.iloc[idx][self.attr_names].values
        attrs = (attrs + 1) // 2  # 将-1转为0，1保持1
        attrs = attrs.astype(np.float32)

        # 获取对应的图像嵌入
        clip_embedding = self.image_embeddings[idx]

        return image, attrs, clip_embedding

# ------------------------ #
# 4. 定义ClipCVAE模型（使用Renyi散度）
# ------------------------ #

class ClipCVAE_Renyi(nn.Module):
    def __init__(self, img_channels=3, img_size=64, latent_dim=128, 
                 cond_dim=40, clip_dim=512):
        super(ClipCVAE_Renyi, self).__init__()
        self.img_size = img_size
        self.latent_dim = latent_dim
        self.cond_dim = cond_dim
        self.clip_dim = clip_dim

        # 编码器部分
        self.encoder = nn.Sequential(
            nn.Conv2d(img_channels + cond_dim + clip_dim, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Flatten()
        )
        self.fc_mu = nn.Linear(512*4*4, latent_dim)
        self.fc_logvar = nn.Linear(512*4*4, latent_dim)

        # 解码器部分
        self.decoder_input = nn.Linear(latent_dim + cond_dim + clip_dim, 512*4*4)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, img_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def encode(self, x, c, clip_embedding):
        # 将条件标签、图像和CLIP嵌入拼接
        c = c.view(c.size(0), self.cond_dim, 1, 1).repeat(1, 1, self.img_size, self.img_size)
        clip_embedding = clip_embedding.view(clip_embedding.size(0), self.clip_dim, 1, 1).repeat(1, 1, self.img_size, self.img_size)
        x = torch.cat([x, c, clip_embedding], dim=1)
        x = self.encoder(x)
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z, c, clip_embedding):
        # 将潜在变量、条件标签和CLIP嵌入拼接
        z = torch.cat([z, c, clip_embedding], dim=1)
        x = self.decoder_input(z)
        x = x.view(-1, 512, 4, 4)
        x = self.decoder(x)
        return x

    def forward(self, x, c, clip_embedding):
        mu, logvar = self.encode(x, c, clip_embedding)
        z = self.reparameterize(mu, logvar)
        recon_x = self.decode(z, c, clip_embedding)
        return recon_x, mu, logvar

# ------------------------ #
# 5. 图像预处理
# ------------------------ #

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # 归一化
])

# ------------------------ #
# 6. 数据集路径设置
# ------------------------ #

# 请根据实际路径修改
img_dir = '/root/autodl-tmp/celeba_datasets/img_align_celeba/img_align_celeba'
attr_path = '/root/autodl-tmp/celeba_datasets/list_attr_celeba.txt'
bbox_path = '/root/autodl-tmp/celeba_datasets/list_bbox_celeba.txt'
partition_path = '/root/autodl-tmp/celeba_datasets/list_eval_partition.txt'
image_embeddings_train_path = '/root/autodl-tmp/clip_embeddings/image_embeddings_partition_0.pt'
image_embeddings_val_path = '/root/autodl-tmp/clip_embeddings/image_embeddings_partition_1.pt'

# ------------------------ #
# 7. 创建训练集和验证集
# ------------------------ #

train_dataset = CelebADataset(img_dir, attr_path, bbox_path, partition_path, 
                              image_embeddings_train_path, transform=transform, partition=0)
val_dataset = CelebADataset(img_dir, attr_path, bbox_path, partition_path, 
                            image_embeddings_val_path, transform=transform, partition=1)

# ------------------------ #
# 8. 数据加载器
# ------------------------ #

batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# ------------------------ #
# 9. 修订版 Rényi 散度损失函数
# ------------------------ #

import torch
import torch.nn.functional as F
import math

def negative_alpha_renyi_divergence_loss(recon_x, x, mu, logvar, epoch=1, alpha=-2.0, K=12):
    """
    使用α=-2.0的Rényi散度损失函数，基于变分Rényi边界理论
    
    参数:
    - recon_x: 重构的图像
    - x: 原始图像
    - mu: 均值向量
    - logvar: 对数方差向量
    - epoch: 当前训练轮数
    - alpha: Rényi散度参数，这里使用-2.0
    - K: 蒙特卡洛样本数量
    """
    # 重构损失计算
    batch_size = x.size(0)
    recon_loss = F.mse_loss(recon_x, x, reduction='mean') * 100
    
    # 限制参数范围 - α=-2.0需要比α=-0.7/-1.0更严格的限制
    logvar = torch.clamp(logvar, min=-16.0, max=12.0)
    var = torch.exp(logvar)
    
    # 标准正态先验
    var0 = torch.ones_like(var)
    mu0 = torch.zeros_like(mu)
    
    # 计算KL散度的成分（用于监控）
    mu_diff_sq = (mu - mu0).pow(2)
    trace_term = var / var0
    logdet_term = torch.log(var0) - logvar
    
    # 基础KL散度项（用于监控）
    kl_base = 0.5 * (mu_diff_sq / var0 + trace_term + logdet_term - 1)
    
    # α=-2.0时的Rényi散度计算 - 使用蒙特卡洛采样
    # 对于α=-2.0，我们需要更多的样本来稳定估计
    mc_samples = []
    for _ in range(max(K, 12)):  # 使用至少12个样本以提高稳定性
        eps = torch.randn_like(std := torch.sqrt(var))
        z = mu + eps * std
        
        # 计算log概率的差值
        log_q = -0.5 * (torch.log(2 * math.pi * var) + (z - mu).pow(2) / var)
        log_p = -0.5 * (torch.log(2 * math.pi * var0) + (z - mu0).pow(2) / var0)
        log_ratio = log_p - log_q
        
        # 对于α=-2.0需要更严格的裁剪
        log_ratio = torch.clamp(log_ratio, min=-15.0, max=15.0)
        mc_samples.append(log_ratio)
    
    # 计算MC样本的Rényi散度估计
    log_weights = torch.stack(mc_samples, dim=0)
    
    # α=-2.0时的公式: 1/(α-1) * log(E[exp((1-α)*log(p/q))])
    # 对于α=-2.0: 1/(-2.0-1) * log(E[exp((1-(-2.0))*log(p/q))]) = -0.333 * log(E[exp(3.0*log(p/q))])
    renyi_div = (1 / (alpha - 1)) * torch.logsumexp((1 - alpha) * log_weights, dim=0).mean()
    
    # 确保数值稳定性 - α=-2.0需要更严格的裁剪
    renyi_div = torch.clamp(renyi_div, min=-15.0, max=15.0)
    
    # α=-2.0的权重设置 - 需要更保守的设置以确保稳定性
    max_weight = 2.8  # 最大权重
    min_weight = 0.3  # 起始权重
    
    # 使用更平滑的权重增长曲线
    progress = min(epoch / 20.0, 1.0)  # 20轮过渡期，比α=-0.7更长
    weight_factor = min_weight + (max_weight - min_weight) * (1 - math.exp(-2.5 * progress))
    
    # 监控信息
    raw_renyi = renyi_div.item()
    raw_kl = kl_base.sum(dim=1).mean().item()
    
    # 总损失计算
    weighted_renyi = renyi_div * weight_factor
    total_loss = recon_loss + weighted_renyi
    
    # 额外监控 MC 样本的统计信息
    with torch.no_grad():
        mc_mean = log_weights.mean().item()
        mc_std = log_weights.std().item()
    
    return total_loss, recon_loss, weighted_renyi, raw_renyi, torch.tensor(0.0).to(recon_x.device), raw_kl, mc_mean, mc_std

# ------------------------ #
# 10. 改进的训练函数
# ------------------------ #
def train_negative_alpha_model(model, train_loader, val_loader, optimizer, scheduler, num_epochs=30, alpha=-2.0):
    """
    使用α=-2.0的Rényi散度训练CVAE模型
    """
    print(f"开始使用负α值(α={alpha})的Rényi散度训练CVAE模型...")
    
    # 记录训练统计信息
    train_stats = {
        'total_loss': [], 'recon_loss': [], 'renyi_loss': [], 
        'raw_renyi': [], 'kl_loss': [], 'raw_kl': [],
        'mc_mean': [], 'mc_std': []  # 添加MC样本统计
    }
    val_stats = {
        'total_loss': [], 'recon_loss': [], 'renyi_loss': [], 
        'raw_renyi': [], 'kl_loss': [], 'raw_kl': [],
        'mc_mean': [], 'mc_std': []
    }
    
    # 最佳模型跟踪
    best_val_loss = float('inf')
    patience = 20  # 对于α=-2.0，使用更长的耐心值
    patience_counter = 0
    
    # 确保保存目录存在
    os.makedirs('model_checkpoints', exist_ok=True)
    
    # α=-2.0的特殊处理 - 增加梯度累积以提高稳定性
    accumulation_steps = 5  # 每5个批次更新一次
    
    for epoch in range(1, num_epochs + 1):
        model.train()
        epoch_stats = {k: 0.0 for k in train_stats.keys()}
        nan_batches = 0
        
        # 计算当前权重
        progress = min(epoch / 20.0, 1.0)
        min_weight = 0.3
        max_weight = 2.8
        weight_factor = min_weight + (max_weight - min_weight) * (1 - math.exp(-2.5 * progress))
        
        print(f"Epoch {epoch}/{num_epochs}, α={alpha} Rényi权重: {weight_factor:.4f}")
        
        # 训练循环
        for batch_idx, (data, attrs, clip_emb) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}")):
            data = data.to(device, non_blocking=True)
            attrs = attrs.to(device, non_blocking=True)
            clip_emb = clip_emb.to(device, non_blocking=True)
            
            # 只在累积步骤的开始时清零梯度
            if batch_idx % accumulation_steps == 0:
                optimizer.zero_grad()
            
            try:
                recon_batch, mu, logvar = model(data, attrs, clip_emb)
                
                # 使用α=-2.0的Rényi散度损失函数
                loss, recon_loss, renyi_loss, raw_renyi, kl_loss, raw_kl, mc_mean, mc_std = negative_alpha_renyi_divergence_loss(
                    recon_batch, data, mu, logvar, epoch, alpha=alpha, K=12
                )
                
                # 按累积步骤缩放损失
                loss = loss / accumulation_steps
                
                # 详细调试输出
                if batch_idx % 100 == 0:
                    print(f"\n批次 {batch_idx} 详细信息:")
                    print(f"  - 原始α={alpha} Rényi值: {raw_renyi:.6f}, 加权Rényi损失: {renyi_loss.item():.6f}")
                    print(f"  - 原始KL值: {raw_kl:.6f}")
                    print(f"  - 重构损失: {recon_loss.item():.6f}, 总损失: {loss.item()*accumulation_steps:.6f}")
                    print(f"  - MC样本均值: {mc_mean:.4f}, MC样本标准差: {mc_std:.4f}")
                    print(f"  - mu范围: [{mu.min().item():.4f}, {mu.max().item():.4f}], "
                          f"var范围: [{torch.exp(logvar).min().item():.6f}, {torch.exp(logvar).max().item():.6f}]")
                    
                    # 额外调试信息，特别关注α=-2.0的数值特性
                    with torch.no_grad():
                        # 收集分布统计数据
                        variance = torch.exp(logvar)  # 计算方差
                        mu_abs_mean = mu.abs().mean().item()
                        var_mean = variance.mean().item() 
                        logvar_mean = logvar.mean().item()
                        
                        print(f"  - 分布统计: |μ|平均={mu_abs_mean:.4f}, σ²平均={var_mean:.4f}, log(σ²)平均={logvar_mean:.4f}")
                        
                        # 检查重参数化过程
                        z = model.reparameterize(mu, logvar)
                        z_mean = z.mean().item()
                        z_std = z.std().item()
                        z_min = z.min().item()
                        z_max = z.max().item()
                        
                        print(f"  - 潜变量z统计: 均值={z_mean:.4f}, 标准差={z_std:.4f}, 范围=[{z_min:.4f}, {z_max:.4f}]")
                
                # 检查NaN和异常值 - α=-2.0需要更严格的检查
                if torch.isnan(loss).any() or torch.isinf(loss).any() or raw_renyi > 30 or raw_renyi < -30:
                    nan_batches += 1
                    if batch_idx % 100 == 0:
                        print(f"警告: 第{epoch}轮, 第{batch_idx}批次出现异常损失值。跳过此批次.")
                    continue
                
                # 反向传播
                loss.backward()
                
                # 仅在累积步骤结束时更新权重
                if (batch_idx + 1) % accumulation_steps == 0 or (batch_idx + 1) == len(train_loader):
                    # 更严格的梯度裁剪 - α=-2.0的适当裁剪值
                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
                    optimizer.step()
                
                # 累计统计信息
                epoch_stats['total_loss'] += loss.item() * accumulation_steps
                epoch_stats['recon_loss'] += recon_loss.item()
                epoch_stats['renyi_loss'] += renyi_loss.item()
                epoch_stats['raw_renyi'] += raw_renyi
                epoch_stats['kl_loss'] += kl_loss.item()
                epoch_stats['raw_kl'] += raw_kl
                epoch_stats['mc_mean'] += mc_mean
                epoch_stats['mc_std'] += mc_std
                
            except RuntimeError as e:
                print(f"批次{batch_idx}处理出错: {e}")
                nan_batches += 1
                continue
        
        # 更新学习率
        scheduler.step()
        current_lr = optimizer.param_groups[0]["lr"]
        
        # 计算平均损失
        processed_batches = len(train_loader) - nan_batches
        if processed_batches > 0:
            for k in epoch_stats:
                epoch_stats[k] /= processed_batches
                train_stats[k].append(epoch_stats[k])
            
            print(f"\nEpoch {epoch} 训练统计:")
            print(f"  - 平均总损失: {epoch_stats['total_loss']:.4f}")
            print(f"  - 重构损失: {epoch_stats['recon_loss']:.4f}")
            print(f"  - α={alpha} Rényi损失: {epoch_stats['renyi_loss']:.4f} (原始值: {epoch_stats['raw_renyi']:.6f})")
            print(f"  - 原始KL值: {epoch_stats['raw_kl']:.6f}")
            print(f"  - MC样本均值: {epoch_stats['mc_mean']:.4f}, MC样本标准差: {epoch_stats['mc_std']:.4f}")
            print(f"  - 学习率: {current_lr:.6f}, NaN批次: {nan_batches}")
        else:
            print(f"Epoch {epoch}, 所有批次均为NaN，无法计算平均损失")
            continue
        
        # 验证集评估
        model.eval()
        val_epoch_stats = {k: 0.0 for k in val_stats.keys()}
        val_nan_batches = 0
        
        with torch.no_grad():
            for data, attrs, clip_emb in val_loader:
                data = data.to(device, non_blocking=True)
                attrs = attrs.to(device, non_blocking=True)
                clip_emb = clip_emb.to(device, non_blocking=True)
                
                try:
                    recon_batch, mu, logvar = model(data, attrs, clip_emb)
                    loss, recon_loss, renyi_loss, raw_renyi, kl_loss, raw_kl, mc_mean, mc_std = negative_alpha_renyi_divergence_loss(
                        recon_batch, data, mu, logvar, epoch, alpha=alpha, K=12
                    )
                    
                    if torch.isnan(loss).any() or torch.isinf(loss).any() or raw_renyi > 30 or raw_renyi < -30:
                        val_nan_batches += 1
                        continue
                    
                    # 累计验证统计
                    val_epoch_stats['total_loss'] += loss.item()
                    val_epoch_stats['recon_loss'] += recon_loss.item()
                    val_epoch_stats['renyi_loss'] += renyi_loss.item()
                    val_epoch_stats['raw_renyi'] += raw_renyi
                    val_epoch_stats['kl_loss'] += kl_loss.item()
                    val_epoch_stats['raw_kl'] += raw_kl
                    val_epoch_stats['mc_mean'] += mc_mean
                    val_epoch_stats['mc_std'] += mc_std
                    
                except RuntimeError:
                    val_nan_batches += 1
                    continue
        
        # 计算验证平均值
        processed_val_batches = len(val_loader) - val_nan_batches
        if processed_val_batches > 0:
            for k in val_epoch_stats:
                val_epoch_stats[k] /= processed_val_batches
                val_stats[k].append(val_epoch_stats[k])
            
            print(f"验证集统计:")
            print(f"  - 平均总损失: {val_epoch_stats['total_loss']:.4f}")
            print(f"  - 重构损失: {val_epoch_stats['recon_loss']:.4f}")
            print(f"  - α={alpha} Rényi损失: {val_epoch_stats['renyi_loss']:.4f} (原始值: {val_epoch_stats['raw_renyi']:.6f})")
            print(f"  - 原始KL值: {val_epoch_stats['raw_kl']:.6f}")
            print(f"  - MC样本均值: {val_epoch_stats['mc_mean']:.4f}, MC样本标准差: {val_epoch_stats['mc_std']:.4f}")
            print(f"  - NaN批次: {val_nan_batches}")
            
            # 对每3个epoch保存一次完整模型，增加保存频率以防止数据丢失
            if epoch % 3 == 0 or epoch == 1 or epoch == num_epochs:
                current_model_path = f'model_checkpoints/cvae_renyi_alpha{alpha}_epoch_{epoch}.pth'
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'train_stats': train_stats,
                    'val_stats': val_stats,
                    'alpha': alpha,
                }, current_model_path)
                print(f"已保存模型至: {current_model_path}")
            
            # 对于α=-2.0，使用重构损失和Rényi值的加权组合作为质量指标
            # α=-2.0时权重更趋向于重构质量
            model_quality = val_epoch_stats['recon_loss'] - 0.15 * val_epoch_stats['raw_renyi']
            
            if model_quality < best_val_loss:
                best_val_loss = model_quality
                patience_counter = 0
                
                best_model_path = f'model_checkpoints/cvae_renyi_alpha{alpha}_best.pth'
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'best_quality': model_quality,
                    'train_stats': train_stats,
                    'val_stats': val_stats,
                    'alpha': alpha,
                }, best_model_path)
                print(f"保存最佳模型: {best_model_path} (质量指标: {model_quality:.4f})")
            else:
                patience_counter += 1
                print(f"模型质量未改善，耐心计数: {patience_counter}/{patience}")
                if patience_counter >= patience:
                    print(f"提前停止训练！模型质量{patience}轮未改善。")
                    return model, train_stats, val_stats
        else:
            print(f"验证集所有批次均为NaN，无法计算平均损失")
    
    return model, train_stats, val_stats


# ------------------------ #
# 11. 可视化函数
# ------------------------ #

def plot_training_stats(train_stats, val_stats, save_path='training_stats.png'):
    """
    绘制训练过程中的各种统计数据
    """
    import matplotlib.pyplot as plt
    
    epochs = range(1, len(train_stats['total_loss']) + 1)
    
    plt.figure(figsize=(18, 12))
    
    # 绘制总损失
    plt.subplot(2, 3, 1)
    plt.plot(epochs, train_stats['total_loss'], 'b-', label='训练')
    plt.plot(epochs, val_stats['total_loss'], 'r-', label='验证')
    plt.title('总损失')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # 绘制重构损失
    plt.subplot(2, 3, 2)
    plt.plot(epochs, train_stats['recon_loss'], 'b-', label='训练')
    plt.plot(epochs, val_stats['recon_loss'], 'r-', label='验证')
    plt.title('重构损失')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # 绘制Rényi损失
    plt.subplot(2, 3, 3)
    plt.plot(epochs, train_stats['renyi_loss'], 'b-', label='训练')
    plt.plot(epochs, val_stats['renyi_loss'], 'r-', label='验证')
    plt.title('Rényi损失 (加权)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # 绘制原始Rényi值
    plt.subplot(2, 3, 4)
    plt.plot(epochs, train_stats['raw_renyi'], 'b-', label='训练')
    plt.plot(epochs, val_stats['raw_renyi'], 'r-', label='验证')
    plt.title('原始Rényi值')
    plt.xlabel('Epoch')
    plt.ylabel('Value')
    plt.legend()
    
    # 绘制KL损失
    plt.subplot(2, 3, 5)
    plt.plot(epochs, train_stats['kl_loss'], 'b-', label='训练')
    plt.plot(epochs, val_stats['kl_loss'], 'r-', label='验证')
    plt.title('KL损失 (加权)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # 绘制原始KL值
    plt.subplot(2, 3, 6)
    plt.plot(epochs, train_stats['raw_kl'], 'b-', label='训练')
    plt.plot(epochs, val_stats['raw_kl'], 'r-', label='验证')
    plt.title('原始KL值')
    plt.xlabel('Epoch')
    plt.ylabel('Value')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()
    print(f"训练统计图已保存至 {save_path}")

# ------------------------ #
# 12. 样本生成与可视化
# ------------------------ #

def generate_samples(model, val_loader, num_samples=10, save_dir='sample_images'):
    """
    生成样本并可视化比较
    """
    import matplotlib.pyplot as plt
    from torchvision.utils import make_grid
    
    os.makedirs(save_dir, exist_ok=True)
    
    model.eval()
    
    # 随机获取一些验证集样本
    data_iter = iter(val_loader)
    data, attrs, clip_emb = next(data_iter)
    
    data = data[:num_samples].to(device)
    attrs = attrs[:num_samples].to(device)
    clip_emb = clip_emb[:num_samples].to(device)
    
    with torch.no_grad():
        # 获取原始图像、重构图像
        recon_img, mu, logvar = model(data, attrs, clip_emb)
        
        # 从潜在空间采样新图像
        z = torch.randn_like(mu)
        sampled_img = model.decode(z, attrs, clip_emb)
    
    # 将张量转换为可视化的图像
    # 去归一化，从[-1, 1]到[0, 1]
    def denormalize(x):
        return (x + 1) / 2
    
    original_imgs = denormalize(data)
    recon_imgs = denormalize(recon_img)
    sampled_imgs = denormalize(sampled_img)
    
    # 显示原始、重构和采样的图像
    fig, axes = plt.subplots(3, num_samples, figsize=(num_samples * 2, 6))
    
    # 去除坐标轴
    for ax in axes.flatten():
        ax.axis('off')
    
    # 标题
    axes[0, num_samples//2].set_title('原始图像', fontsize=12, pad=10)
    axes[1, num_samples//2].set_title('重构图像', fontsize=12, pad=10)
    axes[2, num_samples//2].set_title('从潜在空间采样', fontsize=12, pad=10)
    
    # 显示图像
    for i in range(num_samples):
        # 原始图像
        img = original_imgs[i].cpu().permute(1, 2, 0).numpy()
        img = np.clip(img, 0, 1)
        axes[0, i].imshow(img)
        
        # 重构图像
        img = recon_imgs[i].cpu().permute(1, 2, 0).numpy()
        img = np.clip(img, 0, 1)
        axes[1, i].imshow(img)
        
        # 采样图像
        img = sampled_imgs[i].cpu().permute(1, 2, 0).numpy()
        img = np.clip(img, 0, 1)
        axes[2, i].imshow(img)
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, 'sample_comparison.png'), dpi=150)
    plt.close()
    
    print(f"样本可视化已保存至 {os.path.join(save_dir, 'sample_comparison.png')}")
    
    return {
        'original': original_imgs.cpu(),
        'recon': recon_imgs.cpu(),
        'sampled': sampled_imgs.cpu()
    }

# ------------------------ #
# 13. 主程序
# ------------------------ #
# ------------------------ #
# 13. 主程序
# ------------------------ #
if __name__ == "__main__":
    try:
        # 设置随机种子确保可重复性
        torch.manual_seed(42)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(42)
        np.random.seed(42)
        random.seed(42)
        
        # 设置α=-2.0
        alpha = -2.0
        
        # 创建模型实例
        model_renyi_neg = ClipCVAE_Renyi(img_channels=3, img_size=64, latent_dim=128, 
                                   cond_dim=40, clip_dim=512).to(device)
        
        # 初始化权重 - 为α=-2.0调整初始化参数，更加保守
        for name, param in model_renyi_neg.named_parameters():
            if 'weight' in name:
                if 'fc_mu' in name or 'fc_logvar' in name:
                    # 使用更小的初始化标准差以确保稳定性
                    nn.init.normal_(param, mean=0.0, std=0.001)
                elif param.dim() >= 2:
                    nn.init.kaiming_normal_(param, mode='fan_out', nonlinearity='relu')
                else:
                    nn.init.normal_(param, mean=0.0, std=0.005)
            elif 'bias' in name:
                nn.init.zeros_(param)
        
        print(f"已成功完成模型参数初始化，使用α={alpha}的Rényi散度计算方法")
        
        # 优化器设置 - 对于α=-2.0更保守的学习率和更高的权重衰减
        optimizer = optim.AdamW(model_renyi_neg.parameters(), lr=1e-5, weight_decay=1e-2, eps=1e-6)
        
        # 使用更平缓的学习率调度器
        scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer, T_0=12, T_mult=1, eta_min=1e-7
        )
        
        # 训练模型
        trained_model, train_stats, val_stats = train_negative_alpha_model(
            model_renyi_neg, train_loader, val_loader, optimizer, scheduler, 
            num_epochs=40, alpha=alpha  # 增加轮数以确保充分训练
        )
        
        # 保存最终模型
        final_model_path = f'model_checkpoints/cvae_renyi_alpha{alpha}_final.pth'
        torch.save({
            'model_state_dict': trained_model.state_dict(),
            'train_stats': train_stats,
            'val_stats': val_stats,
            'alpha': alpha
        }, final_model_path)
        print(f"α={alpha}的Rényi散度CVAE模型已保存为: {final_model_path}")
        
        # 使用增强版可视化函数
        plot_enhanced_training_stats(train_stats, val_stats, 
                                     save_path=f'model_checkpoints/training_stats_alpha{alpha}_enhanced.png')
        
        # 使用标准可视化函数
        plot_training_stats(train_stats, val_stats, 
                            save_path=f'model_checkpoints/training_stats_alpha{alpha}.png')
        
        # 生成样本并可视化
        sample_results = generate_samples(
            trained_model, val_loader, num_samples=10, 
            save_dir=f'model_checkpoints/sample_images_alpha{alpha}'
        )
        
        print(f"使用α={alpha}的Rényi散度CVAE模型训练完成！")
        
        # 添加比较功能 - 如果有KL模型和其他alpha值的模型可用
        other_alpha_models = []
        model_paths = {
            'KL': '/root/autodl-tmp/models/clip_cvae_celeba.pth',
            '-0.3': 'model_checkpoints/cvae_renyi_alpha-0.3_final.pth',
            '-0.7': 'model_checkpoints/cvae_renyi_alpha-0.7_final.pth',
            '-1.0': 'model_checkpoints/cvae_renyi_alpha-1.0_final.pth'
        }
        
        available_models = []
        for alpha_value, path in model_paths.items():
            if os.path.exists(path):
                available_models.append((alpha_value, path))
        
        if len(available_models) > 0:
            print(f"检测到{len(available_models)}个可比较的模型:")
            for alpha_value, path in available_models:
                print(f"  - α={alpha_value}: {path}")
            print("请考虑使用'evaluate_models'函数比较不同α值的效果。")
            
    except Exception as e:
        print(f"训练过程中发生错误: {e}")
        import traceback
        traceback.print_exc()
        # 保存紧急状态
        emergency_save_path = f'model_checkpoints/cvae_renyi_alpha{alpha}_emergency_save.pth'
        torch.save(model_renyi_neg.state_dict(), emergency_save_path)
        print(f"发生错误，模型紧急保存为: {emergency_save_path}")

使用设备: cuda
CLIP模型已加载。
Attributes DataFrame columns: ['image_id', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive', 'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose', 'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows', 'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace', 'Wearing_Necktie', 'Young']
Attributes DataFrame head:
      image_id  5_o_Clock_Shadow  Arched_Eyebrows  Attractive  Bags_Under_Eyes  \
0  000001.jpg                -1                1           1               -1   
1  000002.jpg                -1               -1          -1                1   
2  000003.jpg                -1               -1          -1               -1   

Epoch 1/40:   0%|          | 1/1272 [00:01<27:06,  1.28s/it]


批次 0 详细信息:
  - 原始α=-2.0 Rényi值: -0.828302, 加权Rényi损失: -0.491811
  - 原始KL值: 0.000002
  - 重构损失: 36.082302, 总损失: 35.590494
  - MC样本均值: -0.0000, MC样本标准差: 0.0002
  - mu范围: [-0.0007, 0.0008], var范围: [0.999106, 1.000720]
  - 分布统计: |μ|平均=0.0001, σ²平均=1.0000, log(σ²)平均=0.0000
  - 潜变量z统计: 均值=0.0122, 标准差=1.0048, 范围=[-4.3088, 4.1498]


Epoch 1/40:   8%|▊         | 104/1272 [00:06<00:54, 21.47it/s]


批次 100 详细信息:
  - 原始α=-2.0 Rényi值: -0.828303, 加权Rényi损失: -0.491811
  - 原始KL值: 0.000002
  - 重构损失: 35.668842, 总损失: 35.177033
  - MC样本均值: 0.0000, MC样本标准差: 0.0002
  - mu范围: [-0.0008, 0.0008], var范围: [0.999035, 1.000777]
  - 分布统计: |μ|平均=0.0001, σ²平均=1.0000, log(σ²)平均=0.0000
  - 潜变量z统计: 均值=-0.0113, 标准差=1.0023, 范围=[-5.2066, 4.6553]


...


批次 1200 详细信息:
  - 原始α=-2.0 Rényi值: -1.380842, 加权Rényi损失: -3.582992
  - 原始KL值: 605.761475
  - 重构损失: 8.202943, 总损失: 4.619951
  - MC样本均值: -4.3063, MC样本标准差: 4.9015
  - mu范围: [-4.6993, 4.6932], var范围: [0.003484, 11.353487]
  - 分布统计: |μ|平均=2.6339, σ²平均=3.2806, log(σ²)平均=0.5545
  - 潜变量z统计: 均值=-0.3148, 标准差=3.2899, 范围=[-11.4545, 10.3453]


Epoch 40/40: 100%|██████████| 1272/1272 [01:00<00:00, 20.86it/s]


Epoch 40 训练统计:
  - 平均总损失: 4.7000
  - 重构损失: 8.2744
  - α=-2.0 Rényi损失: -3.5744 (原始值: -1.377526)
  - 原始KL值: 601.295894
  - MC样本均值: -4.2614, MC样本标准差: 4.8638
  - 学习率: 0.000008, NaN批次: 0


验证集统计:
  - 平均总损失: 4.5022
  - 重构损失: 8.0773
  - α=-2.0 Rényi损失: -3.5750 (原始值: -1.377771)
  - 原始KL值: 598.110227
  - MC样本均值: -4.2457, MC样本标准差: 4.8507
  - NaN批次: 0
已保存模型至: model_checkpoints/cvae_renyi_alpha-2.0_epoch_40.pth
保存最佳模型: model_checkpoints/cvae_renyi_alpha-2.0_best.pth (质量指标: 8.2839)
α=-2.0的Rényi散度CVAE模型已保存为: model_checkpoints/cvae_renyi_alpha-2.0_final.pth
训练过程中发生错误: name 'plot_enhanced_training_stats' is not defined
发生错误，模型紧急保存为: model_checkpoints/cvae_renyi_alpha-2.0_emergency_save.pth


Traceback (most recent call last):
  File "/tmp/ipykernel_17401/1448840501.py", line 819, in <module>
    plot_enhanced_training_stats(train_stats, val_stats,
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
NameError: name 'plot_enhanced_training_stats' is not defined
